In [ ]:
# reorganize_and_explore.py
from pathlib import Path
import pandas as pd
import numpy as np

base_dir = Path("~/repos/summerschool2023/projects/fall-detection/fall_detection_data").expanduser()
processed_dir = base_dir / "processed"  # Where your preprocessed data is
models_dir = base_dir / "models"        # Where to save trained models
models_dir.mkdir(exist_ok=True)
output_dir = models_dir  # Use this for saving mode

print("=" * 80)
print("CURRENT DIRECTORY STRUCTURE")
print("=" * 80)

# Show current structure
for item in sorted(base_dir.iterdir()):
    if item.is_dir():
        print(f"\n📁 {item.name}/")
        # Show what's inside each directory
        sub_items = list(item.iterdir())[:5]
        for sub in sub_items:
            if sub.is_dir():
                file_count = len(list(sub.glob("*")))
                print(f"   📁 {sub.name}/ ({file_count} files)")
            else:
                print(f"   📄 {sub.name}")
        if len(list(item.iterdir())) > 5:
            print(f"   ... and {len(list(item.iterdir())) - 5} more")

print("\n" + "=" * 80)
print("PROPOSED REORGANIZATION")
print("=" * 80)

proposed_structure = """
fall_detection_data/
├── KFall/
│   ├── sensor_data/
│   │   ├── SA06/
│   │   │   ├── S06T01R01.csv  (KFall format: S##T##R##.csv)
│   │   │   ├── S06T02R01.csv
│   │   │   └── ...
│   │   └── SA07/ ...
│   └── labels/
│       ├── SA06_label.xlsx
│       └── SA07_label.xlsx ...
│
├── SisFall/
│   ├── SA01/
│   │   ├── D01_SA01_R01.txt  (SisFall format: <CODE>_<SUBJECT>_<TRIAL>.txt)
│   │   ├── F01_SA01_R01.txt
│   │   └── ...
│   ├── SA02/ ...
│   └── SE01/ ... (elderly subjects)
│
└── processed/
    ├── kfall_features.pkl
    ├── sisfall_features.pkl
    └── fused_dataset.pkl
"""

print(proposed_structure)

print("\n" + "=" * 80)
print("DATASET COMPARISON")
print("=" * 80)

# KFall structure
kfall_sensor = base_dir / "KFall" / "sensor_data"
if kfall_sensor.exists():
    kfall_subjects = sorted([d.name for d in kfall_sensor.iterdir() if d.is_dir()])
    sample_kfall = kfall_sensor / kfall_subjects[0]
    sample_kfall_file = list(sample_kfall.glob("*.csv"))[0]
    
    df_kfall = pd.read_csv(sample_kfall_file)
    
    print("\n📊 KFALL DATASET:")
    print(f"   Subjects: {len(kfall_subjects)} (SA06-SA38)")
    print(f"   Sampling Rate: 100 Hz (needs upsampling to 200 Hz)")
    print(f"   File Format: S##T##R##.csv")
    print(f"   Columns: {df_kfall.columns.tolist()}")
    print(f"   Data Shape (sample): {df_kfall.shape}")
    print(f"   Has Labels: ✅ Yes (temporal annotations in Excel files)")

# SisFall structure
sisfall_dir = base_dir / "SisFall"
if sisfall_dir.exists():
    sisfall_subjects = sorted([d.name for d in sisfall_dir.iterdir() if d.is_dir()])
    adults = [s for s in sisfall_subjects if s.startswith('SA')]
    elderly = [s for s in sisfall_subjects if s.startswith('SE')]
    
    sample_sisfall = sisfall_dir / adults[0]
    sample_sisfall_file = list(sample_sisfall.glob("*.txt"))[0]
    
    # Read SisFall file - more robust parsing
    try:
        # Method 1: Read line by line and parse manually
        with open(sample_sisfall_file, 'r') as f:
            lines = f.readlines()
        
        data = []
        for line in lines:
            # Remove semicolon and split by comma or whitespace
            line = line.strip().replace(';', '')
            values = line.replace(',', ' ').split()
            if len(values) == 9:  # Should have 9 columns
                data.append([float(v) for v in values])
        
        df_sisfall = pd.DataFrame(data)
        
        print("\n📊 SISFALL DATASET:")
        print(f"   Subjects: {len(sisfall_subjects)} total")
        print(f"     - Adults (SA): {len(adults)} (SA01-SA23)")
        print(f"     - Elderly (SE): {len(elderly)} (SE01-SE15)")
        print(f"   Sampling Rate: 200 Hz ✅")
        print(f"   File Format: <CODE>_<SUBJECT>_<TRIAL>.txt")
        print(f"   Columns: 9 (ADXL345: 0-2, ITG3200: 3-5, MMA8451Q: 6-8)")
        print(f"   Data Shape (sample): {df_sisfall.shape}")
        print(f"   Has Labels: ❌ No (must use Algorithm 1)")
        print(f"   Data Format: Raw bits (needs conversion to physical units)")
        
    except Exception as e:
        print(f"\n❌ Error reading SisFall file: {e}")
        print("   Will handle this in the preprocessing pipeline")

print("\n" + "=" * 80)
print("ACTIVITIES NEEDED FOR PAPER REPRODUCTION")
print("=" * 80)

print("\n📋 FROM KFALL (Table I):")
kfall_needed = {
    'T10': 'Stumble while walking',
    'T28': 'Vertical fall while walking (fainting)',
    'T30': 'Forward fall while walking (trip)',
    'T31': 'Forward fall while jogging (trip)',
    'T32': 'Forward fall while walking (slip)',
    'T33': 'Lateral fall while walking (slip)',
    'T34': 'Backward fall while walking (slip)'
}
for code, desc in kfall_needed.items():
    print(f"   {code}: {desc}")

print("\n📋 FROM SISFALL (Table I):")
print("\n   ADL Activities:")
sisfall_adl = {
    'D01': 'Walking slowly',
    'D02': 'Walking quickly',
    'D03': 'Jogging slowly',
    'D04': 'Jogging quickly',
    'D05': 'Walking upstairs/downstairs slowly',
    'D06': 'Walking upstairs/downstairs quickly',
    'D18': 'Stumble while walking'
}
for code, desc in sisfall_adl.items():
    print(f"   {code}: {desc}")

print("\n   Fall Activities:")
sisfall_falls = {
    'F01': 'Fall forward while walking (slip)',
    'F02': 'Fall backward while walking (slip)',
    'F03': 'Lateral fall while walking (slip)',
    'F04': 'Fall forward while walking (trip)',
    'F05': 'Fall forward while jogging (trip)',
    'F06': 'Vertical fall while walking (fainting)'
}
for code, desc in sisfall_falls.items():
    print(f"   {code}: {desc}")

print("\n" + "=" * 80)
print("NEXT STEPS")
print("=" * 80)
print("""
1. ✅ Data is properly organized
2. ⏭️  Implement preprocessing pipeline:
   - Load and convert SisFall raw bits to physical units
   - Upsample KFall from 100Hz to 200Hz
   - Apply Algorithm 1 for temporal segmentation
   - Extract features according to Table I
3. ⏭️  Z-score normalization and dataset fusion
4. ⏭️  Build and train FallNet
""")

In [ ]:
# %% [markdown]
# # Fall Detection Data Preprocessing Pipeline - CORRECTED VERSION
# 
# This notebook implements the preprocessing methodology from the paper:
# "A novel Feature extraction method for Pre-Impact Fall detection system using Deep learning and wearable sensors"
#
# Key fixes:
# - Removed Sp - 3 bug
# - Fixed transitional window logic (no duplicates)
# - Proper ADL extraction before falls
# - Correct stumble/recovery processing

# %% [markdown]
## 1. Setup and Imports

# %%
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.interpolate import CubicSpline
from sklearn.preprocessing import StandardScaler
import pickle
import json
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Imports complete")

# %% [markdown]
## 2. Define Paths and Configuration

# %%
base_dir = Path("~/repos/summerschool2023/projects/fall-detection/fall_detection_data").expanduser()
kfall_sensor_dir = base_dir / "KFall" / "sensor_data"
kfall_labels_dir = base_dir / "KFall" / "label_data"
sisfall_dir = base_dir / "SisFall"
processed_dir = base_dir / "processed"

# Clean up processed directory
print("🧹 Cleaning processed directory...")
if processed_dir.exists():
    for f in processed_dir.glob("*"):
        if f.is_file():
            f.unlink()
            print(f"  Deleted: {f.name}")
else:
    processed_dir.mkdir(exist_ok=True)

print("\n✅ Directories configured:")
print(f"   KFall sensor data: {kfall_sensor_dir}")
print(f"   KFall labels: {kfall_labels_dir}")
print(f"   SisFall data: {sisfall_dir}")
print(f"   Output: {processed_dir}")

# %% [markdown]
## 3. Activity Mappings and Labels

# %%
# Activities from Table I in the paper
kfall_fall_activities = ['T28', 'T30', 'T31', 'T32', 'T33', 'T34']
kfall_stumble = ['T10']

sisfall_adl_map = {
    'D01': 'Walking', 'D02': 'Walking',
    'D03': 'Jogging', 'D04': 'Jogging',
    'D05': 'Walking_stairs_updown', 'D06': 'Walking_stairs_updown',
    'D18': 'Stumble_while_walking'
}

sisfall_falls = ['F01', 'F02', 'F03', 'F04', 'F05', 'F06']

# Label encoding (8-class classifier)
label_map = {
    'Walking': 0,
    'Jogging': 1,
    'Walking_stairs_updown': 2,
    'Stumble_while_walking': 3,
    'Fall_Recovery': 4,
    'Fall_Initiation': 5,
    'Impact': 6,
    'Aftermath': 7
}

reverse_label_map = {v: k for k, v in label_map.items()}

print("📋 Label Mapping:")
for label_name, label_id in label_map.items():
    print(f"   {label_id}: {label_name}")

# Save label map immediately
with open(processed_dir / "label_map.json", "w") as f:
    json.dump(label_map, f, indent=2)
print("\n✅ Label map saved")

# %% [markdown]
## 4. Data Loading Functions

# %%
def load_sisfall_file(filepath):
    """Load and convert SisFall file from bits to physical units"""
    with open(filepath, 'r') as f:
        lines = f.readlines()
    
    data = []
    for line in lines:
        line = line.strip().replace(';', '').replace(',', ' ')
        values = line.split()
        if len(values) == 9:
            data.append([float(v) for v in values])
    
    if len(data) == 0:
        return None
    
    data = np.array(data)
    
    # Convert to physical units
    converted = np.zeros((data.shape[0], 6))
    
    # ADXL345 (columns 0-2): ±16g, 13-bit
    adxl_factor = (2 * 16) / (2**13)
    converted[:, 0:3] = data[:, 0:3] * adxl_factor
    
    # ITG3200 (columns 3-5): ±2000°/s, 16-bit  
    itg_factor = (2 * 2000) / (2**16)
    converted[:, 3:6] = data[:, 3:6] * itg_factor
    
    return converted

def load_kfall_file(filepath):
    """Load KFall CSV file"""
    try:
        df = pd.read_csv(filepath)
        data = df[['AccX', 'AccY', 'AccZ', 'GyrX', 'GyrY', 'GyrZ']].values
        return data
    except:
        return None

print("✅ Data loading functions defined")

# %% [markdown]
## 5. Upsampling Function (KFall 100Hz → 200Hz)

# %%
def upsample_to_200hz(data, original_freq=100):
    """Upsample KFall data from 100Hz to 200Hz using cubic spline"""
    n_samples, n_features = data.shape
    original_time = np.arange(n_samples) / original_freq
    target_time = np.arange(0, n_samples / original_freq, 1 / 200)
    
    upsampled = np.zeros((len(target_time), n_features))
    for i in range(n_features):
        cs = CubicSpline(original_time, data[:, i])
        upsampled[:, i] = cs(target_time)
    
    return upsampled

print("✅ Upsampling function defined")

# %% [markdown]
## 6. Algorithm 1: Temporal Feature Extraction (CORRECTED)

# %%
def extract_temporal_features(data, sampling_freq=200):
    """
    Algorithm 1 from the paper: Automatic temporal feature extraction
    Uses Y-axis acceleration (gravity direction)
    
    FIXED: Removed Sp - 3 bug
    """
    acc_y = data[:, 1]  # Y-axis
    W_s = sampling_freq // 4  # 50 samples (0.25s)
    
    # Calculate std on non-overlapping windows
    std_devs = []
    window_positions = []
    for i in range(0, len(acc_y) - W_s, W_s):
        window = acc_y[i:i + W_s]
        std_devs.append(np.std(window))
        window_positions.append(i)
    
    if len(std_devs) == 0:
        return None
    
    # Find segmentation point
    # CRITICAL FIX: Don't subtract 3!
    max_std_idx = np.argmax(std_devs)
    Sp = max_std_idx  # Paper says: "The starting frame of Sw will become Sp"
    
    segments = {
        'std_devs': std_devs,
        'window_positions': window_positions,
        'Sp': Sp,
        'W_s': W_s,
        
        # Phase boundaries (in samples)
        'adl_end': Sp * W_s,
        'fall_init_start': Sp * W_s,
        'fall_init_end': min((Sp + 4) * W_s, len(data)),
        'transitional_end': min((Sp + 2) * W_s, len(data)),
        'impact_start': min((Sp + 4) * W_s, len(data)),
        'impact_end': min((Sp + 8) * W_s, len(data)),
        'aftermath_start': min((Sp + 8) * W_s, len(data)),
    }
    
    return segments

print("✅ Algorithm 1 implemented (CORRECTED)")

# %% [markdown]
## 7. Feature Extraction Functions (CORRECTED)

# %%
def process_fall_activity(data):
    """
    Extract features from fall activity using Algorithm 1
    
    FIXED:
    - No duplicate Fall_Initiation samples
    - Properly extracts ADL before fall
    - Uses transitional window for 50% of samples (random selection)
    """
    segments = extract_temporal_features(data)
    if segments is None:
        return []
    
    results = []
    W_s = segments['W_s']
    
    # 1. ADL phase (before fall) - if available
    adl_start = max(0, segments['fall_init_start'] - 200)
    adl_end = segments['fall_init_start']
    
    if adl_end - adl_start >= 200 and adl_start >= 0:
        adl_segment = data[adl_start:adl_end]
        
        # Determine ADL type based on variance
        acc_std = np.std(adl_segment[:, 1])
        if acc_std > 0.5:
            adl_label = label_map['Jogging']
        else:
            adl_label = label_map['Walking']
        
        results.append((adl_segment[:200], adl_label))
    
    # 2. Fall Initiation - ONE sample per fall
    # Randomly choose between transitional window (0.5s) or full window (1s)
    fi_start = segments['fall_init_start']
    
    if np.random.random() < 0.5:
        # Use transitional window (0.5s) for early detection training
        tw_end = segments['transitional_end']
        if tw_end <= len(data) and (tw_end - fi_start) >= 100:
            tw_segment = data[fi_start:tw_end]
            
            # Interpolate to 200 samples
            if len(tw_segment) != 200:
                time_orig = np.linspace(0, 1, len(tw_segment))
                time_new = np.linspace(0, 1, 200)
                tw_interp = np.zeros((200, 6))
                for i in range(6):
                    tw_interp[:, i] = np.interp(time_new, time_orig, tw_segment[:, i])
                results.append((tw_interp, label_map['Fall_Initiation']))
            else:
                results.append((tw_segment, label_map['Fall_Initiation']))
    else:
        # Use full Fall Initiation window (1s)
        fi_end = segments['fall_init_end']
        if fi_end <= len(data) and (fi_end - fi_start) >= 200:
            fi_segment = data[fi_start:fi_end]
            results.append((fi_segment[:200], label_map['Fall_Initiation']))
    
    # 3. Impact
    impact_start = segments['impact_start']
    impact_end = segments['impact_end']
    if impact_end <= len(data) and (impact_end - impact_start) >= 200:
        impact_segment = data[impact_start:impact_end]
        results.append((impact_segment[:200], label_map['Impact']))
    
    # 4. Aftermath
    aftermath_start = segments['aftermath_start']
    if len(data) - aftermath_start >= 200:
        aftermath_segment = data[aftermath_start:aftermath_start + 200]
        results.append((aftermath_segment, label_map['Aftermath']))
    
    return results


def process_stumble_activity(data):
    """
    Process stumble/fall recovery
    
    Stumble = temporary loss of balance WITHOUT falling (recovers)
    """
    segments = extract_temporal_features(data)
    if segments is None:
        return []
    
    results = []
    
    # Extract the "stumble" moment (the imbalance event)
    stumble_start = segments['fall_init_start']
    stumble_end = segments['transitional_end']
    
    if stumble_end <= len(data) and (stumble_end - stumble_start) >= 100:
        stumble_segment = data[stumble_start:stumble_end]
        
        # Interpolate to 200 samples if needed
        if len(stumble_segment) < 200:
            time_orig = np.linspace(0, 1, len(stumble_segment))
            time_new = np.linspace(0, 1, 200)
            stumble_interp = np.zeros((200, 6))
            for i in range(6):
                stumble_interp[:, i] = np.interp(time_new, time_orig, stumble_segment[:, i])
            results.append((stumble_interp, label_map['Stumble_while_walking']))
        else:
            results.append((stumble_segment[:200], label_map['Stumble_while_walking']))
    
    # Fall recovery - the recovery period after stumble
    recovery_start = segments['transitional_end']
    recovery_end = segments['impact_end']
    
    if recovery_end <= len(data) and (recovery_end - recovery_start) >= 200:
        recovery_segment = data[recovery_start:recovery_end]
        results.append((recovery_segment[:200], label_map['Fall_Recovery']))
    
    return results


def process_adl_activity(data, label_name):
    """
    Extract 1-second non-overlapping windows from ADL activities
    """
    results = []
    label = label_map[label_name]
    
    # Extract up to 20 seconds (as per paper)
    max_samples = min(len(data), 4000)  # 20 seconds at 200Hz
    
    # Non-overlapping 1-second windows
    for i in range(0, max_samples - 200, 200):
        segment = data[i:i + 200]
        if len(segment) == 200:
            results.append((segment, label))
    
    return results

print("✅ Feature extraction functions defined (CORRECTED)")

# %% [markdown]
## 8. Process KFall Dataset

# %%
def process_kfall_dataset():
    """Process all KFall data"""
    print("="*80)
    print("PROCESSING KFALL DATASET")
    print("="*80)
    
    X_data = []
    y_labels = []
    
    subjects = sorted([d for d in kfall_sensor_dir.iterdir() if d.is_dir()])
    print(f"Found {len(subjects)} subjects")
    
    for subject_dir in tqdm(subjects, desc="Processing KFall subjects"):
        files = list(subject_dir.glob("*.csv"))
        
        for file in files:
            # Extract activity code from filename: S06T10R01.csv -> T10
            filename = file.stem
            if len(filename) < 6:
                continue
            activity_code = filename[3:6]  # e.g., T10, T28
            
            # Load and upsample
            data = load_kfall_file(file)
            if data is None or len(data) < 100:
                continue
            
            try:
                data_upsampled = upsample_to_200hz(data)
                
                # Process based on activity type
                if activity_code in kfall_fall_activities:
                    features = process_fall_activity(data_upsampled)
                elif activity_code in kfall_stumble:
                    features = process_stumble_activity(data_upsampled)
                else:
                    continue
                
                for segment, label in features:
                    if segment.shape == (200, 6):
                        X_data.append(segment)
                        y_labels.append(label)
            except Exception as e:
                print(f"  Error processing {file.name}: {e}")
                continue
    
    return np.array(X_data), np.array(y_labels)

# Run KFall processing
print("\n" + "="*80)
X_kfall, y_kfall = process_kfall_dataset()
print(f"\n✅ KFall processed:")
print(f"   X shape: {X_kfall.shape}")
print(f"   y shape: {y_kfall.shape}")

# %% [markdown]
## 9. Process SisFall Dataset

# %%
def process_sisfall_dataset():
    """Process all SisFall data"""
    print("="*80)
    print("PROCESSING SISFALL DATASET")
    print("="*80)
    
    X_data = []
    y_labels = []
    
    subjects = sorted([d for d in sisfall_dir.iterdir() 
                      if d.is_dir() and (d.name.startswith('SA') or d.name.startswith('SE'))])
    print(f"Found {len(subjects)} subjects")
    
    for subject_dir in tqdm(subjects, desc="Processing SisFall subjects"):
        files = list(subject_dir.glob("*.txt"))
        
        for file in files:
            # Extract activity code: D01_SA01_R01.txt -> D01
            filename = file.stem
            parts = filename.split('_')
            if len(parts) < 2:
                continue
            activity_code = parts[0]
            
            # Load data
            data = load_sisfall_file(file)
            if data is None or len(data) < 200:
                continue
            
            try:
                # Process based on activity type
                if activity_code in sisfall_adl_map:
                    label_name = sisfall_adl_map[activity_code]
                    features = process_adl_activity(data, label_name)
                elif activity_code in sisfall_falls:
                    features = process_fall_activity(data)
                else:
                    continue
                
                for segment, label in features:
                    if segment.shape == (200, 6):
                        X_data.append(segment)
                        y_labels.append(label)
            except Exception as e:
                # Silently skip problematic files
                continue
    
    return np.array(X_data), np.array(y_labels)

# Run SisFall processing
print("\n" + "="*80)
X_sisfall, y_sisfall = process_sisfall_dataset()
print(f"\n✅ SisFall processed:")
print(f"   X shape: {X_sisfall.shape}")
print(f"   y shape: {y_sisfall.shape}")

# %% [markdown]
## 10. Z-Score Normalization and Dataset Fusion

# %%
def normalize_and_fuse(X_kfall, y_kfall, X_sisfall, y_sisfall):
    """
    Z-score normalization and dataset fusion (paper methodology)
    
    Paper says: "Z-score standardization was again employed before the 
    final data was fed into the network to normalize the features extracted 
    from the two datasets"
    """
    print("="*80)
    print("NORMALIZING AND FUSING DATASETS")
    print("="*80)
    
    # Reshape for normalization
    n_kfall, ts, feat = X_kfall.shape
    X_kfall_flat = X_kfall.reshape(-1, feat)
    
    n_sisfall = X_sisfall.shape[0]
    X_sisfall_flat = X_sisfall.reshape(-1, feat)
    
    print(f"\nStep 1: Normalize each dataset separately")
    # Normalize KFall
    scaler_kfall = StandardScaler()
    X_kfall_norm = scaler_kfall.fit_transform(X_kfall_flat)
    X_kfall_norm = X_kfall_norm.reshape(n_kfall, ts, feat)
    print(f"  KFall normalized: {X_kfall_norm.shape}")
    
    # Normalize SisFall
    scaler_sisfall = StandardScaler()
    X_sisfall_norm = scaler_sisfall.fit_transform(X_sisfall_flat)
    X_sisfall_norm = X_sisfall_norm.reshape(n_sisfall, ts, feat)
    print(f"  SisFall normalized: {X_sisfall_norm.shape}")
    
    print(f"\nStep 2: Fuse datasets")
    # Fuse
    X_fused = np.concatenate([X_kfall_norm, X_sisfall_norm], axis=0)
    y_fused = np.concatenate([y_kfall, y_sisfall], axis=0)
    print(f"  Fused dataset: {X_fused.shape}")
    
    print(f"\nStep 3: Normalize fused dataset")
    # Final normalization
    X_fused_flat = X_fused.reshape(-1, feat)
    scaler_final = StandardScaler()
    X_fused_norm = scaler_final.fit_transform(X_fused_flat)
    X_fused_norm = X_fused_norm.reshape(-1, ts, feat)
    print(f"  Final normalized: {X_fused_norm.shape}")
    
    # Verify normalization
    print(f"\nVerification:")
    print(f"  Mean: {X_fused_norm.mean():.6f} (should be ~0)")
    print(f"  Std:  {X_fused_norm.std():.6f} (should be ~1)")
    
    return X_fused_norm, y_fused, scaler_final

# Normalize and fuse
X_final, y_final, scaler = normalize_and_fuse(X_kfall, y_kfall, X_sisfall, y_sisfall)

print(f"\n✅ Final fused dataset:")
print(f"   X shape: {X_final.shape}")
print(f"   y shape: {y_final.shape}")

# %% [markdown]
## 11. Verify Data Quality

# %%
from collections import Counter

print("="*80)
print("DATA QUALITY VERIFICATION")
print("="*80)

# 1. Class distribution
counts = Counter(y_final)
print("\n1. Class Distribution:")
for cls_id in sorted(counts.keys()):
    count = counts[cls_id]
    pct = count / len(y_final) * 100
    print(f"   {cls_id}: {reverse_label_map[cls_id]:30s} - {count:5d} ({pct:5.2f}%)")

print(f"\n   Total samples: {len(y_final)}")

# 2. CRITICAL: Variance test
print("\n2. Variance Test (Acc-Y axis):")
print(f"   {'Class':<35s} {'Variance':<12s}")
print(f"   {'-'*50}")

variances = []
for cls_id in sorted(counts.keys()):
    class_samples = X_final[y_final == cls_id]
    var = class_samples[:, :, 1].var()
    variances.append((reverse_label_map[cls_id], var))
    print(f"   {reverse_label_map[cls_id]:<35s} {var:>10.4f}")

# Sort by variance
variances_sorted = sorted(variances, key=lambda x: x[1], reverse=True)
print(f"\n3. Variance Ranking:")
for i, (name, var) in enumerate(variances_sorted, 1):
    print(f"   {i}. {name:<35s}: {var:.4f}")

# Check if Fall_Initiation is in top 2
fall_init_rank = next(i for i, (name, _) in enumerate(variances_sorted, 1) if name == 'Fall_Initiation')

if fall_init_rank <= 2:
    print(f"\n✅ PASS: Fall_Initiation ranked #{fall_init_rank} (should be #1 or #2)")
else:
    print(f"\n❌ FAIL: Fall_Initiation ranked #{fall_init_rank} (should be #1 or #2)")
    print("   Segmentation may be incorrect!")

# 4. Check for NaN/Inf
print(f"\n4. Data Integrity:")
print(f"   NaN values: {np.isnan(X_final).sum()}")
print(f"   Inf values: {np.isinf(X_final).sum()}")

# 5. Shape verification
print(f"\n5. Shape Verification:")
print(f"   Expected: (N, 200, 6)")
print(f"   Actual:   {X_final.shape}")
print(f"   ✅ PASS" if X_final.shape[1:] == (200, 6) else "   ❌ FAIL")

# %% [markdown]
## 12. Visualize Samples

# %%
fig, axes = plt.subplots(4, 2, figsize=(16, 14))
axes = axes.flatten()

for class_id in range(8):
    class_indices = np.where(y_final == class_id)[0]
    if len(class_indices) > 0:
        sample_idx = class_indices[0]
        sample_data = X_final[sample_idx]
        
        time = np.arange(200) / 200
        axes[class_id].plot(time, sample_data[:, 0], label='Acc-X', alpha=0.7, linewidth=1)
        axes[class_id].plot(time, sample_data[:, 1], label='Acc-Y', alpha=0.7, linewidth=1)
        axes[class_id].plot(time, sample_data[:, 2], label='Acc-Z', alpha=0.7, linewidth=1)
        
        axes[class_id].set_title(f'Class {class_id}: {reverse_label_map[class_id]}', 
                                fontsize=11, fontweight='bold')
        axes[class_id].set_xlabel('Time (s)')
        axes[class_id].set_ylabel('Normalized Acc')
        axes[class_id].legend(loc='upper right', fontsize=8)
        axes[class_id].grid(True, alpha=0.3)

plt.suptitle('Sample Segments from Each Activity Class', 
             fontsize=14, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig(processed_dir / 'sample_visualization.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✅ Visualization saved to {processed_dir / 'sample_visualization.png'}")

# %% [markdown]
## 13. Save Processed Data

# %%
print("="*80)
print("SAVING PROCESSED DATA")
print("="*80)

# Save arrays
np.save(processed_dir / "X_data.npy", X_final)
np.save(processed_dir / "y_labels.npy", y_final)

# Save scaler
with open(processed_dir / "scaler.pkl", 'wb') as f:
    pickle.dump(scaler, f)

# Save label map (both formats)
with open(processed_dir / "label_map.pkl", 'wb') as f:
    pickle.dump(label_map, f)

with open(processed_dir / "label_map.json", 'w') as f:
    json.dump(label_map, f, indent=2)

print(f"\n✅ Saved to {processed_dir}/")
print(f"   📄 X_data.npy: {X_final.shape}")
print(f"   📄 y_labels.npy: {y_final.shape}")
print(f"   📄 scaler.pkl")
print(f"   📄 label_map.pkl")
print(f"   📄 label_map.json")

# %% [markdown]
## 14. Final Summary

# %%
print("\n" + "="*80)
print("PREPROCESSING PIPELINE COMPLETE")
print("="*80)

summary = f"""
✅ Successfully processed {len(y_final):,} samples

Dataset Breakdown:
  - KFall samples: {len(y_kfall):,}
  - SisFall samples: {len(y_sisfall):,}
  
Data Shape:
  - Features: {X_final.shape}
  - Labels: {y_final.shape}
  
Normalization:
  - Mean: {X_final.mean():.6f}
  - Std: {X_final.std():.6f}
  
Quality Check:
  - Fall_Initiation rank: #{fall_init_rank} (should be ≤2)
  - Status: {'✅ READY FOR TRAINING' if fall_init_rank <= 2 else '❌ NEEDS REVIEW'}

Next Steps:
  1. Load data with: X = np.load('processed/X_data.npy')
  2. Train FallNet model
  3. Evaluate on stratified K-fold cross-validation
"""

print(summary)

# %%

In [ ]:
# Check what files exist in your processed directory
print("="*80)
print("CHECKING PROCESSED DATA FILES")
print("="*80)

processed_files = list(output_dir.glob("*"))
print(f"\nFiles in {output_dir}:")
for f in sorted(processed_files):
    if f.is_file():
        size_mb = f.stat().st_size / (1024 * 1024)
        modified = pd.Timestamp(f.stat().st_mtime, unit='s')
        print(f"  {f.name:30s} | {size_mb:8.2f} MB | Modified: {modified}")

# Check if the data you loaded is actually the new one
print("\n" + "="*80)
print("VERIFY DATA FRESHNESS")
print("="*80)

X_data_path = output_dir / "X_data.npy"
y_labels_path = output_dir / "y_labels.npy"

if X_data_path.exists():
    mod_time = pd.Timestamp(X_data_path.stat().st_mtime, unit='s')
    print(f"\nX_data.npy was last modified: {mod_time}")
    print(f"Current time: {pd.Timestamp.now()}")
    age_minutes = (pd.Timestamp.now() - mod_time).total_seconds() / 60
    print(f"Age: {age_minutes:.1f} minutes ago")
    
    if age_minutes > 30:
        print("\n⚠️  WARNING: Data is more than 30 minutes old!")
        print("   You might be using old preprocessed data.")

In [ ]:
# Load the newly processed data
X_data = np.load(processed_dir / "X_data.npy")
y_labels = np.load(processed_dir / "y_labels.npy")

# ============================================================================
# STEP 1: Merge Impact and Aftermath
# ============================================================================
print("Merging Impact and Aftermath classes...")
y_labels[y_labels == 7] = 6  # Change Aftermath (7) to Impact (6)

# ============================================================================
# STEP 2: Remove Fall_Recovery (NEW!)
# ============================================================================
print("\n" + "="*80)
print("REMOVING FALL_RECOVERY CLASS")
print("="*80)

from collections import Counter

# Show before
counts_before = Counter(y_labels)
print(f"\nBefore removal:")
print(f"  Total samples: {len(y_labels):,}")
print(f"  Fall_Recovery (class 4): {counts_before[4]} samples")

# Remove Fall_Recovery (class 4)
mask = y_labels != 4
X_data = X_data[mask]
y_labels_temp = y_labels[mask]

removed_count = (~mask).sum()
print(f"\n✅ Removed {removed_count} Fall_Recovery samples")

# Shift labels down (5→4, 6→5)
y_labels = y_labels_temp.copy()
y_labels[y_labels_temp > 4] -= 1  # Classes 5,6 become 4,5

print(f"\nAfter removal:")
print(f"  Total samples: {len(y_labels):,}")
print(f"  Removed: {removed_count} samples ({removed_count/(len(y_labels)+removed_count)*100:.2f}%)")

# ============================================================================
# STEP 3: Update label map (NOW 6 CLASSES: 0-5)
# ============================================================================
label_map = {
    'Walking': 0,
    'Jogging': 1,
    'Walking_stairs_updown': 2,
    'Stumble_while_walking': 3,
    'Fall_Initiation': 4,      # Was 5, now 4 ← SHIFTED DOWN!
    'Impact_Aftermath': 5,     # Was 6, now 5 ← SHIFTED DOWN!
}
reverse_label_map = {v: k for k, v in label_map.items()}

print(f"\n✅ Updated to 6 classes (0-5):")
for name, idx in sorted(label_map.items(), key=lambda x: x[1]):
    print(f"  Class {idx}: {name}")

y_categorical = keras.utils.to_categorical(y_labels, num_classes=6)  # ← HERE!
print(f"y_categorical shape: {y_categorical.shape}")

# ============================================================================
# DIAGNOSTICS
# ============================================================================
print("\n" + "="*80)
print("POST-REMOVAL DATA DIAGNOSTICS")
print("="*80)

# 1. Class distribution
class_counts = Counter(y_labels)
print("\n1. Class Distribution (6 classes):")
for cls_idx in sorted(class_counts.keys()):
    count = class_counts[cls_idx]
    pct = count / len(y_labels) * 100
    print(f"   Class {cls_idx} ({reverse_label_map[cls_idx]:30s}): {count:5d} ({pct:5.2f}%)")

# Calculate imbalance
max_count = max(class_counts.values())
min_count = min(class_counts.values())
print(f"\nImbalance ratio: {max_count/min_count:.2f}x (was 36.8x with Fall_Recovery)")

# 2. Per-class signal statistics
print("\n2. Per-Class Signal Statistics (Acc-Y axis):")
print(f"   {'Class':<35s} {'Mean':<10s} {'Std':<10s} {'Min':<10s} {'Max':<10s}")
print(f"   {'-'*75}")
for cls_idx in sorted(class_counts.keys()):
    class_samples = X_data[y_labels == cls_idx]
    acc_y = class_samples[:, :, 1]  # Y-axis acceleration
    
    mean_val = acc_y.mean()
    std_val = acc_y.std()
    min_val = acc_y.min()
    max_val = acc_y.max()
    
    print(f"   {reverse_label_map[cls_idx]:<35s} {mean_val:>8.4f}  {std_val:>8.4f}  {min_val:>8.2f}  {max_val:>8.2f}")

# 3. Variance ranking
print("\n3. Variance Ranking (Fall_Initiation should be #1):")
variances = []
for cls_idx in sorted(class_counts.keys()):
    class_samples = X_data[y_labels == cls_idx]
    acc_y_var = class_samples[:, :, 1].var()
    variances.append((reverse_label_map[cls_idx], acc_y_var, cls_idx))
variances.sort(key=lambda x: x[1], reverse=True)
for i, (name, var, idx) in enumerate(variances, 1):
    print(f"   {i}. {name:<35s}: {var:.4f}")

# 4. Visualize samples (update to 6 classes)
fig, axes = plt.subplots(3, 2, figsize=(15, 10))
axes = axes.flatten()
critical_classes = [
    label_map['Walking'],
    label_map['Fall_Initiation'],
    label_map['Impact_Aftermath'],
    label_map['Stumble_while_walking'],
    label_map['Jogging'],
    label_map['Walking_stairs_updown']
]
for i, cls_idx in enumerate(critical_classes):
    if cls_idx in class_counts:
        sample_idx = np.where(y_labels == cls_idx)[0][0]
        sample_data = X_data[sample_idx]
        
        time = np.arange(200) / 200
        axes[i].plot(time, sample_data[:, 0], label='Acc-X', alpha=0.7, linewidth=1)
        axes[i].plot(time, sample_data[:, 1], label='Acc-Y', alpha=0.7, linewidth=1)
        axes[i].plot(time, sample_data[:, 2], label='Acc-Z', alpha=0.7, linewidth=1)
        
        axes[i].set_title(f'{reverse_label_map[cls_idx]}', fontsize=11, fontweight='bold')
        axes[i].set_xlabel('Time (s)')
        axes[i].set_ylabel('Normalized Acc')
        axes[i].legend(fontsize=8)
        axes[i].grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("✅ DATA READY FOR TRAINING (6 CLASSES)")
print("="*80)

In [ ]:
# %% [markdown]
# # Data Loading and Preprocessing
# Load preprocessed data, merge classes, remove Fall_Recovery

# %%
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from pathlib import Path
from tensorflow import keras

# Setup paths
base_dir = Path("~/repos/summerschool2023/projects/fall-detection/fall_detection_data").expanduser()
processed_dir = base_dir / "processed"
output_dir = base_dir / "models"
output_dir.mkdir(exist_ok=True)

print("="*80)
print("LOADING AND PREPROCESSING DATA")
print("="*80)

# %% Load the newly processed data
X_data = np.load(processed_dir / "X_data.npy")
y_labels = np.load(processed_dir / "y_labels.npy")

print(f"\nOriginal data loaded:")
print(f"  X_data shape: {X_data.shape}")
print(f"  y_labels shape: {y_labels.shape}")

# ============================================================================
# STEP 1: Merge Impact and Aftermath
# ============================================================================
print("\n" + "="*80)
print("STEP 1: MERGING IMPACT AND AFTERMATH")
print("="*80)

counts_before_merge = Counter(y_labels)
print(f"Before merge:")
print(f"  Impact (6): {counts_before_merge[6]} samples")
print(f"  Aftermath (7): {counts_before_merge[7]} samples")

y_labels[y_labels == 7] = 6  # Change Aftermath (7) to Impact (6)

counts_after_merge = Counter(y_labels)
print(f"\nAfter merge:")
print(f"  Impact_Aftermath (6): {counts_after_merge[6]} samples")

# ============================================================================
# STEP 2: Remove Fall_Recovery
# ============================================================================
print("\n" + "="*80)
print("STEP 2: REMOVING FALL_RECOVERY CLASS")
print("="*80)

# Show before
counts_before = Counter(y_labels)
print(f"\nBefore removal:")
print(f"  Total samples: {len(y_labels):,}")
print(f"  Fall_Recovery (class 4): {counts_before[4]} samples")

# Remove Fall_Recovery (class 4)
mask = y_labels != 4
X_data = X_data[mask]
y_labels_temp = y_labels[mask]

removed_count = (~mask).sum()
print(f"\n✅ Removed {removed_count} Fall_Recovery samples")

# Shift labels down (5→4, 6→5)
y_labels = y_labels_temp.copy()
y_labels[y_labels_temp > 4] -= 1  # Classes 5,6 become 4,5

print(f"\nAfter removal:")
print(f"  Total samples: {len(y_labels):,}")
print(f"  Removed: {removed_count} samples ({removed_count/(len(y_labels)+removed_count)*100:.2f}%)")

# ============================================================================
# STEP 3: Update label map (NOW 6 CLASSES: 0-5)
# ============================================================================
print("\n" + "="*80)
print("STEP 3: CREATING 6-CLASS LABEL MAP")
print("="*80)

label_map = {
    'Walking': 0,
    'Jogging': 1,
    'Walking_stairs_updown': 2,
    'Stumble_while_walking': 3,
    'Fall_Initiation': 4,      # Was 5, now 4
    'Impact_Aftermath': 5,     # Was 6, now 5
}
reverse_label_map = {v: k for k, v in label_map.items()}

print(f"\n✅ Updated to 6 classes (0-5):")
for name, idx in sorted(label_map.items(), key=lambda x: x[1]):
    print(f"  Class {idx}: {name}")

# ============================================================================
# STEP 4: Create categorical labels
# ============================================================================
print("\n" + "="*80)
print("STEP 4: CREATING CATEGORICAL LABELS")
print("="*80)

y_categorical = keras.utils.to_categorical(y_labels, num_classes=6)

print(f"\n✅ y_categorical created:")
print(f"   Shape: {y_categorical.shape}")
print(f"   Expected: ({len(y_labels)}, 6)")

# Verification
assert X_data.shape[0] == y_labels.shape[0] == y_categorical.shape[0], \
    "Data shapes don't match!"

print(f"\n✅ All data aligned:")
print(f"   X_data:        {X_data.shape}")
print(f"   y_labels:      {y_labels.shape}")
print(f"   y_categorical: {y_categorical.shape}")
print(f"   All have {X_data.shape[0]:,} samples")

# ============================================================================
# DIAGNOSTICS
# ============================================================================
print("\n" + "="*80)
print("DATA QUALITY DIAGNOSTICS")
print("="*80)

# 1. Class distribution
class_counts = Counter(y_labels)
print("\n1. Class Distribution (6 classes):")
for cls_idx in sorted(class_counts.keys()):
    count = class_counts[cls_idx]
    pct = count / len(y_labels) * 100
    print(f"   Class {cls_idx} ({reverse_label_map[cls_idx]:30s}): {count:5d} ({pct:5.2f}%)")

# Calculate imbalance
max_count = max(class_counts.values())
min_count = min(class_counts.values())
print(f"\nImbalance ratio: {max_count/min_count:.2f}x")
print(f"  (was 36.8x with Fall_Recovery)")

# 2. Per-class signal statistics
print("\n2. Per-Class Signal Statistics (Acc-Y axis):")
print(f"   {'Class':<35s} {'Mean':<10s} {'Std':<10s} {'Min':<10s} {'Max':<10s}")
print(f"   {'-'*75}")
for cls_idx in sorted(class_counts.keys()):
    class_samples = X_data[y_labels == cls_idx]
    acc_y = class_samples[:, :, 1]  # Y-axis acceleration
    
    mean_val = acc_y.mean()
    std_val = acc_y.std()
    min_val = acc_y.min()
    max_val = acc_y.max()
    
    print(f"   {reverse_label_map[cls_idx]:<35s} {mean_val:>8.4f}  {std_val:>8.4f}  {min_val:>8.2f}  {max_val:>8.2f}")

# 3. Variance ranking
print("\n3. Variance Ranking (Fall_Initiation should be #1):")
variances = []
for cls_idx in sorted(class_counts.keys()):
    class_samples = X_data[y_labels == cls_idx]
    acc_y_var = class_samples[:, :, 1].var()
    variances.append((reverse_label_map[cls_idx], acc_y_var, cls_idx))
variances.sort(key=lambda x: x[1], reverse=True)
for i, (name, var, idx) in enumerate(variances, 1):
    print(f"   {i}. {name:<35s}: {var:.4f}")

# 4. Visualize samples
fig, axes = plt.subplots(3, 2, figsize=(15, 10))
axes = axes.flatten()
critical_classes = [
    label_map['Walking'],
    label_map['Fall_Initiation'],
    label_map['Impact_Aftermath'],
    label_map['Stumble_while_walking'],
    label_map['Jogging'],
    label_map['Walking_stairs_updown']
]

for i, cls_idx in enumerate(critical_classes):
    if cls_idx in class_counts:
        sample_idx = np.where(y_labels == cls_idx)[0][0]
        sample_data = X_data[sample_idx]
        
        time = np.arange(200) / 200
        axes[i].plot(time, sample_data[:, 0], label='Acc-X', alpha=0.7, linewidth=1)
        axes[i].plot(time, sample_data[:, 1], label='Acc-Y', alpha=0.7, linewidth=1)
        axes[i].plot(time, sample_data[:, 2], label='Acc-Z', alpha=0.7, linewidth=1)
        
        axes[i].set_title(f'{reverse_label_map[cls_idx]}', fontsize=11, fontweight='bold')
        axes[i].set_xlabel('Time (s)')
        axes[i].set_ylabel('Normalized Acc')
        axes[i].legend(fontsize=8)
        axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("✅ DATA READY FOR TRAINING (6 CLASSES)")
print("="*80)

In [ ]:
# %% [markdown]
# # FallNet Model Architecture
# 
# Implementing the CNN-LSTM ensemble model from Table II in the paper

# %%
import gc
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

# %% [markdown]
## 1. Model Architecture Definition

# %%
class FallNet:
    """
    FallNet: CNN-LSTM Ensemble for Pre-Impact Fall Detection
    Based on Table II in the paper
    """
    
    def __init__(self, input_shape=(200, 6), n_classes=6):
        """
        Args:
            input_shape: (timesteps, features) = (200, 6)
            n_classes: Number of output classes (6)
        """
        self.input_shape = input_shape
        self.n_classes = n_classes
        self.model = None
    
    def build_lstm_branch(self, inputs):
        """
        LSTM Branch:
        - LSTM(256, tanh)
        - Dense(128, ReLU)
        - Dense(64, ReLU)
        - Dense(32, ReLU)
        - Dense(8, Softmax)
        """
        # LSTM layer
        x = layers.LSTM(
            units=256,
            activation='tanh',
            return_sequences=False,
            name='lstm_layer'
        )(inputs)
        
        # Dense layers with dropout
        x = layers.Dense(128, activation='relu', name='lstm_dense1')(x)
        x = layers.Dropout(0.2, name='lstm_dropout1')(x)
        
        x = layers.Dense(64, activation='relu', name='lstm_dense2')(x)
        x = layers.Dropout(0.2, name='lstm_dropout2')(x)
        
        x = layers.Dense(32, activation='relu', name='lstm_dense3')(x)
        x = layers.Dropout(0.2, name='lstm_dropout3')(x)
        
        # Output layer
        lstm_output = layers.Dense(
            self.n_classes, 
            activation='softmax',
            name='lstm_output'
        )(x)
        
        return lstm_output
    
    def build_cnn_branch(self, inputs):
        """
        CNN Branch:
        - Conv1D(128 filters, kernel_size=3, ReLU)
        - MaxPooling1D(pool_size=2)
        - Dense(1024, ReLU)
        - Dense(512, ReLU)
        - Dense(8, Softmax)
        """
        # 1D Convolutional layer
        x = layers.Conv1D(
            filters=128,
            kernel_size=3,
            activation='relu',
            padding='same',
            name='conv1d_layer'
        )(inputs)
        
        # Max pooling
        x = layers.MaxPooling1D(
            pool_size=2,
            name='maxpool_layer'
        )(x)
        # Flatten for dense layers
        x = layers.Flatten(name='flatten_layer')(x)
        
        # Dense layers with dropout
        x = layers.Dense(1024, activation='relu', name='cnn_dense1')(x)
        x = layers.Dropout(0.2, name='cnn_dropout1')(x)
        
        x = layers.Dense(512, activation='relu', name='cnn_dense2')(x)
        x = layers.Dropout(0.2, name='cnn_dropout2')(x)
        
        # Output layer
        cnn_output = layers.Dense(
            self.n_classes,
            activation='softmax',
            name='cnn_output'
        )(x)
        
        return cnn_output
    
    def build_ensemble(self):
        """
        Build the complete ensemble model
        Combines LSTM and CNN branches
        """
        # Input layer
        inputs = layers.Input(shape=self.input_shape, name='input')
        
        # Build both branches
        lstm_output = self.build_lstm_branch(inputs)
        cnn_output = self.build_cnn_branch(inputs)
        
        # Ensemble: Average the predictions from both branches
        ensemble_output = layers.Average(name='ensemble_average')([lstm_output, cnn_output])
        
        # Create model
        self.model = models.Model(
            inputs=inputs,
            outputs=ensemble_output,
            name='FallNet'
        )
        
        return self.model
    
    def compile_model(self, learning_rate=None):
        """
        Compile the model with Adam optimizer and categorical crossentropy
        """
        if self.model is None:
            raise ValueError("Model not built yet. Call build_ensemble() first.")
        
        # Use default Adam learning rate if not specified
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate) if learning_rate else keras.optimizers.Adam()
        
        self.model.compile(
            optimizer=optimizer,
            loss='categorical_crossentropy',
            metrics=['accuracy', 
                    keras.metrics.Precision(name='precision'),
                    keras.metrics.Recall(name='recall')]
        )
        
        return self.model
    
    def get_model(self):
        """Return the compiled model"""
        return self.model

print("✅ FallNet class defined")

# %% [markdown]
## 2. Build and Visualize the Model

# %%
# Create FallNet instance
fallnet = FallNet(input_shape=(200, 6), n_classes=7)

# Build the ensemble
model = fallnet.build_ensemble()

# Compile the model
model = fallnet.compile_model()

print("\n" + "="*80)
print("FALLNET MODEL ARCHITECTURE")
print("="*80)
model.summary()

# %% [markdown]
## 3. Visualize Model Architecture

# %%
# Plot model architecture
try:
    keras.utils.plot_model(
        model,
        to_file=output_dir / 'fallnet_architecture.png',
        show_shapes=True,
        show_layer_names=True,
        rankdir='TB',  # Top to Bottom
        expand_nested=True,
        dpi=96
    )
    
    from IPython.display import Image, display
    display(Image(filename=str(output_dir / 'fallnet_architecture.png')))
    print(f"\n✅ Model architecture saved to {output_dir / 'fallnet_architecture.png'}")
except Exception as e:
    print(f"Could not plot model (requires graphviz): {e}")
    print("Install with: pip install pydot graphviz")

# %% [markdown]
## 4. Count Model Parameters  

# %%
def count_parameters(model):
    """Count trainable and non-trainable parameters"""
    trainable = np.sum([np.prod(v.shape) for v in model.trainable_weights])
    non_trainable = np.sum([np.prod(v.shape) for v in model.non_trainable_weights])
    return trainable, non_trainable

trainable, non_trainable = count_parameters(model)

print("\n" + "="*80)
print("MODEL PARAMETERS")
print("="*80)
print(f"Trainable parameters:     {trainable:,}")
print(f"Non-trainable parameters: {non_trainable:,}")
print(f"Total parameters:         {trainable + non_trainable:,}")

# %% [markdown]
## 5. Define Training Configuration

# %%
# Training hyperparameters from Table II
BATCH_SIZE = 512
EPOCHS = 200
LEARNING_RATE = None  # Use default Adam learning rate


fold_callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=10,
        min_lr=1e-7,
        verbose=1
    ),
    ModelCheckpoint(
            filepath=str(output_dir / f'fallnet_fold_{fold}.keras'),  # ← NOW fold exists!
            monitor='val_accuracy',
            save_best_only=True,
            mode='max',
            verbose=1
    )
        
]


print("✅ Training configuration set:")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Epochs: {EPOCHS}")
print(f"   Callbacks: Early Stopping, ReduceLROnPlateau, ModelCheckpoint")

# %% [markdown]
## 6. Prepare Data for Training

# %%
# Load the preprocessed data
X_data = np.load(processed_dir / "X_data.npy")
y_labels = np.load(processed_dir / "y_labels.npy")

print(f"Loaded data:")
print(f"  X shape: {X_data.shape}")
print(f"  y shape: {y_labels.shape}")

# Convert labels to categorical (one-hot encoding)
from tensorflow.keras.utils import to_categorical

#_categorical = to_categorical(y_labels, num_classes=7)
print(f"  y_categorical shape: {y_categorical.shape}")

# %% [markdown]
## 7. Stratified K-Fold Cross-Validation Setup

# %%
from sklearn.model_selection import StratifiedKFold

# Stratified K-Fold (K=5) as in the paper
K_FOLDS = 5
skf = StratifiedKFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

print(f"\n✅ Stratified {K_FOLDS}-Fold Cross-Validation configured")
print(f"   Total samples: {len(X_data):,}")
print(f"   Samples per fold (approx): {len(X_data) // K_FOLDS:,}")

# %% [markdown]
## 8. Training Loop with K-Fold CV

# %%
# Store results for each fold
fold_results = []
fold_histories = []

print("\n" + "="*80)
print("STARTING K-FOLD CROSS-VALIDATION TRAINING")
print("="*80)

for fold, (train_idx, val_idx) in enumerate(skf.split(X_data, y_labels), 1):
    print(f"\n{'='*80}")
    print(f"FOLD {fold}/{K_FOLDS}")
    print(f"{'='*80}")
    
    # Split data
    X_train, X_val = X_data[train_idx], X_data[val_idx]
    y_train, y_val = y_categorical[train_idx], y_categorical[val_idx]
    
    print(f"Train set: {X_train.shape[0]:,} samples")
    print(f"Val set:   {X_val.shape[0]:,} samples")
    
    # Build a fresh model for this fold
    fallnet_fold = FallNet(input_shape=(200, 6), n_classes=6)
    model_fold = fallnet_fold.build_ensemble()
    model_fold = fallnet_fold.compile_model()
    
    # Train the model
    print(f"\nTraining fold {fold}...")
    history = model_fold.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        callbacks=fold_callbacks,
        verbose=1
    )
    
    # Evaluate on validation set
    val_loss, val_acc, val_precision, val_recall = model_fold.evaluate(X_val, y_val, verbose=0)
    
    # Calculate F1-score
    val_f1 = 2 * (val_precision * val_recall) / (val_precision + val_recall) if (val_precision + val_recall) > 0 else 0
    
    print(f"\n{'='*40}")
    print(f"Fold {fold} Results:")
    print(f"{'='*40}")
    print(f"Validation Loss:      {val_loss:.4f}")
    print(f"Validation Accuracy:  {val_acc:.4f}")
    print(f"Validation Precision: {val_precision:.4f}")
    print(f"Validation Recall:    {val_recall:.4f}")
    print(f"Validation F1-Score:  {val_f1:.4f}")
    
    # Store results
    fold_results.append({
        'fold': fold,
        'val_loss': val_loss,
        'val_accuracy': val_acc,
        'val_precision': val_precision,
        'val_recall': val_recall,
        'val_f1': val_f1
    })
    
    fold_histories.append(history.history)
    
    # Save model for this fold
    model_fold.save(output_dir / f'fallnet_fold_{fold}.keras')
    print(f"\n✅ Model saved: fallnet_fold_{fold}.keras")

print("\n" + "="*80)
print("K-FOLD CROSS-VALIDATION COMPLETE")
print("="*80)

# %% [markdown]
## 9. Aggregate Results Across Folds

# %%
# Convert to DataFrame
results_df = pd.DataFrame(fold_results)

print("\n" + "="*80)
print("RESULTS ACROSS ALL FOLDS")
print("="*80)
print(results_df.to_string(index=False))

print("\n" + "="*80)
print("AVERAGE PERFORMANCE")
print("="*80)
mean_results = results_df.mean(numeric_only=True)
std_results = results_df.std(numeric_only=True)

for metric in ['val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_f1']:
    print(f"{metric:20s}: {mean_results[metric]:.4f} ± {std_results[metric]:.4f}")

# %% [markdown]
## 10. Visualize Training History

# %%
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

metrics = [
    ('loss', 'Loss'),
    ('accuracy', 'Accuracy'),
    ('precision', 'Precision'),
    ('recall', 'Recall')
]

for idx, (metric, title) in enumerate(metrics):
    ax = axes[idx // 2, idx % 2]
    
    for fold, history in enumerate(fold_histories, 1):
        epochs = range(1, len(history[metric]) + 1)
        ax.plot(epochs, history[metric], label=f'Fold {fold} Train', alpha=0.6)
        ax.plot(epochs, history[f'val_{metric}'], label=f'Fold {fold} Val', linestyle='--', alpha=0.6)
    
    ax.set_title(f'{title} Across All Folds', fontsize=12, fontweight='bold')
    ax.set_xlabel('Epoch')
    ax.set_ylabel(title)
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'training_history.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✅ Training history saved to {output_dir / 'training_history.png'}")

# %% [markdown]
## 11. Compare with Paper Results

# %%
print("\n" + "="*80)
print("COMPARISON WITH PAPER RESULTS")
print("="*80)

paper_results = {
    'Accuracy': 0.9752,
    'Precision (avg)': 0.9753,
    'Recall (avg)': 0.9752,
    'F1-score (avg)': 0.9750,
    'Fall_Initiation Recall': 0.9924,
    'Fall_Initiation F1': 0.9879
}

our_results = {
    'Accuracy': mean_results['val_accuracy'],
    'Precision (avg)': mean_results['val_precision'],
    'Recall (avg)': mean_results['val_recall'],
    'F1-score (avg)': mean_results['val_f1']
}

comparison_df = pd.DataFrame({
    'Metric': list(paper_results.keys())[:4],
    'Paper': [paper_results[k] for k in list(paper_results.keys())[:4]],
    'Our Implementation': [our_results[k] for k in our_results.keys()]
})

print(comparison_df.to_string(index=False))

print("\n📝 Note: For detailed class-wise metrics (especially Fall_Initiation),")
print("   we need to generate a confusion matrix and classification report.")

# %%

In [ ]:
# %% [markdown]
# # FallNet Training Pipeline
# CNN-LSTM ensemble for fall detection with 6 classes

# %%
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
import warnings
import gc
warnings.filterwarnings('ignore')
keras.backend.clear_session()
gc.collect()
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

# %% [markdown]
## 1. FallNet Model Architecture

# %%
class FallNet:
    """
    FallNet: CNN-LSTM Ensemble for Pre-Impact Fall Detection
    """
    
    def __init__(self, input_shape=(200, 6), n_classes=6):
        """
        Args:
            input_shape: (timesteps, features) = (200, 6)
            n_classes: Number of output classes (6)
Skip to Main
02_DataExplorationBothDataSets
Last Checkpoint: 5 days ago
[Python 3 (ipykernel)]

import tensorflow as tf
plt.tight_layout()
plt.savefig(output_dir / 'training_history.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✅ Training history saved to {output_dir / 'training_history.png'}")

# %% [markdown]
## 11. Compare with Paper Results

# %%
print("\n" + "="*80)
print("COMPARISON WITH PAPER RESULTS")
print("="*80)

paper_results = {
    'Accuracy': 0.9752,
    'Precision (avg)': 0.9753,
    'Recall (avg)': 0.9752,
    'F1-score (avg)': 0.9750,
    'Fall_Initiation Recall': 0.9924,
    'Fall_Initiation F1': 0.9879
}

our_results = {
    'Accuracy': mean_results['val_accuracy'],
    'Precision (avg)': mean_results['val_precision'],
    'Recall (avg)': mean_results['val_recall'],
    'F1-score (avg)': mean_results['val_f1']
}

comparison_df = pd.DataFrame({
    'Metric': list(paper_results.keys())[:4],
    'Paper': [paper_results[k] for k in list(paper_results.keys())[:4]],
    'Our Implementation': [our_results[k] for k in our_results.keys()]
})

print(comparison_df.to_string(index=False))

print("\n📝 Note: For detailed class-wise metrics (especially Fall_Initiation),")
print("   we need to generate a confusion matrix and classification report.")

# %%

TensorFlow version: 2.20.0
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
✅ FallNet class defined

================================================================================
FALLNET MODEL ARCHITECTURE
================================================================================

Model: "FallNet"


        """
        self.input_shape = input_shape
        self.n_classes = n_classes
        self.model = None
    
    def build_lstm_branch(self, inputs):
        """LSTM Branch"""
        x = layers.LSTM(
            units=256,
            activation='tanh',
            return_sequences=False,
            name='lstm_layer'
        )(inputs)
        
        x = layers.Dense(128, activation='relu', name='lstm_dense1')(x)
        x = layers.Dropout(0.2, name='lstm_dropout1')(x)
        
        x = layers.Dense(64, activation='relu', name='lstm_dense2')(x)
        x = layers.Dropout(0.2, name='lstm_dropout2')(x)
        
        x = layers.Dense(32, activation='relu', name='lstm_dense3')(x)
        x = layers.Dropout(0.2, name='lstm_dropout3')(x)
        
        lstm_output = layers.Dense(
            self.n_classes, 
            activation='softmax',
            name='lstm_output'
        )(x)
        
        return lstm_output
    
    def build_cnn_branch(self, inputs):
        """CNN Branch"""
        x = layers.Conv1D(
            filters=128,
            kernel_size=3,
            activation='relu',
            padding='same',
            name='conv1d_layer'
        )(inputs)
        
        x = layers.MaxPooling1D(pool_size=2, name='maxpool_layer')(x)
        x = layers.Flatten(name='flatten_layer')(x)
        
        x = layers.Dense(1024, activation='relu', name='cnn_dense1')(x)
        x = layers.Dropout(0.2, name='cnn_dropout1')(x)
        
        x = layers.Dense(512, activation='relu', name='cnn_dense2')(x)
        x = layers.Dropout(0.2, name='cnn_dropout2')(x)
        
        cnn_output = layers.Dense(
            self.n_classes,
            activation='softmax',
            name='cnn_output'
        )(x)
        
        return cnn_output
    
    def build_ensemble(self):
        """Build the complete ensemble model"""
        inputs = layers.Input(shape=self.input_shape, name='input')
        
        lstm_output = self.build_lstm_branch(inputs)
        cnn_output = self.build_cnn_branch(inputs)
        
        ensemble_output = layers.Average(name='ensemble_average')([lstm_output, cnn_output])
        
        self.model = models.Model(
            inputs=inputs,
            outputs=ensemble_output,
            name='FallNet'
        )
        
        return self.model
    
    def compile_model(self, learning_rate=None):
        """Compile model"""
        if self.model is None:
            raise ValueError("Model not built yet. Call build_ensemble() first.")
        
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate) if learning_rate else keras.optimizers.Adam()
        
        self.model.compile(
            optimizer=optimizer,
            loss='categorical_crossentropy',
            metrics=[
                'accuracy', 
                keras.metrics.Precision(name='precision'),
                keras.metrics.Recall(name='recall')
            ]
        )
        
        return self.model

print("✅ FallNet class defined")

# %% [markdown]
## 2. Build and Display Model

# %%
print("\n" + "="*80)
print("BUILDING FALLNET MODEL")
print("="*80)

# Create instance with 6 classes
fallnet = FallNet(input_shape=(200, 6), n_classes=6)

# Build ensemble
model = fallnet.build_ensemble()

# Compile
model = fallnet.compile_model()

# Display architecture
print("\n")
model.summary()

# Count parameters
def count_parameters(model):
    trainable = np.sum([np.prod(v.shape) for v in model.trainable_weights])
    non_trainable = np.sum([np.prod(v.shape) for v in model.non_trainable_weights])
    return trainable, non_trainable

trainable, non_trainable = count_parameters(model)

print("\n" + "="*80)
print("MODEL PARAMETERS")
print("="*80)
print(f"Trainable:     {trainable:,}")
print(f"Non-trainable: {non_trainable:,}")
print(f"Total:         {trainable + non_trainable:,}")

# %% [markdown]
## 3. Training Configuration

# %%
BATCH_SIZE = 128
EPOCHS = 200
K_FOLDS = 5

print("\n" + "="*80)
print("TRAINING CONFIGURATION")
print("="*80)
print(f"Batch size: {BATCH_SIZE}")
print(f"Max epochs: {EPOCHS}")
print(f"K-Folds:    {K_FOLDS}")
print(f"Using data from previous cell (6 classes, {len(y_labels):,} samples)")

# %% [markdown]
## 4. Verify Data Before Training

# %%
print("\n" + "="*80)
print("PRE-TRAINING VERIFICATION")
print("="*80)

print(f"✅ Data shapes:")
print(f"   X_data:        {X_data.shape}")
print(f"   y_labels:      {y_labels.shape}")
print(f"   y_categorical: {y_categorical.shape}")
print(f"\n✅ Classes: {len(np.unique(y_labels))} (should be 6)")
print(f"✅ Label range: {y_labels.min()}-{y_labels.max()} (should be 0-5)")
print(f"✅ Model output: {model.output_shape[-1]} (should be 6)")

assert X_data.shape[0] == y_labels.shape[0] == y_categorical.shape[0], "Shape mismatch!"
assert len(np.unique(y_labels)) == 6, "Should have 6 classes!"
assert y_labels.max() == 5, "Max label should be 5!"
assert model.output_shape[-1] == 6, "Model should output 6 classes!"

print("\n✅ All checks passed - ready to train!")

# %% [markdown]
## 5. K-Fold Cross-Validation Training

# %%
skf = StratifiedKFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

fold_results = []
fold_histories = []

print("\n" + "="*80)
print("STARTING K-FOLD CROSS-VALIDATION")
print("="*80)

for fold, (train_idx, val_idx) in enumerate(skf.split(X_data, y_labels), 1):
    print(f"\n{'='*80}")
    print(f"FOLD {fold}/{K_FOLDS}")
    print(f"{'='*80}")
    
    # Split data
    X_train, X_val = X_data[train_idx], X_data[val_idx]
    y_train, y_val = y_categorical[train_idx], y_categorical[val_idx]
    
    print(f"Train: {X_train.shape[0]:,} samples | Val: {X_val.shape[0]:,} samples")
    
    # Build fresh model for this fold
    fallnet_fold = FallNet(input_shape=(200, 6), n_classes=6)
    model_fold = fallnet_fold.build_ensemble()
    model_fold = fallnet_fold.compile_model()
    
    # Define callbacks for THIS fold
    fold_callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=20,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=10,
            min_lr=1e-7,
            verbose=1
        ),
        ModelCheckpoint(
            filepath=str(output_dir / f'fallnet_fold_cnn_lstm_original{fold}.keras'),
            monitor='val_accuracy',
            save_best_only=True,
            mode='max',
            verbose=1
        )
    ]
    # %% [markdown]
## 5.5 Calculate Class Weights

# %%
    from sklearn.utils.class_weight import compute_class_weight

    print("\n" + "="*80)
    print("CALCULATING CLASS WEIGHTS")
    print("="*80)

# Calculate balanced weights
    class_weights_array = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(y_labels),
        y=y_labels
    )

# Cap at 3x to prevent training instability
    MAX_WEIGHT = 3.0
    class_weights_array_capped = np.clip(class_weights_array, None, MAX_WEIGHT)
    class_weights = dict(enumerate(class_weights_array_capped))

    print("\nClass Distribution:")
    from collections import Counter
    counts = Counter(y_labels)
    for cls_idx in range(6):
        count = counts[cls_idx]
        pct = count / len(y_labels) * 100
        weight = class_weights[cls_idx]
        print(f"  {reverse_label_map[cls_idx]:<30s}: {count:>5d} ({pct:>5.2f}%) → weight: {weight:.2f}x")

    print(f"\n✅ Weight range: {min(class_weights.values()):.2f}x to {max(class_weights.values()):.2f}x")
    print(f"✅ Max/Min ratio: {max(class_weights.values())/min(class_weights.values()):.2f}x (was 4.0x without capping)")
    # Train WITHOUT class weights
    print(f"\nTraining fold {fold}...")
    history = model_fold.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        class_weight=class_weights,  # ← ADD THIS LINE!
        callbacks=fold_callbacks,
        verbose=1
    )
    
    # Evaluate
    val_loss, val_acc, val_precision, val_recall = model_fold.evaluate(X_val, y_val, verbose=0)
    val_f1 = 2 * (val_precision * val_recall) / (val_precision + val_recall) if (val_precision + val_recall) > 0 else 0
    
    print(f"\n{'='*50}")
    print(f"Fold {fold} Results:")
    print(f"{'='*50}")
    print(f"Loss:      {val_loss:.4f}")
    print(f"Accuracy:  {val_acc:.4f}")
    print(f"Precision: {val_precision:.4f}")
    print(f"Recall:    {val_recall:.4f}")
    print(f"F1-Score:  {val_f1:.4f}")
    
    # Store results
    fold_results.append({
        'fold': fold,
        'val_loss': val_loss,
        'val_accuracy': val_acc,
        'val_precision': val_precision,
        'val_recall': val_recall,
        'val_f1': val_f1
    })
    
    fold_histories.append(history.history)
    
    print(f"✅ Model saved: fallnet_fold_cnn_lstm_{fold}.keras")

print("\n" + "="*80)
print("K-FOLD CROSS-VALIDATION COMPLETE")
print("="*80)

# %% [markdown]
## 6. Aggregate Results

# %%
results_df = pd.DataFrame(fold_results)

print("\n" + "="*80)
print("RESULTS ACROSS ALL FOLDS")
print("="*80)
print(results_df.to_string(index=False))

print("\n" + "="*80)
print("AVERAGE PERFORMANCE ± STD")
print("="*80)

mean_results = results_df.mean(numeric_only=True)
std_results = results_df.std(numeric_only=True)

metrics_table = []
for metric in ['val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_f1']:
    metrics_table.append({
        'Metric': metric,
        'Mean': f"{mean_results[metric]:.4f}",
        'Std': f"±{std_results[metric]:.4f}"
    })

metrics_df = pd.DataFrame(metrics_table)
print(metrics_df.to_string(index=False))

# %% [markdown]
## 7. Visualize Training History

# %%
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

metrics = [
    ('loss', 'Loss'),
    ('accuracy', 'Accuracy'),
    ('precision', 'Precision'),
    ('recall', 'Recall')
]

for idx, (metric, title) in enumerate(metrics):
    ax = axes[idx // 2, idx % 2]
    
    for fold, history in enumerate(fold_histories, 1):
        epochs = range(1, len(history[metric]) + 1)
        ax.plot(epochs, history[metric], label=f'Fold {fold} Train', alpha=0.5, linewidth=1)
        ax.plot(epochs, history[f'val_{metric}'], label=f'Fold {fold} Val', 
                linestyle='--', alpha=0.7, linewidth=1.5)
    
    ax.set_title(f'{title} Across All Folds', fontsize=13, fontweight='bold')
    ax.set_xlabel('Epoch', fontsize=11)
    ax.set_ylabel(title, fontsize=11)
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=7)
    ax.grid(True, alpha=0.3)

plt.suptitle('FallNet Training History - 5-Fold Cross-Validation', 
             fontsize=15, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig(output_dir / 'training_history.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✅ Training history saved to {output_dir / 'training_history.png'}")

# %% [markdown]
## 8. Detailed Evaluation on Best Fold

# %%
best_fold = int(results_df.loc[results_df['val_f1'].idxmax(), 'fold'])

print("\n" + "="*80)
print(f"DETAILED EVALUATION - BEST FOLD #{best_fold}")
print("="*80)
print(f"Best fold F1-Score: {results_df.loc[results_df['fold']==best_fold, 'val_f1'].values[0]:.4f}")

# Load best model
best_model = keras.models.load_model(output_dir / f'fallnet_fold_{best_fold}.keras')

# Get predictions on ALL data
y_pred_probs = best_model.predict(X_data, verbose=0)
y_pred = np.argmax(y_pred_probs, axis=1)

# Classification report
class_names = [reverse_label_map[i] for i in range(6)]

print("\n" + "="*80)
print("CLASSIFICATION REPORT (Best Fold on All Data)")
print("="*80)
print(classification_report(y_labels, y_pred, target_names=class_names, digits=4))

# %% [markdown]
## 9. Per-Class Detailed Metrics

# %%
print("\n" + "="*80)
print("PER-CLASS DETAILED METRICS")
print("="*80)

print(f"\n{'Class':<40s} {'Precision':<12s} {'Recall':<12s} {'F1-Score':<12s} {'Support'}")
print("-"*90)

for cls_idx in range(6):
    precision = precision_score(y_labels == cls_idx, y_pred == cls_idx, zero_division=0)
    recall = recall_score(y_labels == cls_idx, y_pred == cls_idx, zero_division=0)
    f1 = f1_score(y_labels == cls_idx, y_pred == cls_idx, zero_division=0)
    support = np.sum(y_labels == cls_idx)
    
    print(f"{reverse_label_map[cls_idx]:<40s} {precision:<12.4f} {recall:<12.4f} {f1:<12.4f} {support}")

# %% [markdown]
## 10. Confusion Matrix

# %%
cm = confusion_matrix(y_labels, y_pred)

plt.figure(figsize=(12, 10))
sns.heatmap(
    cm, 
    annot=True, 
    fmt='d', 
    cmap='Blues',
    xticklabels=class_names,
    yticklabels=class_names,
    cbar_kws={'label': 'Count'}
)
plt.title('Confusion Matrix - Best Fold (6 Classes)', fontsize=15, fontweight='bold', pad=20)
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.yticks(rotation=0, fontsize=10)
plt.tight_layout()
plt.savefig(output_dir / 'confusion_matrix_cnn_lstm.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✅ Confusion matrix saved to {output_dir / 'confusion_matrix_lstm.png'}")

# %% [markdown]
## 11. Final Summary

# %%
# Get Fall_Initiation metrics
fall_init_idx = label_map["Fall_Initiation"]
fall_init_precision = precision_score(y_labels == fall_init_idx, y_pred == fall_init_idx)
fall_init_recall = recall_score(y_labels == fall_init_idx, y_pred == fall_init_idx)
fall_init_f1 = f1_score(y_labels == fall_init_idx, y_pred == fall_init_idx)

print("\n" + "="*80)
print("TRAINING COMPLETE - FINAL SUMMARY")
print("="*80)

summary = f"""
✅ Successfully trained FallNet with 5-fold cross-validation

Configuration:
  - Model: CNN-LSTM Ensemble (6 classes)
  - Total samples: {len(y_labels):,}
  - Training samples per fold: ~{len(y_labels)*0.8//K_FOLDS:,.0f}
  - Validation samples per fold: ~{len(y_labels)*0.2//K_FOLDS:,.0f}

Average Performance (5-fold CV):
  - Accuracy:  {mean_results['val_accuracy']:.4f} ± {std_results['val_accuracy']:.4f}
  - Precision: {mean_results['val_precision']:.4f} ± {std_results['val_precision']:.4f}
  - Recall:    {mean_results['val_recall']:.4f} ± {std_results['val_recall']:.4f}
  - F1-Score:  {mean_results['val_f1']:.4f} ± {std_results['val_f1']:.4f}

Fall_Initiation Performance (Critical Class):
  - Recall (Sensitivity): {fall_init_recall:.4f}
  - F1-Score:             {fall_init_f1:.4f}

Saved Files:
  - Training history:    {output_dir / 'training_history.png'}
  - Confusion matrix:    {output_dir / 'confusion_matrix.png'}
  - Best model:          {output_dir / f'fallnet_fold_cnn_lstm{best_fold}.keras'}
  - All fold models:     {output_dir / 'fallnet_fold_*.keras'}
"""

print(summary)

with open(output_dir / 'training_summary_cnn_lstm.txt', 'w') as f:
    f.write(summary)

print(f"✅ Summary saved to {output_dir / 'training_summary.txt'}")

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

model_dir = Path.home() / 'repos/summerschool2023/projects/fall-detection/fall_detection_data/models'

print("="*80)
print("DETAILED ANALYSIS: Fold 1 - fallnet_fold_cnn_lstm_original1.keras")
print("="*80)

class_names = ['Walking', 'Jogging', 'Walking_stairs_updown', 
               'Stumble_while_walking', 'Fall_Initiation', 'Impact_Aftermath']

# Get Fold 1 validation data
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(skf.split(X_data, y_labels), 1):
    if fold == 1:  # Only Fold 1
        X_val = X_data[val_idx]
        y_val = y_labels[val_idx]
        y_val_cat = y_categorical[val_idx]
        
        print(f"\nValidation set size: {len(X_val)}")
        print(f"\nClass distribution in Fold 1 validation:")
        for i, class_name in enumerate(class_names):
            count = np.sum(y_val == i)
            print(f"  {class_name:30s}: {count:4d} samples")
        
        # Load model
        model_path = model_dir / 'fallnet_fold_cnn_lstm_original1.keras'
        print(f"\nLoading: {model_path.name}")
        model = tf.keras.models.load_model(model_path)
        
        # Predict
        print("Predicting...")
        y_pred_proba = model.predict(X_val, batch_size=64, verbose=0)
        y_pred = np.argmax(y_pred_proba, axis=1)
        
        # Overall metrics
        val_loss, val_acc, val_precision, val_recall = model.evaluate(
            X_val, y_val_cat, batch_size=64, verbose=0
        )
        
        print(f"\nOverall Metrics:")
        print(f"  Accuracy:  {val_acc:.4f} ({val_acc*100:.2f}%)")
        print(f"  Precision: {val_precision:.4f}")
        print(f"  Recall:    {val_recall:.4f}")
        
        # Detailed classification report
        print("\n" + "="*80)
        print("CLASSIFICATION REPORT")
        print("="*80)
        report = classification_report(y_val, y_pred, target_names=class_names, digits=4)
        print(report)
        
        # Focus on Fall_Initiation
        print("\n" + "="*80)
        print("FALL_INITIATION DETAILED ANALYSIS")
        print("="*80)
        
        fall_init_idx = 4  # Fall_Initiation is class 4
        fall_init_mask = (y_val == fall_init_idx)
        fall_init_true = y_val[fall_init_mask]
        fall_init_pred = y_pred[fall_init_mask]
        
        total_falls = len(fall_init_true)
        correct_falls = np.sum(fall_init_pred == fall_init_idx)
        
        print(f"\nTotal Fall_Initiation samples: {total_falls}")
        print(f"Correctly predicted as falls:  {correct_falls} ({correct_falls/total_falls*100:.2f}%)")
        print(f"Incorrectly predicted:          {total_falls - correct_falls} ({(total_falls-correct_falls)/total_falls*100:.2f}%)")
        
        print(f"\nWhat did the model predict instead?")
        for i, class_name in enumerate(class_names):
            count = np.sum(fall_init_pred == i)
            if count > 0:
                print(f"  Predicted as {class_name:30s}: {count:4d} ({count/total_falls*100:.2f}%)")
        
        # Confusion matrix
        print("\n" + "="*80)
        print("CONFUSION MATRIX")
        print("="*80)
        
        cm = confusion_matrix(y_val, y_pred)
        
        # Print Fall_Initiation row specifically
        print(f"\nFall_Initiation (row 4) predictions:")
        for i, class_name in enumerate(class_names):
            print(f"  Predicted as {class_name:30s}: {cm[fall_init_idx, i]:4d}")
        
        # Visualize confusion matrix
        fig, ax = plt.subplots(figsize=(10, 8))
        
        cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        
        sns.heatmap(cm_normalized, annot=True, fmt='.3f', cmap='Blues',
                   xticklabels=class_names, yticklabels=class_names,
                   ax=ax, cbar_kws={'label': 'Proportion'})
        
        ax.set_title('Fold 1 CNN-LSTM Confusion Matrix (Normalized)', 
                    fontsize=14, fontweight='bold')
        ax.set_ylabel('True Label', fontsize=12)
        ax.set_xlabel('Predicted Label', fontsize=12)
        plt.setp(ax.get_xticklabels(), rotation=45, ha='right')
        plt.setp(ax.get_yticklabels(), rotation=0)
        
        # Highlight Fall_Initiation row
        ax.add_patch(plt.Rectangle((0, fall_init_idx), 6, 1, 
                                   fill=False, edgecolor='red', lw=3))
        
        plt.tight_layout()
        plt.savefig(model_dir / 'fold1_confusion_matrix.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        print(f"\n✓ Confusion matrix saved: {model_dir / 'fold1_confusion_matrix.png'}")
        
        # Show prediction confidence for Fall_Initiation samples
        print("\n" + "="*80)
        print("PREDICTION CONFIDENCE ANALYSIS")
        print("="*80)
        
        fall_init_proba = y_pred_proba[fall_init_mask]
        
        print(f"\nFor the {total_falls} Fall_Initiation samples:")
        print(f"  Mean confidence in Fall_Initiation class: {fall_init_proba[:, fall_init_idx].mean():.4f}")
        print(f"  Max confidence in Fall_Initiation class:  {fall_init_proba[:, fall_init_idx].max():.4f}")
        print(f"  Min confidence in Fall_Initiation class:  {fall_init_proba[:, fall_init_idx].min():.4f}")
        
        # Find what the model is most confident about for these fall samples
        most_confident_class = np.argmax(fall_init_proba, axis=1)
        print(f"\nWhat class does the model think these falls are?")
        for i, class_name in enumerate(class_names):
            count = np.sum(most_confident_class == i)
            if count > 0:
                avg_conf = fall_init_proba[most_confident_class == i, i].mean()
                print(f"  {class_name:30s}: {count:4d} samples (avg confidence: {avg_conf:.4f})")
        
        break

print("\n" + "="*80)
print("FOLD 1 ANALYSIS COMPLETE")
print("="*80)

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from pathlib import Path

model_dir = Path.home() / 'repos/summerschool2023/projects/fall-detection/fall_detection_data/models'

# Check the specific file
model_path = model_dir / 'fallnet_fold_cnn_lstm_original5.keras'

print(f"Checking: {model_path.name}")
print(f"File exists: {model_path.exists()}")

if model_path.exists():
    print(f"File size: {model_path.stat().st_size / (1024*1024):.1f} MB")
    print(f"Modified: {model_path.stat().st_mtime}")
    
    # Load it
    try:
        print("\nLoading model...")
        model = tf.keras.models.load_model(model_path)
        print("✓ Model loaded successfully")
        
        # Check architecture
        print("\n" + "="*80)
        print("MODEL SUMMARY")
        print("="*80)
        model.summary()
        
        # Evaluate on Fold 5 validation data
        print("\n" + "="*80)
        print("EVALUATING ON FOLD 5 VALIDATION DATA")
        print("="*80)
        
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        
        for fold, (train_idx, val_idx) in enumerate(skf.split(X_data, y_labels), 1):
            if fold == 5:  # Only check fold 5
                X_val = X_data[val_idx]
                y_val = y_labels[val_idx]
                y_val_cat = y_categorical[val_idx]
                
                # Predict
                y_pred_proba = model.predict(X_val, batch_size=64, verbose=1)
                y_pred = np.argmax(y_pred_proba, axis=1)
                
                # Evaluate
                val_loss, val_acc, val_precision, val_recall = model.evaluate(
                    X_val, y_val_cat, batch_size=64, verbose=0
                )
                
                print(f"\nFold 5 Results:")
                print(f"  Accuracy:  {val_acc:.4f} ({val_acc*100:.2f}%)")
                print(f"  Precision: {val_precision:.4f}")
                print(f"  Recall:    {val_recall:.4f}")
                print(f"  Loss:      {val_loss:.4f}")
                
                # Detailed classification report
                class_names = ['Walking', 'Jogging', 'Walking_stairs_updown', 
                              'Stumble_while_walking', 'Fall_Initiation', 'Impact_Aftermath']
                
                print("\n" + "="*80)
                print("DETAILED CLASSIFICATION REPORT")
                print("="*80)
                
                report = classification_report(
                    y_val, y_pred,
                    target_names=class_names,
                    digits=3
                )
                print(report)
                
                break
        
    except Exception as e:
        print(f"✗ Error loading model: {e}")
        import traceback
        traceback.print_exc()

# Also check what other "original" models exist
print("\n" + "="*80)
print("ALL 'ORIGINAL' MODEL FILES")
print("="*80)

for model_file in sorted(model_dir.glob("*original*.keras")):
    print(f"{model_file.name:50s} {model_file.stat().st_size / (1024*1024):6.1f} MB")

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
import gc

# Clear Keras session
K.clear_session()
tf.keras.backend.clear_session()

# Force garbage collection
gc.collect()

# Try to reset GPU memory stats (if available)
try:
    tf.config.experimental.reset_memory_stats('GPU:0')
    print("✓ GPU memory stats reset")
except:
    print("⚠️  Could not reset GPU memory stats")

print("✓ Memory cleared")

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from pathlib import Path
import pandas as pd

model_dir = Path.home() / 'repos/summerschool2023/projects/fall-detection/fall_detection_data/models'

print("="*80)
print("EVALUATING ALL 5: fallnet_fold_cnn_lstm_original[1-5].keras")
print("="*80)

class_names = ['Walking', 'Jogging', 'Walking_stairs_updown', 
               'Stumble_while_walking', 'Fall_Initiation', 'Impact_Aftermath']

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

all_y_true = []
all_y_pred = []
fold_results = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X_data, y_labels), 1):
    model_path = model_dir / f'fallnet_fold_cnn_lstm_{fold}.keras'
    
    if not model_path.exists():
        print(f"\nFold {fold}: ✗ File not found: {model_path.name}")
        continue
    
    print(f"\n{'='*80}")
    print(f"FOLD {fold}: {model_path.name}")
    print(f"{'='*80}")
    print(f"File size: {model_path.stat().st_size / (1024*1024):.1f} MB")
    
    try:
        # Load model
        print("Loading model...", end=' ')
        model = tf.keras.models.load_model(model_path)
        print("✓")
        
        # Get validation data
        X_val = X_data[val_idx]
        y_val = y_labels[val_idx]
        y_val_cat = y_categorical[val_idx]
        
        print(f"Validation samples: {len(X_val)}")
        
        # Predict
        print("Predicting...", end=' ')
        y_pred_proba = model.predict(X_val, batch_size=64, verbose=0)
        y_pred = np.argmax(y_pred_proba, axis=1)
        print("✓")
        
        # Evaluate
        print("Evaluating...", end=' ')
        val_loss, val_acc, val_precision, val_recall = model.evaluate(
            X_val, y_val_cat, batch_size=64, verbose=0
        )
        print("✓")
        
        # Get per-class metrics for this fold
        report_dict = classification_report(
            y_val, y_pred,
            target_names=class_names,
            output_dict=True,
            zero_division=0
        )
        
        fall_init_recall = report_dict['Fall_Initiation']['recall']
        fall_init_f1 = report_dict['Fall_Initiation']['f1-score']
        fall_init_precision = report_dict['Fall_Initiation']['precision']
        
        print(f"\nResults:")
        print(f"  Overall Accuracy:       {val_acc:.4f} ({val_acc*100:.2f}%)")
        print(f"  Overall Precision:      {val_precision:.4f}")
        print(f"  Overall Recall:         {val_recall:.4f}")
        print(f"  Loss:                   {val_loss:.4f}")
        print(f"\n  Fall_Initiation:")
        print(f"    Precision: {fall_init_precision:.4f}")
        print(f"    Recall:    {fall_init_recall:.4f} ({'✓' if fall_init_recall > 0.90 else '✗'})")
        print(f"    F1-Score:  {fall_init_f1:.4f}")
        
        # Store results
        all_y_true.extend(y_val)
        all_y_pred.extend(y_pred)
        fold_results.append({
            'fold': fold,
            'accuracy': val_acc,
            'precision': val_precision,
            'recall': val_recall,
            'loss': val_loss,
            'fall_init_precision': fall_init_precision,
            'fall_init_recall': fall_init_recall,
            'fall_init_f1': fall_init_f1
        })
        
        # Clear memory
        del model
        tf.keras.backend.clear_session()
        
    except Exception as e:
        print(f"\n✗ Error: {e}")
        import traceback
        traceback.print_exc()

# ============================================================================
# SUMMARY
# ============================================================================

print("\n" + "="*80)
print("SUMMARY: ALL 5 FOLDS")
print("="*80)

if len(fold_results) > 0:
    df = pd.DataFrame(fold_results)
    
    # Format table nicely
    print("\nPer-Fold Results:")
    print(df.to_string(index=False, float_format=lambda x: f'{x:.4f}'))
    
    print("\n" + "="*80)
    print("AVERAGE PERFORMANCE ± STANDARD DEVIATION")
    print("="*80)
    print(f"Overall Accuracy:          {df['accuracy'].mean():.4f} ± {df['accuracy'].std():.4f}")
    print(f"Overall Precision:         {df['precision'].mean():.4f} ± {df['precision'].std():.4f}")
    print(f"Overall Recall:            {df['recall'].mean():.4f} ± {df['recall'].std():.4f}")
    print(f"Loss:                      {df['loss'].mean():.4f} ± {df['loss'].std():.4f}")
    print(f"\nFall_Initiation Precision: {df['fall_init_precision'].mean():.4f} ± {df['fall_init_precision'].std():.4f}")
    print(f"Fall_Initiation Recall:    {df['fall_init_recall'].mean():.4f} ± {df['fall_init_recall'].std():.4f}")
    print(f"Fall_Initiation F1:        {df['fall_init_f1'].mean():.4f} ± {df['fall_init_f1'].std():.4f}")
    
    # Best and worst folds
    best_fold = df.loc[df['accuracy'].idxmax()]
    worst_fold = df.loc[df['accuracy'].idxmin()]
    
    print("\n" + "="*80)
    print("BEST & WORST FOLDS")
    print("="*80)
    print(f"Best:  Fold {best_fold['fold']:.0f} - {best_fold['accuracy']:.4f} ({best_fold['accuracy']*100:.2f}%)")
    print(f"Worst: Fold {worst_fold['fold']:.0f} - {worst_fold['accuracy']:.4f} ({worst_fold['accuracy']*100:.2f}%)")
    print(f"Range: {(best_fold['accuracy'] - worst_fold['accuracy'])*100:.2f}%")

# Detailed per-class performance (all folds combined)
if len(all_y_true) > 0:
    print("\n" + "="*80)
    print("DETAILED PER-CLASS PERFORMANCE (ALL FOLDS COMBINED)")
    print("="*80)
    
    report = classification_report(
        all_y_true, all_y_pred,
        target_names=class_names,
        digits=3
    )
    print(report)
    
    # Get report as dict
    report_dict = classification_report(
        all_y_true, all_y_pred,
        target_names=class_names,
        output_dict=True
    )
    
    print("\n" + "="*80)
    print("CLASSES RANKED BY F1-SCORE (WORST TO BEST)")
    print("="*80)
    
    class_f1 = [(cn, report_dict[cn]['f1-score'], report_dict[cn]['recall']) 
                for cn in class_names]
    class_f1.sort(key=lambda x: x[1])
    
    for cn, f1, recall in class_f1:
        print(f"{cn:30s}: F1={f1:.3f}, Recall={recall:.3f}")

# ============================================================================
# COMPARISON TO OTHER MODELS
# ============================================================================

print("\n" + "="*80)
print("COMPARISON TO OTHER MODELS")
print("="*80)

if len(fold_results) > 0:
    avg_acc = df['accuracy'].mean()
    avg_std = df['accuracy'].std()
    
    print(f"CNN-LSTM 'original':  {avg_acc:.4f} ± {avg_std:.4f} ({avg_acc*100:.2f}%)")
    print(f"CNN-only:             0.8998 ± 0.0042 (89.98%)")
    print(f"CNN-LMU:              0.8997 ± 0.0029 (89.97%)")
    print(f"CNN-LSTM 'ensemble':  0.8858 ± 0.0385 (88.58%)")
    print(f"\nPaper target:         0.9752 (97.52%)")
    
    # Determine winner
    print("\n" + "="*80)
    if avg_acc > 0.8998:
        print(f"🏆 CNN-LSTM 'original' WINS!")
        print(f"   Better than CNN-only by: +{(avg_acc - 0.8998)*100:.2f}%")
    elif avg_acc > 0.8997 and avg_acc <= 0.8998:
        print(f"⚖️  CNN-LSTM 'original' ties with CNN-only")
        print(f"   Difference: {(avg_acc - 0.8998)*100:.2f}%")
    else:
        print(f"❌ CNN-LSTM 'original' loses to CNN-only")
        print(f"   Worse by: {(avg_acc - 0.8998)*100:.2f}%")
    
    print(f"\n   Gap to paper: {(0.9752 - avg_acc)*100:.2f}%")
    print("="*80)

print("\n✅ EVALUATION COMPLETE!\n")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from pathlib import Path
from keras_lmu import LMU  # Import LMU

# Setup
class_names = ['Walking', 'Jogging', 'Walking_stairs_updown', 
               'Stumble_while_walking', 'Fall_Initiation', 'Impact_Aftermath']

model_dir = Path.home() / 'repos/summerschool2023/projects/fall-detection/fall_detection_data/models'

# Just analyze ONE model first to test
print("Analyzing CNN-LMU (your best model)...")

all_y_true = []
all_y_pred = []

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(skf.split(X_data, y_labels), 1):
    model_path = model_dir / f'fallnet_fold_{fold}.keras'  # CNN-LMU
    
    if not model_path.exists():
        print(f"Fold {fold}: File not found")
        continue
    
    print(f"Loading fold {fold}...", end=' ')
    
    try:
        # Load model WITH custom objects
        model = tf.keras.models.load_model(
            model_path,
            custom_objects={'LMU': LMU}  # ← THIS IS THE KEY FIX
        )
        
        # Get validation data
        X_val = X_data[val_idx]
        y_val = y_labels[val_idx]
        
        # Predict
        y_pred_proba = model.predict(X_val, batch_size=64, verbose=0)
        y_pred = np.argmax(y_pred_proba, axis=1)
        
        # Store
        all_y_true.extend(y_val)
        all_y_pred.extend(y_pred)
        
        print("✓")
        
    except Exception as e:
        print(f"✗ Error: {e}")

# Generate report
print("\n" + "="*80)
print("CNN-LMU Per-Class Performance (All Folds Combined)")
print("="*80)

report = classification_report(
    all_y_true, all_y_pred,
    target_names=class_names,
    digits=3
)

print(report)

# Extract per-class metrics
report_dict = classification_report(
    all_y_true, all_y_pred,
    target_names=class_names,
    output_dict=True
)

# Simple bar chart
fig, ax = plt.subplots(figsize=(12, 6))

classes = class_names
f1_scores = [report_dict[cn]['f1-score'] for cn in class_names]
recalls = [report_dict[cn]['recall'] for cn in class_names]
precisions = [report_dict[cn]['precision'] for cn in class_names]

x = np.arange(len(classes))
width = 0.25

ax.bar(x - width, precisions, width, label='Precision', alpha=0.8)
ax.bar(x, recalls, width, label='Recall', alpha=0.8)
ax.bar(x + width, f1_scores, width, label='F1-Score', alpha=0.8)

ax.set_xlabel('Class', fontsize=12)
ax.set_ylabel('Score', fontsize=12)
ax.set_title('CNN-LMU: Per-Class Performance', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(classes, rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)
ax.set_ylim([0, 1.05])

plt.tight_layout()
plt.savefig(model_dir / 'cnn_lmu_per_class.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✓ Saved to: {model_dir / 'cnn_lmu_per_class.png'}")

# Show which classes are weakest
print("\n" + "="*80)
print("Classes Ranked by F1-Score (Worst to Best)")
print("="*80)

class_f1 = [(cn, report_dict[cn]['f1-score']) for cn in class_names]
class_f1.sort(key=lambda x: x[1])

for cn, f1 in class_f1:
    print(f"{cn:30s}: {f1:.3f}")

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from pathlib import Path
import pandas as pd

model_dir = Path.home() / 'repos/summerschool2023/projects/fall-detection/fall_detection_data/models'

print("="*80)
print("EVALUATING: fallnet_fold_cnn_lstm_original[1-5].keras")
print("="*80)

class_names = ['Walking', 'Jogging', 'Walking_stairs_updown', 
               'Stumble_while_walking', 'Fall_Initiation', 'Impact_Aftermath']

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

all_y_true = []
all_y_pred = []
fold_results = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X_data, y_labels), 1):
    model_path = model_dir / f'fallnet_fold_cnn_lstm_{fold}.keras'
    
    if not model_path.exists():
        print(f"\nFold {fold}: ✗ File not found")
        continue
    
    print(f"\nFold {fold}: {model_path.name}")
    print(f"  File size: {model_path.stat().st_size / (1024*1024):.1f} MB")
    
    try:
        # Load model
        model = tf.keras.models.load_model(model_path)
        
        # Get validation data
        X_val = X_data[val_idx]
        y_val = y_labels[val_idx]
        y_val_cat = y_categorical[val_idx]
        
        # Predict
        y_pred_proba = model.predict(X_val, batch_size=64, verbose=0)
        y_pred = np.argmax(y_pred_proba, axis=1)
        
        # Evaluate
        val_loss, val_acc, val_precision, val_recall = model.evaluate(
            X_val, y_val_cat, batch_size=64, verbose=0
        )
        
        # Get per-class metrics for this fold
        report_dict = classification_report(
            y_val, y_pred,
            target_names=class_names,
            output_dict=True,
            zero_division=0
        )
        
        fall_init_recall = report_dict['Fall_Initiation']['recall']
        fall_init_f1 = report_dict['Fall_Initiation']['f1-score']
        
        print(f"  Accuracy:  {val_acc:.4f} ({val_acc*100:.2f}%)")
        print(f"  Precision: {val_precision:.4f}")
        print(f"  Recall:    {val_recall:.4f}")
        print(f"  Fall_Initiation Recall: {fall_init_recall:.4f}")
        print(f"  Fall_Initiation F1:     {fall_init_f1:.4f}")
        
        # Store results
        all_y_true.extend(y_val)
        all_y_pred.extend(y_pred)
        fold_results.append({
            'fold': fold,
            'accuracy': val_acc,
            'precision': val_precision,
            'recall': val_recall,
            'loss': val_loss,
            'fall_init_recall': fall_init_recall,
            'fall_init_f1': fall_init_f1
        })
        
        # Clear memory
        del model
        tf.keras.backend.clear_session()
        
    except Exception as e:
        print(f"  ✗ Error: {e}")
        import traceback
        traceback.print_exc()

# Summary table
print("\n" + "="*80)
print("PER-FOLD SUMMARY")
print("="*80)

if len(fold_results) > 0:
    df = pd.DataFrame(fold_results)
    print(df.to_string(index=False))
    
    print("\n" + "="*80)
    print("AVERAGE PERFORMANCE ± STD")
    print("="*80)
    print(f"Accuracy:             {df['accuracy'].mean():.4f} ± {df['accuracy'].std():.4f}")
    print(f"Precision:            {df['precision'].mean():.4f} ± {df['precision'].std():.4f}")
    print(f"Recall:               {df['recall'].mean():.4f} ± {df['recall'].std():.4f}")
    print(f"Fall_Initiation Recall: {df['fall_init_recall'].mean():.4f} ± {df['fall_init_recall'].std():.4f}")
    print(f"Fall_Initiation F1:     {df['fall_init_f1'].mean():.4f} ± {df['fall_init_f1'].std():.4f}")

# Detailed per-class performance (all folds combined)
if len(all_y_true) > 0:
    print("\n" + "="*80)
    print("DETAILED PER-CLASS PERFORMANCE (ALL FOLDS)")
    print("="*80)
    
    report = classification_report(
        all_y_true, all_y_pred,
        target_names=class_names,
        digits=3
    )
    print(report)
    
    # Get report as dict for comparison
    report_dict = classification_report(
        all_y_true, all_y_pred,
        target_names=class_names,
        output_dict=True
    )
    
    print("\n" + "="*80)
    print("PER-CLASS F1-SCORES RANKED")
    print("="*80)
    
    class_f1 = [(cn, report_dict[cn]['f1-score']) for cn in class_names]
    class_f1.sort(key=lambda x: x[1])
    
    for cn, f1 in class_f1:
        print(f"{cn:30s}: {f1:.3f}")

print("\n" + "="*80)
print("COMPARISON TO OTHER MODELS")
print("="*80)

if len(fold_results) > 0:
    avg_acc = df['accuracy'].mean()
    
    print(f"CNN-LSTM 'original' (these models): {avg_acc:.4f} ({avg_acc*100:.2f}%)")
    print(f"CNN-only:                           0.8998 (89.98%)")
    print(f"CNN-LMU:                            0.8997 (89.97%)")
    print(f"CNN-LSTM 'ensemble':                0.8858 (88.58%)")
    print(f"\nPaper target:                       0.9752 (97.52%)")
    
    if avg_acc > 0.8998:
        print(f"\n✓ These CNN-LSTM models are the BEST! (+{(avg_acc - 0.8998)*100:.2f}%)")
    else:
        print(f"\n✗ Still behind CNN-only by {(0.8998 - avg_acc)*100:.2f}%")

print("\n" + "="*80)
print("EVALUATION COMPLETE")
print("="*80)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from pathlib import Path
from keras_lmu import LMU  # For CNN-LMU

# Setup
class_names = ['Walking', 'Jogging', 'Walking_stairs_updown', 
               'Stumble_while_walking', 'Fall_Initiation', 'Impact_Aftermath']

model_dir = Path.home() / 'repos/summerschool2023/projects/fall-detection/fall_detection_data/models'

# Define models to compare
models_to_compare = {
    'CNN-only': {
        'pattern': 'cnn_only_fold_{}.keras',
        'custom_objects': None
    },
    'CNN-LSTM': {
        'pattern': 'fallnet_fold_cnn_lstm_{}.keras',
        'custom_objects': None
    },
    'CNN-LMU': {
        'pattern': 'fallnet_reg_lmu_fold_{}.keras',
        'custom_objects': {'LMU': LMU}  # Need this for LMU
    }
}

# Collect per-class metrics for each model
all_results = {}

for model_name, model_info in models_to_compare.items():
    print(f"\n{'='*80}")
    print(f"Analyzing {model_name}")
    print(f"{'='*80}")
    
    all_y_true = []
    all_y_pred = []
    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X_data, y_labels), 1):
        model_path = model_dir / model_info['pattern'].format(fold)
        
        if not model_path.exists():
            print(f"  ⚠️  Fold {fold} not found")
            continue
        
        print(f"  Loading fold {fold}...", end=' ')
        
        try:
            # Load model with custom objects if needed
            if model_info['custom_objects']:
                model = tf.keras.models.load_model(
                    model_path,
                    custom_objects=model_info['custom_objects']
                )
            else:
                model = tf.keras.models.load_model(model_path)
            
            # Get validation data
            X_val = X_data[val_idx]
            y_val = y_labels[val_idx]
            
            # Predict
            y_pred_proba = model.predict(X_val, batch_size=64, verbose=0)
            y_pred = np.argmax(y_pred_proba, axis=1)
            
            # Store
            all_y_true.extend(y_val)
            all_y_pred.extend(y_pred)
            
            print("✓")
            
            # Clear memory
            del model
            tf.keras.backend.clear_session()
            
        except Exception as e:
            print(f"✗ Error: {e}")
            continue
    
    # Generate classification report
    if len(all_y_true) > 0:
        report_dict = classification_report(
            all_y_true, all_y_pred,
            target_names=class_names,
            output_dict=True,
            zero_division=0
        )
        all_results[model_name] = {
            'report': report_dict,
            'y_true': all_y_true,
            'y_pred': all_y_pred
        }
        
        # Print summary
        print(f"\n  {model_name} Overall: {report_dict['accuracy']:.3f}")

print("\n" + "="*80)
print("ANALYSIS COMPLETE - Generating Visualizations...")
print("="*80)

# ============================================================================
# VISUALIZATION 1: Per-Class Metrics Comparison (Side-by-Side Bars)
# ============================================================================

fig, axes = plt.subplots(1, 3, figsize=(18, 6))
metrics_to_plot = ['precision', 'recall', 'f1-score']

for idx, metric in enumerate(metrics_to_plot):
    ax = axes[idx]
    
    x = np.arange(len(class_names))
    width = 0.25
    
    for i, model_name in enumerate(models_to_compare.keys()):
        if model_name in all_results:
            scores = []
            for cn in class_names:
                if cn in all_results[model_name]['report']:
                    scores.append(all_results[model_name]['report'][cn][metric])
                else:
                    scores.append(0)
            
            ax.bar(x + i*width, scores, width, label=model_name, alpha=0.8)
    
    ax.set_xlabel('Class', fontsize=12, fontweight='bold')
    ax.set_ylabel(metric.capitalize(), fontsize=12, fontweight='bold')
    ax.set_title(f'{metric.capitalize()} by Class', fontsize=14, fontweight='bold')
    ax.set_xticks(x + width)
    ax.set_xticklabels(class_names, rotation=45, ha='right', fontsize=10)
    ax.legend(fontsize=10)
    ax.grid(axis='y', alpha=0.3)
    ax.set_ylim([0.80, 1.05])
    ax.axhline(y=0.9, color='red', linestyle='--', alpha=0.3, label='90%')

plt.tight_layout()
plt.savefig(model_dir / 'per_class_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Saved: {model_dir / 'per_class_comparison.png'}")

# ============================================================================
# VISUALIZATION 2: F1-Score Heatmap
# ============================================================================

fig, ax = plt.subplots(figsize=(12, 5))

heatmap_data = []
model_labels = []

for model_name in models_to_compare.keys():
    if model_name in all_results:
        row = []
        for class_name in class_names:
            if class_name in all_results[model_name]['report']:
                row.append(all_results[model_name]['report'][class_name]['f1-score'])
            else:
                row.append(0)
        heatmap_data.append(row)
        model_labels.append(model_name)

heatmap_df = pd.DataFrame(heatmap_data, columns=class_names, index=model_labels)

sns.heatmap(heatmap_df, annot=True, fmt='.3f', cmap='RdYlGn', 
            vmin=0.85, vmax=1.0, center=0.925,
            cbar_kws={'label': 'F1-Score'}, ax=ax, linewidths=0.5)

ax.set_title('F1-Score Heatmap: Model vs Class Performance', 
             fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Class', fontsize=12, fontweight='bold')
ax.set_ylabel('Model', fontsize=12, fontweight='bold')
plt.setp(ax.get_xticklabels(), rotation=45, ha='right', fontsize=10)
plt.setp(ax.get_yticklabels(), rotation=0, fontsize=11)

plt.tight_layout()
plt.savefig(model_dir / 'f1_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Saved: {model_dir / 'f1_heatmap.png'}")

# ============================================================================
# VISUALIZATION 3: Overall Model Comparison (Bar Chart)
# ============================================================================

fig, ax = plt.subplots(figsize=(10, 6))

models = []
accuracies = []
precisions = []
recalls = []
f1s = []

for model_name in models_to_compare.keys():
    if model_name in all_results:
        report = all_results[model_name]['report']
        models.append(model_name)
        accuracies.append(report['accuracy'])
        precisions.append(report['weighted avg']['precision'])
        recalls.append(report['weighted avg']['recall'])
        f1s.append(report['weighted avg']['f1-score'])

x = np.arange(len(models))
width = 0.2

ax.bar(x - 1.5*width, accuracies, width, label='Accuracy', alpha=0.8)
ax.bar(x - 0.5*width, precisions, width, label='Precision', alpha=0.8)
ax.bar(x + 0.5*width, recalls, width, label='Recall', alpha=0.8)
ax.bar(x + 1.5*width, f1s, width, label='F1-Score', alpha=0.8)

ax.set_xlabel('Model', fontsize=12, fontweight='bold')
ax.set_ylabel('Score', fontsize=12, fontweight='bold')
ax.set_title('Overall Model Performance Comparison', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(models, fontsize=11)
ax.legend(fontsize=10)
ax.grid(axis='y', alpha=0.3)
ax.set_ylim([0.85, 0.95])

# Add value labels on bars
for i, (acc, prec, rec, f1) in enumerate(zip(accuracies, precisions, recalls, f1s)):
    ax.text(i - 1.5*width, acc + 0.003, f'{acc:.3f}', ha='center', fontsize=9)
    ax.text(i - 0.5*width, prec + 0.003, f'{prec:.3f}', ha='center', fontsize=9)
    ax.text(i + 0.5*width, rec + 0.003, f'{rec:.3f}', ha='center', fontsize=9)
    ax.text(i + 1.5*width, f1 + 0.003, f'{f1:.3f}', ha='center', fontsize=9)

plt.tight_layout()
plt.savefig(model_dir / 'overall_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Saved: {model_dir / 'overall_comparison.png'}")

# ============================================================================
# DETAILED TEXT SUMMARY
# ============================================================================

print("\n" + "="*80)
print("DETAILED PER-CLASS COMPARISON")
print("="*80)

summary_data = []
for class_name in class_names:
    row = {'Class': class_name}
    for model_name in models_to_compare.keys():
        if model_name in all_results:
            report = all_results[model_name]['report']
            if class_name in report:
                f1 = report[class_name]['f1-score']
                recall = report[class_name]['recall']
                precision = report[class_name]['precision']
                row[f'{model_name}_F1'] = f'{f1:.3f}'
                row[f'{model_name}_Recall'] = f'{recall:.3f}'
                row[f'{model_name}_Precision'] = f'{precision:.3f}'
    summary_data.append(row)

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))

# Save summary
summary_df.to_csv(model_dir / 'per_class_comparison.csv', index=False)
print(f"\n✓ Summary CSV saved: {model_dir / 'per_class_comparison.csv'}")

# ============================================================================
# FIND BEST MODEL PER CLASS
# ============================================================================

print("\n" + "="*80)
print("BEST MODEL FOR EACH CLASS (by F1-Score)")
print("="*80)

for class_name in class_names:
    best_model = None
    best_f1 = 0
    
    for model_name in models_to_compare.keys():
        if model_name in all_results:
            report = all_results[model_name]['report']
            if class_name in report:
                f1 = report[class_name]['f1-score']
                if f1 > best_f1:
                    best_f1 = f1
                    best_model = model_name
    
    print(f"{class_name:30s}: {best_model:15s} (F1={best_f1:.3f})")

# ============================================================================
# OVERALL WINNER
# ============================================================================

print("\n" + "="*80)
print("OVERALL BEST MODEL (by Accuracy)")
print("="*80)

best_overall = None
best_acc = 0

for model_name in models_to_compare.keys():
    if model_name in all_results:
        acc = all_results[model_name]['report']['accuracy']
        print(f"{model_name:15s}: {acc:.3f} ({acc*100:.2f}%)")
        if acc > best_acc:
            best_acc = acc
            best_overall = model_name

print(f"\n🏆 Winner: {best_overall} with {best_acc:.3f} ({best_acc*100:.2f}%) accuracy")

print("\n" + "="*80)
print("ALL VISUALIZATIONS COMPLETE!")
print("="*80)

In [ ]:
import tensorflow as tf
from pathlib import Path
import numpy as np

model_dir = Path.home() / 'repos/summerschool2023/projects/fall-detection/fall_detection_data/models'

# Load the best model
best_model_path = model_dir / 'fallnet_best_model.keras'
print(f"Loading: {best_model_path}")

best_model = tf.keras.models.load_model(best_model_path)

# 1. Check the architecture
print("\n" + "="*80)
print("MODEL ARCHITECTURE")
print("="*80)
best_model.summary()

# 2. Check what accuracy it gets NOW on your current data
print("\n" + "="*80)
print("EVALUATING ON CURRENT DATA")
print("="*80)

# Use the same K-fold split to get validation data
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(skf.split(X_data, y_labels), 1):
    X_val = X_data[val_idx]
    y_val = y_categorical[val_idx]
    
    val_loss, val_acc, val_precision, val_recall = best_model.evaluate(
        X_val, y_val, batch_size=64, verbose=0
    )
    
    print(f"Fold {fold}: Accuracy = {val_acc:.4f} ({val_acc*100:.2f}%)")
    print(f"          Precision = {val_precision:.4f}")
    print(f"          Recall = {val_recall:.4f}")
    
    # Just check fold 1 for now
    break

# 3. Compare architecture to your current FallNet
print("\n" + "="*80)
print("COMPARING TO CURRENT FALLNET")
print("="*80)

current_model = FallNet(input_shape=(200, 6), n_classes=6).build_ensemble()
print("\nCurrent model:")
current_model.summary()

# 4. Check for key differences
print("\n" + "="*80)
print("KEY CHECKS")
print("="*80)
print(f"Best model input shape: {best_model.input_shape}")
print(f"Best model output shape: {best_model.output_shape}")
print(f"Best model total params: {best_model.count_params():,}")
print(f"\nCurrent model input shape: {current_model.input_shape}")
print(f"Current model output shape: {current_model.output_shape}")
print(f"Current model total params: {current_model.count_params():,}")

# 5. Layer-by-layer comparison
print("\n" + "="*80)
print("LAYER COMPARISON")
print("="*80)
print(f"\nBest model has {len(best_model.layers)} layers")
print(f"Current model has {len(current_model.layers)} layers")

print("\nBest model layers:")
for i, layer in enumerate(best_model.layers[:10]):  # First 10 layers
    print(f"  {i}: {layer.name} - {layer.__class__.__name__}")

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers
import gc



keras.backend.clear_session()
gc.collect()

class FallNet:
    """
    FallNet: CNN-LSTM Ensemble (EXACT replication of paper)
    Reference: Jain & Semwal, IEEE Sensors Journal 2022
    """
    
    def __init__(self, input_shape=(200, 6), n_classes=6):  # 8 classes per paper
        self.input_shape = input_shape
        self.n_classes = n_classes
        self.model = None
    
    def build_lstm_branch(self, inputs):
        """LSTM Branch - EXACT from paper Table II"""
        x = layers.LSTM(
            units=256,
            activation='tanh',
            return_sequences=False,
            name='lstm_layer'
        )(inputs)
        
        x = layers.Dense(128, activation='relu', 
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        name='lstm_dense1')(x)
        x = layers.Dropout(0.2, name='lstm_dropout1')(x)
        
        x = layers.Dense(64, activation='relu',
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        name='lstm_dense2')(x)
        x = layers.Dropout(0.2, name='lstm_dropout2')(x)
        
        x = layers.Dense(32, activation='relu',
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        name='lstm_dense3')(x)
        x = layers.Dropout(0.2, name='lstm_dropout3')(x)
        
        lstm_output = layers.Dense(
            self.n_classes, 
            activation='softmax',
            name='lstm_output'
        )(x)
        
        return lstm_output
    
    def build_cnn_branch(self, inputs):
        """CNN Branch - EXACT from paper Table II"""
        # Single Conv1D layer (paper specifies ONE, not two!)
        x = layers.Conv1D(
            filters=128,
            kernel_size=3,
            activation='relu',
            padding='same',
            name='conv1d_layer'
        )(inputs)
        
        x = layers.MaxPooling1D(pool_size=2, name='maxpool_layer')(x)
        x = layers.Flatten(name='flatten_layer')(x)
        
        # Large dense layers (CRITICAL for 97% accuracy)
        x = layers.Dense(1024, activation='relu',
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        name='cnn_dense1')(x)
        x = layers.Dropout(0.2, name='cnn_dropout1')(x)
        
        x = layers.Dense(512, activation='relu',
                        kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                        name='cnn_dense2')(x)
        x = layers.Dropout(0.2, name='cnn_dropout2')(x)
        
        cnn_output = layers.Dense(
            self.n_classes,
            activation='softmax',
            name='cnn_output'
        )(x)
        
        return cnn_output
    
    def build_ensemble(self):
        """Build ensemble - average of CNN + LSTM"""
        inputs = layers.Input(shape=self.input_shape, name='input')
        
        lstm_output = self.build_lstm_branch(inputs)
        cnn_output = self.build_cnn_branch(inputs)
        
        # Average ensemble
        ensemble_output = layers.Average(name='ensemble_average')([lstm_output, cnn_output])
        
        self.model = models.Model(
            inputs=inputs,
            outputs=ensemble_output,
            name='FallNet_CNN_LSTM'
        )
        
        return self.model
    
    def compile_model(self):
        """Compile with Adam optimizer (default LR per paper)"""
        if self.model is None:
            raise ValueError("Model not built yet")
        
        self.model.compile(
            optimizer=keras.optimizers.Adam(),  # Default LR = 0.001
            loss='categorical_crossentropy',
            metrics=[
                'accuracy', 
                keras.metrics.Precision(name='precision'),
                keras.metrics.Recall(name='recall')
            ]
        )
        
        return self.model

# ==========================================
# TRAINING CONFIGURATION (EXACT from paper)
# ==========================================

BATCH_SIZE = 64  # Start here (was 512)
EPOCHS = 200 # ← Paper uses 200
K_FOLDS = 5
MODEL_NAME = "cnn_lstm_ensemble"

# Training loop (with memory management)
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=K_FOLDS, shuffle=True, random_state=42)
fold_results = []
fold_histories = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X_data, y_labels), 1):
    print(f"\n{'='*80}")
    print(f"FOLD {fold}/{K_FOLDS} - CNN-LSTM ENSEMBLE")
    print(f"{'='*80}")
    
    # Clear memory
    keras.backend.clear_session()
    gc.collect()
    
    # Split data
    X_train, X_val = X_data[train_idx], X_data[val_idx]
    y_train, y_val = y_categorical[train_idx], y_categorical[val_idx]
    
    print(f"Train: {X_train.shape[0]:,} samples | Val: {X_val.shape[0]:,} samples")
    
    # Build model
    fallnet_fold = FallNet(input_shape=(200, 6), n_classes=6)  # 8 classes!
    model_fold = fallnet_fold.build_ensemble()
    model_fold = fallnet_fold.compile_model()
    
    # Callbacks (EXACT from paper)
    fold_callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=20,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=10,
            min_lr=1e-7,
            verbose=1
        ),
        ModelCheckpoint(
            filepath=str(output_dir / f'{MODEL_NAME}_fold_{fold}.keras'),
            monitor='val_accuracy',
            save_best_only=True,
            mode='max',
            verbose=1
        )
    ]
    
    # Class weights
    from sklearn.utils.class_weight import compute_class_weight
    class_weights_array = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(y_labels),
        y=y_labels
    )
    MAX_WEIGHT = 3.0
    class_weights_array_capped = np.clip(class_weights_array, None, MAX_WEIGHT)
    class_weights = dict(enumerate(class_weights_array_capped))
    
    # Train with BATCH_SIZE=512
    print(f"\nTraining fold {fold}...")
    history = model_fold.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        batch_size=BATCH_SIZE,  # 512!
        epochs=EPOCHS,          # 200!
        class_weight=class_weights,
        callbacks=fold_callbacks,
        verbose=2
    )
    
    # Evaluate
    val_loss, val_acc, val_precision, val_recall = model_fold.evaluate(
        X_val, y_val, batch_size=BATCH_SIZE, verbose=0
    )
    val_f1 = 2 * (val_precision * val_recall) / (val_precision + val_recall) if (val_precision + val_recall) > 0 else 0
    
    print(f"\n{'='*50}")
    print(f"Fold {fold} Results:")
    print(f"{'='*50}")
    print(f"Loss:      {val_loss:.4f}")
    print(f"Accuracy:  {val_acc:.4f}")
    print(f"Precision: {val_precision:.4f}")
    print(f"Recall:    {val_recall:.4f}")
    print(f"F1-Score:  {val_f1:.4f}")
    
    fold_results.append({
        'fold': fold,
        'val_loss': val_loss,
        'val_accuracy': val_acc,
        'val_precision': val_precision,
        'val_recall': val_recall,
        'val_f1': val_f1
    })
    
    fold_histories.append(history.history)
    
    # Clear memory
    del model_fold, fallnet_fold
    del X_train, X_val, y_train, y_val, history
    keras.backend.clear_session()
    gc.collect()
    
    print(f"\n✓ Fold {fold} complete, memory cleared")

In [ ]:
#from tensorflow import keras
import numpy as np
from sklearn.model_selection import StratifiedKFold
from pathlib import Path

# Paths
base_dir = Path.home() / 'repos/summerschool2023/projects/fall-detection/fall_detection_data'
model_dir = base_dir / 'models'

# Load data (assuming you have X_data, y_labels, y_categorical loaded)
# If not, load them first

# Setup same K-fold split used during training
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("\n" + "="*80)
print("EVALUATING TRAINED CNN-LSTM MODELS")
print("="*80)

fold_results = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X_data, y_labels), 1):
    print(f"\nEvaluating Fold {fold}...")
    
    # Load model
    model_path = model_dir / f'fallnet_fold_cnn_lstm_original{fold}.keras'
    model = keras.models.load_model(model_path)
    
    # Get validation data for this fold
    X_val = X_data[val_idx]
    y_val = y_categorical[val_idx]
    
    # Evaluate
    val_loss, val_acc, val_precision, val_recall = model.evaluate(
        X_val, y_val, batch_size=64, verbose=0
    )
    
    val_f1 = 2 * (val_precision * val_recall) / (val_precision + val_recall) if (val_precision + val_recall) > 0 else 0
    
    print(f"  Accuracy:  {val_acc:.4f}")
    print(f"  Precision: {val_precision:.4f}")
    print(f"  Recall:    {val_recall:.4f}")
    print(f"  F1-Score:  {val_f1:.4f}")
    
    fold_results.append({
        'fold': fold,
        'accuracy': val_acc,
        'precision': val_precision,
        'recall': val_recall,
        'f1': val_f1
    })

print("\n" + "="*80)
print("SUMMARY")
print("="*80)

import pandas as pd
df = pd.DataFrame(fold_results)

print("\nPer-Fold Results:")
print(df.to_string(index=False))

print("\n" + "="*80)
print("AVERAGE PERFORMANCE")
print("="*80)

mean_acc = df['accuracy'].mean()
std_acc = df['accuracy'].std()
mean_recall = df['recall'].mean()
std_recall = df['recall'].std()

print(f"Accuracy:  {mean_acc:.4f} ± {std_acc:.4f} ({mean_acc*100:.2f}%)")
print(f"Precision: {df['precision'].mean():.4f} ± {df['precision'].std():.4f}")
print(f"Recall:    {mean_recall:.4f} ± {std_recall:.4f}")
print(f"F1-Score:  {df['f1'].mean():.4f} ± {df['f1'].std():.4f}")

print("\n" + "="*80)
print("COMPARISON TO TARGET")
print("="*80)
print(f"Target (Paper):    97.52%")
print(f"Your Result:       {mean_acc*100:.2f}%")
print(f"Difference:        {(mean_acc - 0.9752)*100:+.2f}%")

if mean_acc >= 0.97:
    print("\n✓ SUCCESS: Achieved target performance!")
elif mean_acc >= 0.95:
    print("\n→ CLOSE: Within 2-3% of target")
else:
    print("\n⚠️  GAP: More than 2% below target")

In [ ]:
# Load best fold model
best_fold = int(results_df.loc[results_df['val_f1'].idxmax(), 'fold'])
best_model = keras.models.load_model(output_dir / f'fallnet_fold_{best_fold}.keras')

# Get predictions on ALL data
y_pred_probs = best_model.predict(X_data, verbose=0)
y_pred = np.argmax(y_pred_probs, axis=1)

# Classification report
from sklearn.metrics import classification_report
class_names = [reverse_label_map[i] for i in range(6)]
print("\n" + "="*80)
print("PER-CLASS PERFORMANCE (BEST FOLD)")
print("="*80)
print(classification_report(y_labels, y_pred, target_names=class_names, digits=4))

# Fall_Initiation specific
fall_init_idx = 4
fall_init_recall = recall_score(y_labels == fall_init_idx, y_pred == fall_init_idx)
fall_init_precision = precision_score(y_labels == fall_init_idx, y_pred == fall_init_idx)
fall_init_f1 = f1_score(y_labels == fall_init_idx, y_pred == fall_init_idx)

print("\n" + "="*80)
print("FALL_INITIATION PERFORMANCE (CRITICAL CLASS)")
print("="*80)
print(f"Precision: {fall_init_precision:.4f}")
print(f"Recall:    {fall_init_recall:.4f}")
print(f"F1-Score:  {fall_init_f1:.4f}")
print(f"\nPaper's Fall_Initiation Recall: 0.9924")
print(f"Your Fall_Initiation Recall:    {fall_init_recall:.4f}")
print(f"Difference:                     {fall_init_recall - 0.9924:+.4f}")

In [ ]:
import numpy as np
import json
from pathlib import Path
from tensorflow import keras
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score

base_dir = Path("~/repos/summerschool2023/projects/fall-detection/fall_detection_data").expanduser()
models_dir = base_dir / "models"
processed_dir = base_dir / "processed"

print("="*80)
print("EVALUATING fallnet_fold_1.keras (Jan 5, 14:39)")
print("="*80)

# Load 6-class data
X_data = np.load(processed_dir / "X_data.npy")
y_labels = np.load(processed_dir / "y_labels.npy")

print(f"\n📊 Data loaded:")
print(f"   X_data: {X_data.shape}")
print(f"   y_labels: {y_labels.shape}")
print(f"   Classes: {sorted(np.unique(y_labels))}")

# Load label map
with open(processed_dir / "label_map.json", "r") as f:
    label_map = json.load(f)
reverse_label_map = {v: k for k, v in label_map.items()}

class_names = [reverse_label_map[i] for i in range(6)]

print(f"\n📋 Classes:")
for idx, name in enumerate(class_names):
    print(f"   {idx}: {name}")

# Load model
print(f"\n🔧 Loading model...")
model = keras.models.load_model(models_dir / 'fallnet_fold_1.keras')

print(f"✅ Model loaded successfully")
print(f"   Input shape: {model.input_shape}")
print(f"   Output shape: {model.output_shape}")
print(f"   Total params: {model.count_params():,}")

# Make predictions
print(f"\n🔮 Making predictions on {len(X_data):,} samples...")
y_pred_probs = model.predict(X_data, verbose=0)
y_pred = np.argmax(y_pred_probs, axis=1)

# Overall accuracy
overall_acc = (y_pred == y_labels).mean()

print(f"\n" + "="*80)
print(f"OVERALL ACCURACY: {overall_acc:.4f} ({overall_acc*100:.2f}%)")
print(f"="*80)

# Classification Report
print(f"\n" + "="*80)
print("CLASSIFICATION REPORT")
print("="*80)
print(classification_report(y_labels, y_pred, target_names=class_names, digits=4))

# Confusion Matrix
cm = confusion_matrix(y_labels, y_pred)

print(f"\n" + "="*80)
print("CONFUSION MATRIX")
print("="*80)
print(f"{'True \\ Pred':<25s}", end="")
for name in class_names:
    print(f"{name[:10]:>10s}", end="")
print()

for i, true_name in enumerate(class_names):
    print(f"{true_name[:25]:<25s}", end="")
    for j in range(6):
        print(f"{cm[i,j]:>10d}", end="")
    total = cm[i].sum()
    correct = cm[i,i]
    acc_row = correct / total if total > 0 else 0
    print(f"  | {correct:>5d}/{total:<5d} ({acc_row*100:>5.1f}%)")

# Per-class metrics
print(f"\n" + "="*80)
print("PER-CLASS METRICS")
print("="*80)
print(f"{'Class':<30s} {'Precision':<12s} {'Recall':<12s} {'F1-Score':<12s} {'Support'}")
print("-"*90)

for cls_idx in range(6):
    precision = precision_score(y_labels == cls_idx, y_pred == cls_idx, zero_division=0)
    recall = recall_score(y_labels == cls_idx, y_pred == cls_idx, zero_division=0)
    f1 = f1_score(y_labels == cls_idx, y_pred == cls_idx, zero_division=0)
    support = (y_labels == cls_idx).sum()
    
    status = "✅" if recall > 0.90 else "⚠️" if recall > 0.80 else "❌"
    print(f"{class_names[cls_idx]:<30s} {precision:<12.4f} {recall:<12.4f} {f1:<12.4f} {support:>7d} {status}")

# Fall_Initiation (Critical Class)
fall_init_idx = label_map["Fall_Initiation"]
fall_init_precision = precision_score(y_labels == fall_init_idx, y_pred == fall_init_idx)
fall_init_recall = recall_score(y_labels == fall_init_idx, y_pred == fall_init_idx)
fall_init_f1 = f1_score(y_labels == fall_init_idx, y_pred == fall_init_idx)

print(f"\n" + "="*80)
print("FALL_INITIATION (CRITICAL CLASS)")
print("="*80)
print(f"Precision: {fall_init_precision:.4f} ({fall_init_precision*100:.2f}%)")
print(f"Recall:    {fall_init_recall:.4f} ({fall_init_recall*100:.2f}%)")
print(f"F1-Score:  {fall_init_f1:.4f} ({fall_init_f1*100:.2f}%)")
print(f"\n📊 Comparison with Paper:")
print(f"   Paper:  99.24% recall")
print(f"   Yours:  {fall_init_recall*100:.2f}% recall")
print(f"   Diff:   {(fall_init_recall - 0.9924)*100:+.2f}%")

# Final Summary
print(f"\n" + "="*80)
print("SUMMARY")
print("="*80)

if overall_acc > 0.95:
    print(f"🎉 EXCELLENT! {overall_acc*100:.2f}% accuracy - Publication quality!")
elif overall_acc > 0.85:
    print(f"✅ GOOD! {overall_acc*100:.2f}% accuracy - Solid baseline for SNN comparison!")
elif overall_acc > 0.70:
    print(f"🤔 MODERATE: {overall_acc*100:.2f}% - Matches K-fold expectations")
else:
    print(f"❌ POOR: {overall_acc*100:.2f}% - This might be the wrong model")

print(f"\n📁 Model used: fallnet_fold_1.keras")
print(f"📅 Saved: Jan 5, 2025 at 14:39")


In [ ]:
# %% [markdown]
## First Epoch Performance Analysis

# %%
print("\n" + "="*80)
print("ANALYZING FIRST EPOCH RESULTS")
print("="*80)

# You need to have saved the fold_histories from training
# If you have them:

for fold in range(1, 6):
    if fold-1 < len(fold_histories):
        history = fold_histories[fold-1]
        
        print(f"\n{'='*50}")
        print(f"FOLD {fold} - FIRST EPOCH")
        print(f"{'='*50}")
        print(f"Train Loss:      {history['loss'][0]:.4f}")
        print(f"Train Accuracy:  {history['accuracy'][0]:.4f}")
        print(f"Val Loss:        {history['val_loss'][0]:.4f}")
        print(f"Val Accuracy:    {history['val_accuracy'][0]:.4f}")
        
        # Check for warning signs
        if history['val_loss'][0] > 2.0:
            print("⚠️  WARNING: Very high initial loss!")
        if history['val_accuracy'][0] < 0.20:
            print("⚠️  WARNING: Worse than random guessing!")

In [ ]:
# After training, add this cell:
import numpy as np
from sklearn.metrics import confusion_matrix
import pandas as pd

# Get predictions on all data
best_model = keras.models.load_model(output_dir / f'fallnet_fold_{best_fold}.keras')
y_pred_probs = best_model.predict(X_data, verbose=0)
y_pred = np.argmax(y_pred_probs, axis=1)

# Create confusion matrix
cm = confusion_matrix(y_labels, y_pred)

# Create DataFrame for better visualization
class_names = [reverse_label_map[i] for i in range(8)]
cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)

print("\n" + "="*80)
print("CONFUSION MATRIX (Counts)")
print("="*80)
print(cm_df)

# Normalized version (percentages)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
cm_normalized_df = pd.DataFrame(cm_normalized, index=class_names, columns=class_names)

print("\n" + "="*80)
print("CONFUSION MATRIX (Percentages - Row-wise)")
print("="*80)
print(cm_normalized_df.round(3))

# Check prediction distribution
print("\n" + "="*80)
print("PREDICTION DISTRIBUTION")
print("="*80)
pred_counts = pd.Series(y_pred).value_counts().sort_index()
for cls_idx, count in pred_counts.items():
    pct = count / len(y_pred) * 100
    print(f"{reverse_label_map[cls_idx]:<40s} {count:>6d} ({pct:>5.1f}%)")

In [ ]:
import numpy as np
from pathlib import Path
from tensorflow import keras

base_dir = Path("~/repos/summerschool2023/projects/fall-detection/fall_detection_data").expanduser()
models_dir = base_dir / "models"
processed_dir = base_dir / "processed"

print("="*80)
print("COMPLETE DATA/MODEL AUDIT")
print("="*80)

# 1. Check what's in processed data on disk
print("\n📊 PROCESSED DATA ON DISK:")
y_disk = np.load(processed_dir / "y_labels.npy")
X_disk = np.load(processed_dir / "X_data.npy")

print(f"X_data.npy:")
print(f"  Shape: {X_disk.shape}")
print(f"y_labels.npy:")
print(f"  Shape: {y_disk.shape}")
print(f"  Classes: {sorted(np.unique(y_disk))}")
print(f"  Number of classes: {len(np.unique(y_disk))}")
print(f"  Range: {y_disk.min()}-{y_disk.max()}")

if len(np.unique(y_disk)) == 8:
    print("\n❌ DISK DATA: 8 classes (0-7) - ORIGINAL preprocessed data")
elif len(np.unique(y_disk)) == 6:
    print("\n✅ DISK DATA: 6 classes (0-5) - CLEANED data")
else:
    print(f"\n⚠️  DISK DATA: {len(np.unique(y_disk))} classes - UNEXPECTED!")

# 2. Check what the models expect
print("\n🤖 MODEL EXPECTATIONS:")
model = keras.models.load_model(models_dir / 'fallnet_fold_1.keras')
print(f"Model input shape: {model.input_shape}")
print(f"Model output shape: {model.output_shape}")
print(f"Model expects {model.output_shape[-1]} classes")

# 3. Try a prediction to see what happens
print("\n🔮 TEST PREDICTION:")
try:
    y_pred_probs = model.predict(X_disk[:10], verbose=0)
    print(f"✅ Prediction successful!")
    print(f"   Prediction shape: {y_pred_probs.shape}")
    print(f"   Model outputs: {y_pred_probs.shape[-1]} class probabilities")
    
    # Check what classes it's actually predicting
    y_pred = np.argmax(y_pred_probs, axis=1)
    print(f"   Predicted classes (first 10): {y_pred}")
    print(f"   Actual classes (first 10):    {y_disk[:10]}")
except Exception as e:
    print(f"❌ Prediction failed: {e}")

# 4. The verdict
print("\n" + "="*80)
print("DIAGNOSIS")
print("="*80)

if len(np.unique(y_disk)) == model.output_shape[-1]:
    print(f"✅ MATCH: Data has {len(np.unique(y_disk))} classes, model outputs {model.output_shape[-1]} classes")
    print("\n   The model SHOULD work with this data!")
    print("   If evaluation fails, something else is wrong.")
else:
    print(f"❌ MISMATCH!")
    print(f"   Data has {len(np.unique(y_disk))} classes (range: {y_disk.min()}-{y_disk.max()})")
    print(f"   Model expects {model.output_shape[-1]} classes")
    print(f"\n   This is why evaluation fails!")
    
    if len(np.unique(y_disk)) == 8 and model.output_shape[-1] == 6:
        print("\n   SOLUTION: Your models were trained on 6-class data IN MEMORY")
        print("   But y_labels.npy on disk still has 8 classes!")
        print("   You need to save the 6-class data to disk.")

In [ ]:
# %% [markdown]
# # Data Loading and Preprocessing
# Load preprocessed data, merge classes, remove Fall_Recovery

# %%
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from pathlib import Path
from tensorflow import keras

# Setup paths
base_dir = Path("~/repos/summerschool2023/projects/fall-detection/fall_detection_data").expanduser()
processed_dir = base_dir / "processed"
output_dir = base_dir / "models"
output_dir.mkdir(exist_ok=True)

print("="*80)
print("LOADING AND PREPROCESSING DATA")
print("="*80)

# %% Load the newly processed data
X_data = np.load(processed_dir / "X_data.npy")
y_labels = np.load(processed_dir / "y_labels.npy")

print(f"\nOriginal data loaded:")
print(f"  X_data shape: {X_data.shape}")
print(f"  y_labels shape: {y_labels.shape}")

# ============================================================================
# STEP 1: Merge Impact and Aftermath
# ============================================================================
print("\n" + "="*80)
print("STEP 1: MERGING IMPACT AND AFTERMATH")
print("="*80)

counts_before_merge = Counter(y_labels)
print(f"Before merge:")
print(f"  Impact (6): {counts_before_merge[6]} samples")
print(f"  Aftermath (7): {counts_before_merge[7]} samples")

y_labels[y_labels == 7] = 6  # Change Aftermath (7) to Impact (6)

counts_after_merge = Counter(y_labels)
print(f"\nAfter merge:")
print(f"  Impact_Aftermath (6): {counts_after_merge[6]} samples")

# ============================================================================
# STEP 2: Remove Fall_Recovery
# ============================================================================
print("\n" + "="*80)
print("STEP 2: REMOVING FALL_RECOVERY CLASS")
print("="*80)

# Show before
counts_before = Counter(y_labels)
print(f"\nBefore removal:")
print(f"  Total samples: {len(y_labels):,}")
print(f"  Fall_Recovery (class 4): {counts_before[4]} samples")

# Remove Fall_Recovery (class 4)
mask = y_labels != 4
X_data = X_data[mask]
y_labels_temp = y_labels[mask]

removed_count = (~mask).sum()
print(f"\n✅ Removed {removed_count} Fall_Recovery samples")

# Shift labels down (5→4, 6→5)
y_labels = y_labels_temp.copy()
y_labels[y_labels_temp > 4] -= 1  # Classes 5,6 become 4,5

print(f"\nAfter removal:")
print(f"  Total samples: {len(y_labels):,}")
print(f"  Removed: {removed_count} samples ({removed_count/(len(y_labels)+removed_count)*100:.2f}%)")

# ============================================================================
# STEP 3: Update label map (NOW 6 CLASSES: 0-5)
# ============================================================================
print("\n" + "="*80)
print("STEP 3: CREATING 6-CLASS LABEL MAP")
print("="*80)

label_map = {
    'Walking': 0,
    'Jogging': 1,
    'Walking_stairs_updown': 2,
    'Stumble_while_walking': 3,
    'Fall_Initiation': 4,      # Was 5, now 4
    'Impact_Aftermath': 5,     # Was 6, now 5
}
reverse_label_map = {v: k for k, v in label_map.items()}

print(f"\n✅ Updated to 6 classes (0-5):")
for name, idx in sorted(label_map.items(), key=lambda x: x[1]):
    print(f"  Class {idx}: {name}")

# ============================================================================
# STEP 4: Create categorical labels
# ============================================================================
print("\n" + "="*80)
print("STEP 4: CREATING CATEGORICAL LABELS")
print("="*80)

y_categorical = keras.utils.to_categorical(y_labels, num_classes=6)

print(f"\n✅ y_categorical created:")
print(f"   Shape: {y_categorical.shape}")
print(f"   Expected: ({len(y_labels)}, 6)")

# Verification
assert X_data.shape[0] == y_labels.shape[0] == y_categorical.shape[0], \
    "Data shapes don't match!"

print(f"\n✅ All data aligned:")
print(f"   X_data:        {X_data.shape}")
print(f"   y_labels:      {y_labels.shape}")
print(f"   y_categorical: {y_categorical.shape}")
print(f"   All have {X_data.shape[0]:,} samples")

# ============================================================================
# DIAGNOSTICS
# ============================================================================
print("\n" + "="*80)
print("DATA QUALITY DIAGNOSTICS")
print("="*80)

# 1. Class distribution
class_counts = Counter(y_labels)
print("\n1. Class Distribution (6 classes):")
for cls_idx in sorted(class_counts.keys()):
    count = class_counts[cls_idx]
    pct = count / len(y_labels) * 100
    print(f"   Class {cls_idx} ({reverse_label_map[cls_idx]:30s}): {count:5d} ({pct:5.2f}%)")

# Calculate imbalance
max_count = max(class_counts.values())
min_count = min(class_counts.values())
print(f"\nImbalance ratio: {max_count/min_count:.2f}x")
print(f"  (was 36.8x with Fall_Recovery)")

# 2. Per-class signal statistics
print("\n2. Per-Class Signal Statistics (Acc-Y axis):")
print(f"   {'Class':<35s} {'Mean':<10s} {'Std':<10s} {'Min':<10s} {'Max':<10s}")
print(f"   {'-'*75}")
for cls_idx in sorted(class_counts.keys()):
    class_samples = X_data[y_labels == cls_idx]
    acc_y = class_samples[:, :, 1]  # Y-axis acceleration
    
    mean_val = acc_y.mean()
    std_val = acc_y.std()
    min_val = acc_y.min()
    max_val = acc_y.max()
    
    print(f"   {reverse_label_map[cls_idx]:<35s} {mean_val:>8.4f}  {std_val:>8.4f}  {min_val:>8.2f}  {max_val:>8.2f}")

# 3. Variance ranking
print("\n3. Variance Ranking (Fall_Initiation should be #1):")
variances = []
for cls_idx in sorted(class_counts.keys()):
    class_samples = X_data[y_labels == cls_idx]
    acc_y_var = class_samples[:, :, 1].var()
    variances.append((reverse_label_map[cls_idx], acc_y_var, cls_idx))
variances.sort(key=lambda x: x[1], reverse=True)
for i, (name, var, idx) in enumerate(variances, 1):
    print(f"   {i}. {name:<35s}: {var:.4f}")

# 4. Visualize samples
fig, axes = plt.subplots(3, 2, figsize=(15, 10))
axes = axes.flatten()
critical_classes = [
    label_map['Walking'],
    label_map['Fall_Initiation'],
    label_map['Impact_Aftermath'],
    label_map['Stumble_while_walking'],
    label_map['Jogging'],
    label_map['Walking_stairs_updown']
]

for i, cls_idx in enumerate(critical_classes):
    if cls_idx in class_counts:
        sample_idx = np.where(y_labels == cls_idx)[0][0]
        sample_data = X_data[sample_idx]
        
        time = np.arange(200) / 200
        axes[i].plot(time, sample_data[:, 0], label='Acc-X', alpha=0.7, linewidth=1)
        axes[i].plot(time, sample_data[:, 1], label='Acc-Y', alpha=0.7, linewidth=1)
        axes[i].plot(time, sample_data[:, 2], label='Acc-Z', alpha=0.7, linewidth=1)
        
        axes[i].set_title(f'{reverse_label_map[cls_idx]}', fontsize=11, fontweight='bold')
        axes[i].set_xlabel('Time (s)')
        axes[i].set_ylabel('Normalized Acc')
        axes[i].legend(fontsize=8)
        axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("✅ DATA READY FOR TRAINING (6 CLASSES)")
print("="*80)

In [ ]:
import json

print("\n" + "="*80)
print("SAVING 6-CLASS DATA TO DISK")
print("="*80)

# Save 6-class data with different names (preserve 8-class original)
np.save(processed_dir / "X_data_6class.npy", X_data)
np.save(processed_dir / "y_labels_6class.npy", y_labels)

# Save 6-class label map
with open(processed_dir / "label_map_6class.json", 'w') as f:
    json.dump(label_map, f, indent=2)

print(f"\n✅ Saved 6-class data:")
print(f"   📄 X_data_6class.npy: {X_data.shape}")
print(f"   📄 y_labels_6class.npy: {y_labels.shape}")
print(f"   📄 label_map_6class.json")

print(f"\n✅ Original 8-class data preserved:")
print(f"   📄 X_data.npy: (16891, 200, 6) - untouched")
print(f"   📄 y_labels.npy: (16891,) - untouched")
print(f"   📄 label_map.json - untouched")

print("\n" + "="*80)
print("NOW YOU CAN EVALUATE YOUR MODELS!")
print("="*80)
print("\nTo load 6-class data in future sessions:")
print("   X_data = np.load(processed_dir / 'X_data_6class.npy')")
print("   y_labels = np.load(processed_dir / 'y_labels_6class.npy')")
print("\nTo load 8-class data (original):")
print("   X_data = np.load(processed_dir / 'X_data.npy')")
print("   y_labels = np.load(processed_dir / 'y_labels.npy')")

In [ ]:
# %% [markdown]
# # Evaluate FallNet Model (6-Class Data)

# %%
import numpy as np
import json
from pathlib import Path
from tensorflow import keras
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
from collections import Counter

# ============================================================================
# Setup Paths
# ============================================================================
base_dir = Path("~/repos/summerschool2023/projects/fall-detection/fall_detection_data").expanduser()
processed_dir = base_dir / "processed"
models_dir = base_dir / "models"

print("="*80)
print("LOADING 6-CLASS DATA AND MODEL")
print("="*80)

# ============================================================================
# Load 6-Class Data
# ============================================================================
X_data = np.load(processed_dir / "X_data_6class.npy")
y_labels = np.load(processed_dir / "y_labels_6class.npy")

print(f"\n📊 Data loaded:")
print(f"   X_data: {X_data.shape}")
print(f"   y_labels: {y_labels.shape}")
print(f"   Unique classes: {sorted(np.unique(y_labels))}")
print(f"   Number of classes: {len(np.unique(y_labels))}")

# ============================================================================
# Load 6-Class Label Map
# ============================================================================
with open(processed_dir / "label_map_6class.json", "r") as f:
    label_map = json.load(f)

reverse_label_map = {v: k for k, v in label_map.items()}
class_names = [reverse_label_map[i] for i in range(6)]

print(f"\n📋 Class names:")
for idx, name in enumerate(class_names):
    count = (y_labels == idx).sum()
    pct = count / len(y_labels) * 100
    print(f"   {idx}: {name:<30s} ({count:>5,} samples, {pct:>5.2f}%)")

# ============================================================================
# Load Model
# ============================================================================
print(f"\n🔧 Loading model...")
model_path = models_dir / 'fallnet_fold_1.keras'

if not model_path.exists():
    print(f"\n❌ ERROR: Model not found at {model_path}")
    print("Available models:")
    for f in models_dir.glob("*.keras"):
        print(f"  - {f.name}")
else:
    model = keras.models.load_model(model_path)
    print(f"✅ Model loaded: {model_path.name}")
    print(f"   Input shape: {model.input_shape}")
    print(f"   Output shape: {model.output_shape}")
    print(f"   Total params: {model.count_params():,}")

# ============================================================================
# Verify Compatibility
# ============================================================================
print("\n" + "="*80)
print("COMPATIBILITY CHECK")
print("="*80)

n_classes_data = len(np.unique(y_labels))
n_classes_model = model.output_shape[-1]

print(f"Data classes: {n_classes_data}")
print(f"Model outputs: {n_classes_model}")

if n_classes_data == n_classes_model:
    print(f"✅ MATCH! Data and model are compatible!")
else:
    print(f"❌ MISMATCH! Cannot evaluate!")
    raise ValueError(f"Data has {n_classes_data} classes but model expects {n_classes_model}")

# ============================================================================
# Make Predictions
#      ============================================================================
print("\n" + "="*80)
print("MAKING PREDICTIONS")
print("="*80)

print(f"Predicting on {len(X_data):,} samples...")
y_pred_probs = model.predict(X_data, verbose=1)
y_pred = np.argmax(y_pred_probs, axis=1)

print(f"\n✅ Predictions complete")
print(f"   Predicted classes: {sorted(np.unique(y_pred))}")

# ============================================================================
# Overall Accuracy
# ============================================================================
overall_acc = (y_pred == y_labels).mean()

print("\n" + "="*80)
print("OVERALL PERFORMANCE")
print("="*80)
print(f"Accuracy: {overall_acc:.4f} ({overall_acc*100:.2f}%)")

# ============================================================================
# Classification Report
# ============================================================================
print("\n" + "="*80)
print("CLASSIFICATION REPORT")
print("="*80)
print(classification_report(y_labels, y_pred, target_names=class_names, digits=4))

# ============================================================================
# Confusion Matrix
# ============================================================================
cm = confusion_matrix(y_labels, y_pred)

print("\n" + "="*80)
print("CONFUSION MATRIX")
print("="*80)
print(f"{'True \\ Pred':<25s}", end="")
for name in class_names:
    print(f"{name[:10]:>10s}", end="")
print()

for i, true_name in enumerate(class_names):
    print(f"{true_name[:25]:<25s}", end="")
    for j in range(6):
        print(f"{cm[i,j]:>10d}", end="")
    total = cm[i].sum()
    correct = cm[i,i]
    acc_row = correct / total if total > 0 else 0
    print(f"  | {correct:>5d}/{total:<5d} ({acc_row*100:>5.1f}%)")

print()

# ============================================================================
# Per-Class Detailed Metrics
# ============================================================================
print("\n" + "="*80)
print("PER-CLASS DETAILED METRICS")
print("="*80)

print(f"\n{'Class':<30s} {'Precision':<12s} {'Recall':<12s} {'F1-Score':<12s} {'Support'}")
print("-"*90)

for cls_idx in range(6):
    precision = precision_score(y_labels == cls_idx, y_pred == cls_idx, zero_division=0)
    recall = recall_score(y_labels == cls_idx, y_pred == cls_idx, zero_division=0)
    f1 = f1_score(y_labels == cls_idx, y_pred == cls_idx, zero_division=0)
    support = (y_labels == cls_idx).sum()
    
    status = "✅" if recall > 0.90 else "⚠️" if recall > 0.80 else "❌"
    print(f"{class_names[cls_idx]:<30s} {precision:<12.4f} {recall:<12.4f} {f1:<12.4f} {support:>7d} {status}")

# ============================================================================
# Fall_Initiation Focus (Critical Class)
# ============================================================================
fall_init_idx = label_map["Fall_Initiation"]
fall_init_precision = precision_score(y_labels == fall_init_idx, y_pred == fall_init_idx)
fall_init_recall = recall_score(y_labels == fall_init_idx, y_pred == fall_init_idx)
fall_init_f1 = f1_score(y_labels == fall_init_idx, y_pred == fall_init_idx)

print("\n" + "="*80)
print("FALL_INITIATION PERFORMANCE (CRITICAL CLASS)")
print("="*80)
print(f"Precision: {fall_init_precision:.4f} ({fall_init_precision*100:.2f}%)")
print(f"Recall:    {fall_init_recall:.4f} ({fall_init_recall*100:.2f}%)")
print(f"F1-Score:  {fall_init_f1:.4f} ({fall_init_f1*100:.2f}%)")

print("\n📊 Comparison with Paper:")
print(f"   Paper (8 classes):  99.24% recall")
print(f"   Yours (6 classes):  {fall_init_recall*100:.2f}% recall")
print(f"   Difference:         {(fall_init_recall - 0.9924)*100:+.2f}%")

# ============================================================================
# Final Summary
# ============================================================================
print("\n" + "="*80)
print("FINAL SUMMARY")
print("="*80)

all_recalls_good = all(recall_score(y_labels == i, y_pred == i, zero_division=0) > 0.85 for i in range(6))

print(f"\n✅ Overall Accuracy:        {overall_acc:.4f} ({overall_acc*100:.2f}%)")
print(f"✅ Fall_Initiation Recall:  {fall_init_recall:.4f} ({fall_init_recall*100:.2f}%)")
print(f"✅ All classes >85% recall: {all_recalls_good}")

if overall_acc > 0.95:
    print("\n🎉 EXCELLENT! Model is performing at publication quality!")
    print("   Ready for Phase 2: SNN conversion!")
elif overall_acc > 0.85:
    print("\n✅ VERY GOOD! Solid baseline for SNN comparison!")
    print("   This matches your K-fold CV expectations (87% avg)!")
elif overall_acc > 0.70:
    print("\n🤔 MODERATE: Acceptable but room for improvement")
else:
    print("\n⚠️  LOWER THAN EXPECTED: May need investigation")

print(f"\n📁 Model evaluated: {model_path.name}")
print(f"📅 Model saved: Jan 5, 2026 at 18:39")
print(f"📊 Dataset: 6-class (16,732 samples)")

print("\n" + "="*80)

In [ ]:
from tensorflow import keras
import numpy as np

# Load 6-class data
X_data = np.load(processed_dir / "X_data_6class.npy")
y_labels = np.load(processed_dir / "y_labels_6class.npy")

# Create categorical labels
y_categorical = keras.utils.to_categorical(y_labels, num_classes=6)

print(f"✅ Created y_categorical: {y_categorical.shape}")
print(f"   From y_labels: {y_labels.shape}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import legendre

# Create a fall signal
t = np.linspace(-1, 1, 200)

# Simulate: Walking → Fall Initiation → Impact
signal = np.zeros(200)
signal[:66] = 0.5 + 0.1*np.sin(10*t[:66])  # Walking (oscillation)
signal[66:133] = 0.5 + 2.5*(t[66:133] + 0.33)**2  # Fall initiation (acceleration)
signal[133:] = 3.5 + 0.2*np.random.randn(67)  # Impact (high + noisy)

# Show what each coefficient captures
fig, axes = plt.subplots(4, 2, figsize=(14, 12))

# Original signal
axes[0, 0].plot(t, signal, 'b-', linewidth=2)
axes[0, 0].set_title('Original Fall Signal', fontweight='bold')
axes[0, 0].axvline(-0.33, color='r', linestyle='--', alpha=0.5, label='Fall starts')
axes[0, 0].axvline(0.33, color='orange', linestyle='--', alpha=0.5, label='Impact')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Show what each polynomial contributes
components_to_show = [0, 1, 2, 3, 5, 10, 20]
coeffs = np.polyfit(t, signal, deg=20)  # Simplified - use proper Legendre fitting

for idx, n in enumerate(components_to_show):
    ax = axes[(idx+1)//2, (idx+1)%2]
    
    # Get the nth Legendre polynomial
    P_n = legendre(n)
    
    # Compute coefficient (project signal onto this polynomial)
    c_n = np.trapz(signal * P_n(t), t) * (2*n + 1) / 2
    
    # Show the contribution of this polynomial
    contribution = c_n * P_n(t)
    
    ax.plot(t, P_n(t), 'gray', alpha=0.3, linewidth=1, label=f'P_{n}(t)')
    ax.plot(t, contribution, 'r-', linewidth=2, label=f'c_{n} · P_{n}(t)')
    ax.axhline(0, color='black', linewidth=0.5, linestyle='--', alpha=0.3)
    
    ax.set_title(f'Component {n}: c_{n} = {c_n:.3f}', fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Add interpretation
    if n == 0:
        ax.text(0.5, 0.95, 'Average level', transform=ax.transAxes, 
                ha='center', va='top', bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.3))
    elif n == 1:
        ax.text(0.5, 0.95, 'Overall trend', transform=ax.transAxes,
                ha='center', va='top', bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.3))
    elif n == 2:
        ax.text(0.5, 0.95, 'Curvature', transform=ax.transAxes,
                ha='center', va='top', bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.3))
    elif n == 3:
        ax.text(0.5, 0.95, 'S-curve pattern', transform=ax.transAxes,
                ha='center', va='top', bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.3))
    else:
        ax.text(0.5, 0.95, f'Fine detail ({n}th order)', transform=ax.transAxes,
                ha='center', va='top', bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.3))

plt.tight_layout()
plt.suptitle('What Each Legendre Coefficient Captures', fontsize=14, fontweight='bold', y=1.00)
plt.show()

# Print what each captures
print("="*80)
print("LEGENDRE COEFFICIENT INTERPRETATION")
print("="*80)
print("\nFor a fall detection signal:")
print(f"\nc₀ = {c_n:.3f}  → Average acceleration level")
print("            Large positive = high activity")
print("            Near zero = stationary")
print(f"\nc₁ → Trend direction")
print("            Positive = acceleration increasing over time")
print("            Negative = acceleration decreasing")
print(f"\nc₂ → Curvature/acceleration of acceleration")
print("            Large value = rapid change (like fall initiation!)")
print(f"\nc₃₋₁₀ → Medium temporal features")
print("            Captures rhythmic patterns (walking, stumbling)")
print(f"\nc₁₁₊ → Fine temporal details")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import legendre

x = np.linspace(-1, 1, 1000)

# Two EVEN polynomials
P0 = legendre(0)(x)
P2 = legendre(2)(x)
product_even = P0 * P2

# Two ODD polynomials
P1 = legendre(1)(x)
P3 = legendre(3)(x)
product_odd = P1 * P3

fig, axes = plt.subplots(2, 3, figsize=(15, 8))

# Even example: P₀ × P₂
axes[0, 0].plot(x, P0, 'b-', linewidth=2, label='P₀(x)')
axes[0, 0].set_title('P₀(x) - EVEN', fontweight='bold')
axes[0, 0].axhline(0, color='black', linewidth=0.5, alpha=0.3)
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].legend()

axes[0, 1].plot(x, P2, 'r-', linewidth=2, label='P₂(x)')
axes[0, 1].set_title('P₂(x) - EVEN', fontweight='bold')
axes[0, 1].axhline(0, color='black', linewidth=0.5, alpha=0.3)
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].legend()

axes[0, 2].plot(x, product_even, 'purple', linewidth=2, label='P₀ × P₂')
axes[0, 2].fill_between(x, 0, product_even, alpha=0.3, color='purple')
axes[0, 2].axhline(0, color='black', linewidth=0.5, alpha=0.3)
axes[0, 2].set_title('P₀ × P₂ - Product', fontweight='bold')
axes[0, 2].grid(True, alpha=0.3)
axes[0, 2].legend()

# Calculate integral
integral_even = np.trapz(product_even, x)
axes[0, 2].text(0.5, 0.95, f'∫ = {integral_even:.6f} ≈ 0', 
                transform=axes[0, 2].transAxes, ha='center', va='top',
                bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))

# Odd example: P₁ × P₃
axes[1, 0].plot(x, P1, 'g-', linewidth=2, label='P₁(x)')
axes[1, 0].set_title('P₁(x) - ODD', fontweight='bold')
axes[1, 0].axhline(0, color='black', linewidth=0.5, alpha=0.3)
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].legend()

axes[1, 1].plot(x, P3, 'orange', linewidth=2, label='P₃(x)')
axes[1, 1].set_title('P₃(x) - ODD', fontweight='bold')
axes[1, 1].axhline(0, color='black', linewidth=0.5, alpha=0.3)
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].legend()

axes[1, 2].plot(x, product_odd, 'brown', linewidth=2, label='P₁ × P₃')
axes[1, 2].fill_between(x, 0, product_odd, alpha=0.3, color='brown')
axes[1, 2].axhline(0, color='black', linewidth=0.5, alpha=0.3)
axes[1, 2].set_title('P₁ × P₃ - Product', fontweight='bold')
axes[1, 2].grid(True, alpha=0.3)
axes[1, 2].legend()

# Calculate integral
integral_odd = np.trapz(product_odd, x)
axes[1, 2].text(0.5, 0.95, f'∫ = {integral_odd:.6f} ≈ 0', 
                transform=axes[1, 2].transAxes, ha='center', va='top',
                bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))

plt.suptitle('Orthogonality: Same Parity Polynomials', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("="*80)
print("KEY OBSERVATION")
print("="*80)
print("\nNotice the product P₀ × P₂ (both EVEN):")
print("  - Has both positive and negative areas")
print("  - Areas cancel out when integrated")
print("  - Total integral ≈ 0")
print("\nSame for P₁ × P₃ (both ODD):")
print("  - Product has balanced positive/negative regions")
print("  - Integral ≈ 0")
print("\n ALL pairs are orthogonal, not just odd-even!")
```

---

##  The Mathematical Reason:

Legendre polynomials are constructed using the **Gram-Schmidt orthogonalization process**:
```
Start  monomials: 1, x, x², x³, x⁴, ...

P₀ = 1  (already normalized)

P₁ = x - ⟨x, P₀⟩·P₀  (make orthogonal to P₀)
   = x - 0
   = x

P₂ = x² - ⟨x², P₀⟩·P₀ - ⟨x², P₁⟩·P₁  (make orthogonal to P₀ AND P₁)
   = x² - (1/3)·1 - 0·x
   = (3x² - 1)/2  (after normalization)

P₃ = x³ - ⟨x³, P₀⟩·P₀ - ⟨x³, P₁⟩·P₁ - ⟨x³, P₂⟩·P₂
   = ... (orthogonal to P₀, P₁, AND P₂!)

And so on...
```

**Each new polynomial is constructed to be orthogonal to ALL previous ones!**

---

## Complete Orthogonality Table:

Here's the mathematical statement:
```
∫₋₁¹ Pₙ(x)·Pₘ(x) dx = {  0           if n ≠ m
                        {  2/(2n+1)   if n = m
```

**Examples:**
```
∫₋₁¹ P₀·P₀ dx = 2/(2·0+1) = 2
∫₋₁¹ P₁·P₁ dx = 2/(2·1+1) = 2/3
∫₋₁¹ P₂·P₂ dx = 2/(2·2+1) = 2/5
∫₋₁¹ P₃·P₃ dx = 2/(2·3+1) = 2/7

∫₋₁¹ P₀·P₁ dx = 0  ← Different!
∫₋₁¹ P₀·P₂ dx = 0  ← Different!
∫₋₁¹ P₁·P₂ dx = 0  ← Different!
∫₋₁¹ P₂·P₃ dx = 0  ← Different!
∫₋₁¹ P₅·P₇ dx = 0  ← Different!

ALL pairs with n≠m give 0!
```

---

## 💡 Why This Matters for LMUs:

**Because ALL polynomials are orthogonal to each other:**

1. **Each coefficient is independent:**
```
   c₀ affects only P₀ contribution
   c₁ affects only P₁ contribution
   c₂ affects only P₂ contribution
   ...
   They don't interfere!
```

2. **Easy to extract coefficients:**
```
   cₙ = (2n+1)/2 · ∫₋₁¹ signal(t)·Pₙ(t) dt
   
   No need to worry about other polynomials!
```

3. **Clean memory updates:**
```
   Each coefficient evolves independently:
   dc₀/dt = a₀·c₀ + b₀·x(t)
   dc₁/dt = a₁·c₁ + b₁·x(t)
   dc₂/dt = a₂·c₂ + b₂·x(t)
   ...
```

**This independence is KEY to why LMUs are efficient!**

---

## Summary:

**Q: Are only odd-even pairs orthogonal?**

**A: NO! ALL pairs are orthogonal:**
- ✅ Odd ⊥ Even (P₁ ⊥ P₂, P₃ ⊥ P₄, ...)
- ✅ Even ⊥ Even (P₀ ⊥ P₂, P₂ ⊥ P₄, ...)
- ✅ Odd ⊥ Odd (P₁ ⊥ P₃, P₃ ⊥ P₅, ...)

**Key insight:**
```
∫₋₁¹ Pₙ(x)·Pₘ(x) dx = 0  for ANY n ≠ m

In [ ]:
from keras_lmu import LMU

In [ ]:
# In a notebook cell
!uv pip install nvidia-smi

In [ ]:
import tensorflow as tf

# Prevent TensorFlow from allocating all GPU memory at once
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ GPU memory growth enabled")
    except RuntimeError as e:
        print(e)

In [ ]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("\nGPU devices:")
print(tf.config.list_physical_devices('GPU'))

# More detailed info
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"\n✅ Found {len(gpus)} GPU(s):")
    for gpu in gpus:
        print(f"   {gpu}")
        
    # Get GPU details
    gpu_details = tf.config.experimental.get_device_details(gpus[0])
    print(f"\nGPU Details:")
    for key, value in gpu_details.items():
        print(f"   {key}: {value}")
else:
    print("\n❌ No GPU found - running on CPU")

# Test if TensorFlow is actually using GPU
print("\n" + "="*50)
print("Testing GPU usage...")
with tf.device('/GPU:0'):
    try:
        a = tf.constant([[1.0, 2.0], [3.0, 4.0]])
        b = tf.constant([[1.0, 2.0], [3.0, 4.0]])
        c = tf.matmul(a, b)
        print("✅ Successfully ran operation on GPU")
        print(f"   Result device: {c.device}")
    except RuntimeError as e:
        print(f"❌ Failed to use GPU: {e}")

In [ ]:
from tensorflow import keras
import tensorflow as tf

# Add this BEFORE building any model
keras.backend.clear_session()
tf.compat.v1.reset_default_graph()

# Also manually delete old models
import gc
gc.collect()

In [ ]:
# %% [markdown]
# # FallNet Training Pipeline
# CNN-LMU ensemble for fall detection with 6 classes

# %%
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings('ignore')
from keras_lmu import LMU
fallnet_fold
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

# %% [markdown]
## 1. FallNet Model Architecture

# %%
class FallNet:
    """
    FallNet: CNN-LmU Ensemble for Pre-Impact Fall Detection
    """
    
    def __init__(self, input_shape=(200, 6), n_classes=6):
        """
        Args:
            input_shape: (timesteps, features) = (200, 6)
            n_classes: Number of output classes (6)
        """
        self.input_shape = input_shape
        self.n_classes = n_classes
        self.model = None
    
    def build_lmu_branch(self, inputs):
        """LMU Branch"""
        x = LMU(
            memory_d=32,      # REDUCED from 64
            order=32,          # REDUCED from 64
            theta=200.0,
            hidden_cell=None,
            kernel_regularizer=keras.regularizers.L1L2(l1=5e-4, l2=5e-4),  # INCREASED from 1e-4
            recurrent_regularizer=keras.regularizers.L1L2(l1=5e-4, l2=5e-4),
            dropout=0.3,       # NEW
            recurrent_dropout=0.2,  # NEW
            name='lmu'
            )(inputs)
    
        
        # Dense layers with stronger regularization
        x = layers.Dense(
            128,
            activation='relu',
            kernel_regularizer=keras.regularizers.L1L2(l1=5e-4, l2=5e-4),
            name='lmu_dense1'
            )(x)
        x = layers.Dropout(0.4, name='lmu_dropout1')(x)  # INCREASED
    
        x = layers.Dense(
            64,
            activation='relu',
            kernel_regularizer=keras.regularizers.L1L2(l1=5e-4, l2=5e-4),
            name='lmu_dense2'
            )(x)
        x = layers.Dropout(0.3, name='lmu_dropout2')(x)  # INCREASED
    
        x = layers.Dense(
            32,
            activation='relu',
            kernel_regularizer=keras.regularizers.L1L2(l1=5e-4, l2=5e-4),
            name='lmu_dense3'
            )(x)
        x = layers.Dropout(0.2, name='lmu_dropout3')(x)
    
        lmu_output = layers.Dense(
            self.n_classes,
            activation='softmax',
            name='lmu_output'
            )(x)
    
        return lmu_output
        

    def build_lstm_branch(self, inputs):
        """LSTM Branch"""
        x = layers.LSTM(
            units=256,
            activation='tanh',
            return_sequences=False,
            name='lstm_layer'
        )(inputs)
        
        x = layers.Dense(128, activation='relu', name='lstm_dense1')(x)
        x = layers.Dropout(0.2, name='lstm_dropout1')(x)
        
        x = layers.Dense(64, activation='relu', name='lstm_dense2')(x)
        x = layers.Dropout(0.2, name='lstm_dropout2')(x)
        
        x = layers.Dense(32, activation='relu', name='lstm_dense3')(x)
        x = layers.Dropout(0.2, name='lstm_dropout3')(x)
        
        lstm_output = layers.Dense(
            self.n_classes, 
            activation='softmax',
            name='lstm_output'
        )(x)
        
        return lstm_output
    
    def build_cnn_branch(self, inputs):
        """CNN Branch"""
        x = layers.Conv1D(
            filters=128,
            kernel_size=3,
            activation='relu',
            padding='same',
            name='conv1d_layer'
        )(inputs)
        
        x = layers.MaxPooling1D(pool_size=2, name='maxpool_layer')(x)
        x = layers.Flatten(name='flatten_layer')(x)
        
        x = layers.Dense(1024, activation='relu', name='cnn_dense1')(x)
        x = layers.Dropout(0.2, name='cnn_dropout1')(x)
        
        x = layers.Dense(512, activation='relu', name='cnn_dense2')(x)
        x = layers.Dropout(0.2, name='cnn_dropout2')(x)
        
        cnn_output = layers.Dense(
            self.n_classes,
            activation='softmax',
            name='cnn_output'
        )(x)
        
        return cnn_output

    def build_cnn_only(self):
        """Build CNN-only model (no temporal component)"""
        inputs = layers.Input(shape=self.input_shape, name='input')
        cnn_output = self.build_cnn_branch(inputs)
        
        self.model = models.Model(
            inputs=inputs,
            outputs=cnn_output,
            name='FallNet_CNN_Only'
        )
        return self.model
    
    def build_lstm_only(self):
        """Build LSTM-only model (temporal encoding via gates)"""
        inputs = layers.Input(shape=self.input_shape, name='input')
        lstm_output = self.build_lstm_branch(inputs)
        
        self.model = models.Model(
            inputs=inputs,
            outputs=lstm_output,
            name='FallNet_LSTM_Only'
        )
        return self.model
    
    def build_lmu_only(self):
        """Build LMU-only model (temporal encoding via Legendre polynomials)"""
        inputs = layers.Input(shape=self.input_shape, name='input')
        lmu_output = self.build_lmu_branch(inputs)
        
        self.model = models.Model(
            inputs=inputs,
            outputs=lmu_output,
            name='FallNet_LMU_Only'
        )
        return self.model
    
    def build_ensemble(self):
        """Build the complete ensemble model"""
        inputs = layers.Input(shape=self.input_shape, name='input')
        
        lmu_output = self.build_lstm_branch(inputs)
        cnn_output = self.build_cnn_branch(inputs)
        
        ensemble_output = layers.Average(name='ensemble_average')([lmu_output, cnn_output])
        
        self.model = models.Model(
            inputs=inputs,
            outputs=ensemble_output,
            name='FallNet_CNN_LSTM'
        )
        
        return self.model
    
    def compile_model(self, learning_rate=None):
        """Compile model"""
        if self.model is None:
            raise ValueError("Model not built yet. Call build_ensemble() first.")
        
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate) if learning_rate else keras.optimizers.Adam()
        
        self.model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=1e-5),  # 100x smaller
            loss='categorical_crossentropy',
            metrics=[
                'accuracy', 
                keras.metrics.Precision(name='precision'),
                keras.metrics.Recall(name='recall')fallnet_fold
            ]
        )
        
        return self.model

print("✅ FallNet class defined")

# %% [markdown]
## 2. Build and Display Model

# %%
print("\n" + "="*80)
print("BUILDING FALLNET MODEL")
print("="*80)

# Create instance with 6 classes
fallnet = FallNet(input_shape=(200, 6), n_classes=6)

# Build ensemble
model = fallnet.build_ensemble()

# Compile
model = fallnet.compile_model()

# Display architecture
print("\n")
model.summary()

# Count parameters
def count_parameters(model):
    trainable = np.sum([np.prod(v.shape) for v in model.trainable_weights])
    non_trainable = np.sum([np.prod(v.shape) for v in model.non_trainable_weights])
    return trainable, non_trainable

trainable, non_trainable = count_parameters(model)

print("\n" + "="*80)
print("MODEL PARAMETERS")
print("="*80)
print(f"Trainable:     {trainable:,}")
print(f"Non-trainable: {non_trainable:,}")
print(f"Total:         {trainable + non_trainable:,}")

# %% [markdown]
## 3. Training Configuration

# %%
BATCH_SIZE = 2
EPOCHS = 50
K_FOLDS = 5

print("\n" + "="*80)
print("TRAINING CONFIGURATION")
print("="*80)
print(f"Batch size: {BATCH_SIZE}")
print(f"Max epochs: {EPOCHS}")
print(f"K-Folds:    {K_FOLDS}")
print(f"Using data from previous cell (6 classes, {len(y_labels):,} samples)")

# %% [markdown]
## 4. Verify Data Before Training

# %%
print("\n" + "="*80)
print("PRE-TRAINING VERIFICATION")
print("="*80)

print(f"✅ Data shapes:")
print(f"   X_data:        {X_data.shape}")
print(f"   y_labels:      {y_labels.shape}")
print(f"   y_categorical: {y_categorical.shape}")
print(f"\n✅ Classes: {len(np.unique(y_labels))} (should be 6)")
print(f"✅ Label range: {y_labels.min()}-{y_labels.max()} (should be 0-5)")
print(f"✅ Model output: {model.output_shape[-1]} (should be 6)")

assert X_data.shape[0] == y_labels.shape[0] == y_categorical.shape[0], "Shape mismatch!"
assert len(np.unique(y_labels)) == 6, "Should have 6 classes!"
assert y_labels.max() == 5, "Max label should be 5!"
assert model.output_shape[-1] == 6, "Model should output 6 classes!"

print("\n✅ All checks passed - ready to train!")

# %% [markdown]
## 5. K-Fold Cross-Validation Training

# %%
skf = StratifiedKFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

fold_results = []
fold_histories = []

print("\n" + "="*80)
print("STARTING K-FOLD CROSS-VALIDATION")
print("="*80)

for fold, (train_idx, val_idx) in enumerate(skf.split(X_data, y_labels), 1):
    print(f"\n{'='*80}")
    print(f"FOLD {fold}/{K_FOLDS}")
    print(f"{'='*80}")
    
    # Split data
    X_train, X_val = X_data[train_idx], X_data[val_idx]
    y_train, y_val = y_categorical[train_idx], y_categorical[val_idx]
    
    print(f"Train: {X_train.shape[0]:,} samples | Val: {X_val.shape[0]:,} samples")
    
    # Build fresh model for this fold
    fallnet_fold = FallNet(input_shape=(200, 6), n_classes=6)
    model_fold = fallnet_fold.build_ensemble()
    model_fold = fallnet_fold.compile_model()
    
    # Define callbacks for THIS fold
    fold_callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=20,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=10,
            min_lr=1e-7,
            verbose=1
        ),
        ModelCheckpoint(
            filepath=str(output_dir / f'fallnet_lstm_fold_{fold}.keras'),
            monitor='val_accuracy',
            save_best_only=True,
            mode='max',
            verbose=1
        )
    ]
    # %% [markdown]
## 5.5 Calculate Class Weights

# %%
    from sklearn.utils.class_weight import compute_class_weight

    print("\n" + "="*80)
    print("CALCULATING CLASS WEIGHTS")
    print("="*80)

# Calculate balanced weights
    class_weights_array = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(y_labels),
        y=y_labels
    )

# Cap at 3x to prevent training instability
    MAX_WEIGHT = 3.0
    class_weights_array_capped = np.clip(class_weights_array, None, MAX_WEIGHT)
    class_weights = dict(enumerate(class_weights_array_capped))

    print("\nClass Distribution:")
    from collections import Counter
    counts = Counter(y_labels)
    for cls_idx in range(6):
        count = counts[cls_idx]
        pct = count / len(y_labels) * 100
        weight = class_weights[cls_idx]
        print(f"  {reverse_label_map[cls_idx]:<30s}: {count:>5d} ({pct:>5.2f}%) → weight: {weight:.2f}x")

    print(f"\n✅ Weight range: {min(class_weights.values()):.2f}x to {max(class_weights.values()):.2f}x")
    print(f"✅ Max/Min ratio: {max(class_weights.values())/min(class_weights.values()):.2f}x (was 4.0x without capping)")
    # Train WITHOUT class weights
    print(f"\nTraining fold {fold}...")
    history = model_fold.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        class_weight=class_weights,  # ← ADD THIS LINE!
        callbacks=fold_callbacks,
        verbose=1
    )
    
    # Evaluate
    val_loss, val_acc, val_precision, val_recall = model_fold.evaluate(X_val, y_val, batch_size=2, verbose=0)
    val_f1 = 2 * (val_precision * val_recall) / (val_precision + val_recall) if (val_precision + val_recall) > 0 else 0
    
    print(f"\n{'='*50}")
    print(f"Fold {fold} Results:")
    print(f"{'='*50}")
    print(f"Loss:      {val_loss:.4f}")
    print(f"Accuracy:  {val_acc:.4f}")
    print(f"Precision: {val_precision:.4f}")
    print(f"Recall:    {val_recall:.4f}")
    print(f"F1-Score:  {val_f1:.4f}")
    
    # Store results
    fold_results.append({
        'fold': fold,
        'val_loss': val_loss,
        'val_accuracy': val_acc,
        'val_precision': val_precision,
        'val_recall': val_recall,
        'val_f1': val_f1
    })
    
    fold_histories.append(history.history)
    
    print(f"✅ Model saved: fallnet_lstm_fold_{fold}.keras")

print("\n" + "="*80)
print("K-FOLD CROSS-VALIDATION COMPLETE")
print("="*80)

# %% [markdown]
## 6. Aggregate Results

# %%
results_df = pd.DataFrame(fold_results)

print("\n" + "="*80)
print("RESULTS ACROSS ALL FOLDS")
print("="*80)
print(results_df.to_string(index=False))

print("\n" + "="*80)
print("AVERAGE PERFORMANCE ± STD")
print("="*80)

mean_results = results_df.mean(numeric_only=True)
std_results = results_df.std(numeric_only=True)

metrics_table = []
for metric in ['val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_f1']:
    metrics_table.append({
        'Metric': metric,
        'Mean': f"{mean_results[metric]:.4f}",
        'Std': f"±{std_results[metric]:.4f}"
    })

metrics_df = pd.DataFrame(metrics_table)
print(metrics_df.to_string(index=False))

# %% [markdown]
## 7. Visualize Training History

# %%
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

metrics = [
    ('loss', 'Loss'),
    ('accuracy', 'Accuracy'),
    ('precision', 'Precision'),
    ('recall', 'Recall')
]

for idx, (metric, title) in enumerate(metrics):
    ax = axes[idx // 2, idx % 2]
    
    for fold, history in enumerate(fold_histories, 1):
        epochs = range(1, len(history[metric]) + 1)
        ax.plot(epochs, history[metric], label=f'Fold {fold} Train', alpha=0.5, linewidth=1)
        ax.plot(epochs, history[f'val_{metric}'], label=f'Fold {fold} Val', 
                linestyle='--', alpha=0.7, linewidth=1.5)
    
    ax.set_title(f'{title} Across All Folds', fontsize=13, fontweight='bold')
    ax.set_xlabel('Epoch', fontsize=11)
    ax.set_ylabel(title, fontsize=11)
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=7)
    ax.grid(True, alpha=0.3)

plt.suptitle('FallNet Training History - 5-Fold Cross-Validation', 
             fontsize=15, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig(output_dir / 'training_history.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✅ Training history saved to {output_dir / 'training_history.png'}")

# %% [markdown]
## 8. Detailed Evaluation on Best Fold

# %%
best_fold = int(results_df.loc[results_df['val_f1'].idxmax(), 'fold'])

print("\n" + "="*80)
print(f"DETAILED EVALUATION - BEST FOLD #{best_fold}")
print("="*80)
print(f"Best fold F1-Score: {results_df.loc[results_df['fold']==best_fold, 'val_f1'].values[0]:.4f}")

# Load best model
best_model = keras.models.load_model(output_dir / f'fallnet_fold_lstm_{best_fold}.keras')

# Get predictions on ALL data
y_pred_probs = best_model.predict(X_data, verbose=0)
y_pred = np.argmax(y_pred_probs, axis=1)

# Classification report
class_names = [reverse_label_map[i] for i in range(6)]

print("\n" + "="*80)
print("CLASSIFICATION REPORT (Best Fold on All Data)")
print("="*80)
print(classification_report(y_labels, y_pred, target_names=class_names, digits=4))

# %% [markdown]
## 9. Per-Class Detailed Metrics

# %%
print("\n" + "="*80)
print("PER-CLASS DETAILED METRICS")
print("="*80)

print(f"\n{'Class':<40s} {'Precision':<12s} {'Recall':<12s} {'F1-Score':<12s} {'Support'}")
print("-"*90)

for cls_idx in range(6):
    precision = precision_score(y_labels == cls_idx, y_pred == cls_idx, zero_division=0)
    recall = recall_score(y_labels == cls_idx, y_pred == cls_idx, zero_division=0)
    f1 = f1_score(y_labels == cls_idx, y_pred == cls_idx, zero_division=0)
    support = np.sum(y_labels == cls_idx)
    
    print(f"{reverse_label_map[cls_idx]:<40s} {precision:<12.4f} {recall:<12.4f} {f1:<12.4f} {support}")

# %% [markdown]
## 10. Confusion Matrix

# %%
cm = confusion_matrix(y_labels, y_pred)

plt.figure(figsize=(12, 10))
sns.heatmap(
    cm, 
    annot=True, 
    fmt='d', 
    cmap='Blues',
    xticklabels=class_names,
    yticklabels=class_names,
    cbar_kws={'label': 'Count'}
)
plt.title('Confusion Matrix - Best Fold (6 Classes)', fontsize=15, fontweight='bold', pad=20)
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.yticks(rotation=0, fontsize=10)
plt.tight_layout()
plt.savefig(output_dir / 'confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✅ Confusion matrix saved to {output_dir / 'confusion_matrix.png'}")

# %% [markdown]
## 11. Final Summary

# %%
# Get Fall_Initiation metrics
fall_init_idx = label_map["Fall_Initiation"]
fall_init_precision = precision_score(y_labels == fall_init_idx, y_pred == fall_init_idx)
fall_init_recall = recall_score(y_labels == fall_init_idx, y_pred == fall_init_idx)
fall_init_f1 = f1_score(y_labels == fall_init_idx, y_pred == fall_init_idx)

print("\n" + "="*80)
print("TRAINING COMPLETE - FINAL SUMMARY")
print("="*80)

summary = f"""
✅ Successfully trained FallNet with 5-fold cross-validation

Configuration:
  - Model: CNN-LSTM Ensemble (6 classes)
  - Total samples: {len(y_labels):,}
  - Training samples per fold: ~{len(y_labels)*0.8//K_FOLDS:,.0f}
  - Validation samples per fold: ~{len(y_labels)*0.2//K_FOLDS:,.0f}

Average Performance (5-fold CV):
  - Accuracy:  {mean_results['val_accuracy']:.4f} ± {std_results['val_accuracy']:.4f}
  - Precision: {mean_results['val_precision']:.4f} ± {std_results['val_precision']:.4f}
  - Recall:    {mean_results['val_recall']:.4f} ± {std_results['val_recall']:.4f}
  - F1-Score:  {mean_results['val_f1']:.4f} ± {std_results['val_f1']:.4f}

Fall_Initiation Performance (Critical Class):
  - Recall (Sensitivity): {fall_init_recall:.4f}
  - F1-Score:             {fall_init_f1:.4f}

Saved Files:
  - Training history:    {output_dir / 'training_history.png'}
  - Confusion matrix:    {output_dir / 'confusion_matrix.png'}
  - Best model:          {output_dir / f'fallnet_fold_{best_fold}.keras'}
  - All fold models:     {output_dir / 'fallnet_fold_*.keras'}
"""

print(summary)

with open(output_dir / 'training_summary.txt', 'w') as f:
    f.write(summary)

print(f"✅ Summary saved to {output_dir / 'training_summary.txt'}")

In [ ]:
# Can LMU overfit a tiny dataset?
X_tiny = X_data[:100]
y_tiny = y_categorical[:100]

# Simple LMU-only model
model_test = keras.Sequential([
    LMU(memory_d=64, order=64, theta=200.0, hidden_cell=None, input_shape=(200, 6)),
    layers.Dense(6, activation='softmax')
])

model_test.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model_test.fit(X_tiny, y_tiny, epochs=100, batch_size=10, verbose=1)

print(f"\nFinal accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"Should be >0.90 if LMU works properly")

print("TensorFlow version:", tf.__version__)
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("GPUs available:", tf.config.list_physical_devices('GPU'))


In [ ]:
# Can LMU overfit a tiny dataset?
X_tiny = X_data[:100]
y_tiny = y_categorical[:100]

# Simple LMU-only model
model_test = keras.Sequential([
    LMU(memory_d=64, order=64, theta=200.0, hidden_cell=None, input_shape=(200, 6)),
    layers.Dense(6, activation='softmax')
])

model_test.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model_test.fit(X_tiny, y_tiny, epochs=100, batch_size=10, verbose=1)

# Should reach >95% accuracy if LMU works

In [ ]:
from tensorflow import keras
import gc

# Clear session ONCE at start
keras.backend.clear_session()
gc.collect()

print("\n" + "="*80)
print("TRAINING CNN-ONLY MODEL")
print("="*80)

BATCH_SIZE = 1  # ← Reduce to 1 for safety
EPOCHS = 50
K_FOLDS = 5
MODEL_NAME = "cnn_only"

skf = StratifiedKFold(n_splits=K_FOLDS, shuffle=True, random_state=42)
fold_results = []
fold_histories = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X_data, y_labels), 1):
    print(f"\n{'='*80}")
    print(f"FOLD {fold}/{K_FOLDS} - CNN-ONLY")
    print(f"{'='*80}")
    
    # ⭐ CLEAR MEMORY AT START OF EACH FOLD
    keras.backend.clear_session()
    gc.collect()
    
    # Split data
    X_train, X_val = X_data[train_idx], X_data[val_idx]
    y_train, y_val = y_categorical[train_idx], y_categorical[val_idx]
    
    print(f"Train: {X_train.shape[0]:,} samples | Val: {X_val.shape[0]:,} samples")
    
    # Build CNN-only model
    fallnet_fold = FallNet(input_shape=(200, 6), n_classes=6)
    model_fold = fallnet_fold.build_cnn_only()
    model_fold = fallnet_fold.compile_model()
    
    # Callbacks
    fold_callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=20,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=10,
            min_lr=1e-7,
            verbose=1
        ),
        ModelCheckpoint(
            filepath=str(output_dir / f'{MODEL_NAME}_fold_{fold}.keras'),
            monitor='val_accuracy',
            save_best_only=True,
            mode='max',
            verbose=1
        )
    ]
    
    # Class weights
    from sklearn.utils.class_weight import compute_class_weight
    class_weights_array = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(y_labels),
        y=y_labels
    )
    MAX_WEIGHT = 3.0
    class_weights_array_capped = np.clip(class_weights_array, None, MAX_WEIGHT)
    class_weights = dict(enumerate(class_weights_array_capped))
    
    # Train
    print(f"\nTraining fold {fold}...")
    history = model_fold.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        batch_size=BATCH_SIZE,  # Now 1
        epochs=EPOCHS,
        class_weight=class_weights,
        callbacks=fold_callbacks,
        verbose=2  # Less verbose output
    )
    
    # Evaluate
    val_loss, val_acc, val_precision, val_recall = model_fold.evaluate(
        X_val, y_val, 
        batch_size=1,  # Match training batch size
        verbose=0
    )
    val_f1 = 2 * (val_precision * val_recall) / (val_precision + val_recall) if (val_precision + val_recall) > 0 else 0
    
    print(f"\n{'='*50}")
    print(f"Fold {fold} Results:")
    print(f"{'='*50}")
    print(f"Loss:      {val_loss:.4f}")
    print(f"Accuracy:  {val_acc:.4f}")
    print(f"Precision: {val_precision:.4f}")
    print(f"Recall:    {val_recall:.4f}")
    print(f"F1-Score:  {val_f1:.4f}")
    
    fold_results.append({
        'fold': fold,
        'val_loss': val_loss,
        'val_accuracy': val_acc,
        'val_precision': val_precision,
        'val_recall': val_recall,
        'val_f1': val_f1
    })
    
    fold_histories.append(history.history)
    
    # ⭐ CLEAR MEMORY AT END OF EACH FOLD
    del model_fold, fallnet_fold
    del X_train, X_val, y_train, y_val, history
    keras.backend.clear_session()
    gc.collect()
    
    print(f"\n✓ Fold {fold} complete, memory cleared")

# Results summary
results_df = pd.DataFrame(fold_results)
print("\n" + "="*80)
print("CNN-ONLY RESULTS ACROSS ALL FOLDS")
print("="*80)
print(results_df.to_string(index=False))

mean_results = results_df.mean(numeric_only=True)
std_results = results_df.std(numeric_only=True)

print("\n" + "="*80)
print("CNN-ONLY AVERAGE PERFORMANCE")
print("="*80)
print(f"Accuracy:  {mean_results['val_accuracy']:.4f} ± {std_results['val_accuracy']:.4f}")
print(f"Precision: {mean_results['val_precision']:.4f} ± {std_results['val_precision']:.4f}")
print(f"Recall:    {mean_results['val_recall']:.4f} ± {std_results['val_recall']:.4f}")
print(f"F1-Score:  {mean_results['val_f1']:.4f} ± {std_results['val_f1']:.4f}")

In [ ]:
# %% [markdown]
# ## Save CNN-Only Summary

# %%
import json
from pathlib import Path

# CNN-only results
cnn_only_summary = {
    "model_type": "CNN-Only",
    "architecture": {
        "input_shape": "(200, 6)",
        "conv1d": "128 filters, kernel=3",
        "maxpool": "pool_size=2",
        "dense_layers": [1024, 512],
        "dropout": 0.2,
        "output": "6 classes (softmax)"
    },
    "training_config": {
        "batch_size": 2,
        "learning_rate": 1e-5,
        "epochs": 50,
        "k_folds": 5,
        "class_weights": "balanced (capped at 3.0x)"
    },
    "results": {
        "fold_1": {"accuracy": 0.8921, "precision": 0.9084, "recall": 0.8799, "f1": 0.8939},
        "fold_2": {"accuracy": 0.8901, "precision": 0.9041, "recall": 0.8763, "f1": 0.8900},
        "fold_3": {"accuracy": 0.8960, "precision": 0.9059, "recall": 0.8834, "f1": 0.8945},
        "fold_4": {"accuracy": 0.8805, "precision": 0.8939, "recall": 0.8640, "f1": 0.8787},
        "fold_5": {"accuracy": 0.8825, "precision": 0.9023, "recall": 0.8667, "f1": 0.8841}
    },
    "average_performance": {
        "accuracy": 0.8882,
        "accuracy_std": 0.0066,
        "precision": 0.9029,
        "precision_std": 0.0055,
        "recall": 0.8741,
        "recall_std": 0.0084,
        "f1": 0.8883,
        "f1_std": 0.0068
    },
    "comparison_to_ensemble": {
        "cnn_lmu_ensemble": 0.951,
        "cnn_only": 0.8882,
        "improvement_from_lmu": 0.063,
        "interpretation": "LMU adds 6.3% accuracy by providing temporal encoding"
    }
}

# Save to JSON
with open(output_dir / 'cnn_only_summary.json', 'w') as f:
    json.dump(cnn_only_summary, f, indent=2)

# Save to text
summary_text = f"""
================================================================================
CNN-ONLY MODEL SUMMARY
================================================================================

Model Architecture:
  - Input: (200, 6) - 200 timesteps × 6 IMU features
  - Conv1D: 128 filters, kernel_size=3, activation=relu
  - MaxPooling1D: pool_size=2
  - Flatten
  - Dense(1024, relu) + Dropout(0.2)
  - Dense(512, relu) + Dropout(0.2)
  - Dense(6, softmax)

Training Configuration:
  - Batch size: 2
  - Learning rate: 1e-5
  - Epochs: 50 (with early stopping)
  - K-Folds: 5
  - Class weights: Balanced (capped at 3.0x)

Results (5-Fold Cross-Validation):
  Fold 1: Accuracy = 89.21%, F1 = 89.39%
  Fold 2: Accuracy = 89.01%, F1 = 89.00%
  Fold 3: Accuracy = 89.60%, F1 = 89.45%
  Fold 4: Accuracy = 88.05%, F1 = 87.87%
  Fold 5: Accuracy = 88.25%, F1 = 88.41%

Average Performance:
  Accuracy:  88.82% ± 0.66%
  Precision: 90.29% ± 0.55%
  Recall:    87.41% ± 0.84%
  F1-Score:  88.83% ± 0.68%

Comparison to Ensemble:
  CNN-LMU Ensemble: 95.1%
  CNN-Only:         88.8%
  Improvement:      +6.3% from adding LMU

Interpretation:
  The CNN alone achieves 88.8% by capturing spatial correlations between
  accelerometer/gyroscope channels. Adding the LMU branch provides temporal
  encoding (sequence dynamics over 200 timesteps), boosting performance to
  95.1%. This 6.3% improvement demonstrates that temporal information is
  critical for fall detection.

Model Files:
  - cnn_only_fold_1.keras
  - cnn_only_fold_2.keras
  - cnn_only_fold_3.keras
  - cnn_only_fold_4.keras
  - cnn_only_fold_5.keras

Best Fold: Fold 3 (89.60% accuracy)
================================================================================
"""

with open(output_dir / 'cnn_only_summary.txt', 'w') as f:
    f.write(summary_text)

print("✅ CNN-only summary saved!")
print(f"   JSON: {output_dir / 'cnn_only_summary.json'}")
print(f"   Text: {output_dir / 'cnn_only_summary.txt'}")
print(f"   Models: {output_dir / 'cnn_only_fold_*.keras'}")

In [ ]:
# %% [markdown]
# ## Branch Isolation Test 2: LSTM-Only

# %%
from tensorflow import keras
import gc

# Clear session
keras.backend.clear_session()
gc.collect()

print("\n" + "="*80)
print("TRAINING LSTM-ONLY MODEL")
print("="*80)

BATCH_SIZE = 2
EPOCHS = 50
K_FOLDS = 5
MODEL_NAME = "lstm_only"

skf = StratifiedKFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

fold_results = []
fold_histories = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X_data, y_labels), 1):
    print(f"\n{'='*80}")
    print(f"FOLD {fold}/{K_FOLDS} - LSTM-ONLY")
    print(f"{'='*80}")
    
    # Split data
    X_train, X_val = X_data[train_idx], X_data[val_idx]
    y_train, y_val = y_categorical[train_idx], y_categorical[val_idx]
    
    print(f"Train: {X_train.shape[0]:,} samples | Val: {X_val.shape[0]:,} samples")
    
    # Build LSTM-only model
    fallnet_fold = FallNet(input_shape=(200, 6), n_classes=6)
    model_fold = fallnet_fold.build_lstm_only()  # ← LSTM-ONLY
    model_fold = fallnet_fold.compile_model()
    
    # Callbacks
    fold_callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=20,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=10,
            min_lr=1e-7,
            verbose=1
        ),
        ModelCheckpoint(
            filepath=str(output_dir / f'{MODEL_NAME}_fold_{fold}.keras'),
            monitor='val_accuracy',
            save_best_only=True,
            mode='max',
            verbose=1
        )
    ]
    
    # Class weights
    from sklearn.utils.class_weight import compute_class_weight
    class_weights_array = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(y_labels),
        y=y_labels
    )
    MAX_WEIGHT = 3.0
    class_weights_array_capped = np.clip(class_weights_array, None, MAX_WEIGHT)
    class_weights = dict(enumerate(class_weights_array_capped))
    
    # Train
    print(f"\nTraining fold {fold}...")
    history = model_fold.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        class_weight=class_weights,
        callbacks=fold_callbacks,
        verbose=1
    )
    
    # Evaluate
    val_loss, val_acc, val_precision, val_recall = model_fold.evaluate(X_val, y_val, batch_size=2, verbose=0)
    val_f1 = 2 * (val_precision * val_recall) / (val_precision + val_recall) if (val_precision + val_recall) > 0 else 0
    
    print(f"\n{'='*50}")
    print(f"Fold {fold} Results:")
    print(f"{'='*50}")
    print(f"Loss:      {val_loss:.4f}")
    print(f"Accuracy:  {val_acc:.4f}")
    print(f"Precision: {val_precision:.4f}")
    print(f"Recall:    {val_recall:.4f}")
    print(f"F1-Score:  {val_f1:.4f}")
    
    fold_results.append({
        'fold': fold,
        'val_loss': val_loss,
        'val_accuracy': val_acc,
        'val_precision': val_precision,
        'val_recall': val_recall,
        'val_f1': val_f1
    })
    
    fold_histories.append(history.history)
    
    # Clear memory
    keras.backend.clear_session()
    gc.collect()

# Results
results_df = pd.DataFrame(fold_results)

print("\n" + "="*80)
print("LSTM-ONLY RESULTS ACROSS ALL FOLDS")
print("="*80)
print(results_df.to_string(index=False))

mean_results = results_df.mean(numeric_only=True)
std_results = results_df.std(numeric_only=True)

print("\n" + "="*80)
print("LSTM-ONLY AVERAGE PERFORMANCE")
print("="*80)
print(f"Accuracy:  {mean_results['val_accuracy']:.4f} ± {std_results['val_accuracy']:.4f}")
print(f"Precision: {mean_results['val_precision']:.4f} ± {std_results['val_precision']:.4f}")
print(f"Recall:    {mean_results['val_recall']:.4f} ± {std_results['val_recall']:.4f}")
print(f"F1-Score:  {mean_results['val_f1']:.4f} ± {std_results['val_f1']:.4f}")

In [ ]:
# %% [markdown]
# ## Branch Isolation Test 3: LMU-Only

# %%
from tensorflow import keras
import gc

# Clear session
keras.backend.clear_session()
gc.collect()

print("\n" + "="*80)
print("TRAINING LMU-ONLY MODEL")
print("="*80)

BATCH_SIZE = 2
EPOCHS = 50
K_FOLDS = 5
MODEL_NAME = "lmu_only"

skf = StratifiedKFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

fold_results = []
fold_histories = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X_data, y_labels), 1):
    print(f"\n{'='*80}")
    print(f"FOLD {fold}/{K_FOLDS} - LMU-ONLY")
    print(f"{'='*80}")
    
    # Split data
    X_train, X_val = X_data[train_idx], X_data[val_idx]
    y_train, y_val = y_categorical[train_idx], y_categorical[val_idx]
    
    print(f"Train: {X_train.shape[0]:,} samples | Val: {X_val.shape[0]:,} samples")
    
    # Build LMU-only model
    fallnet_fold = FallNet(input_shape=(200, 6), n_classes=6)
    model_fold = fallnet_fold.build_lmu_only()  # ← LMU-ONLY
    model_fold = fallnet_fold.compile_model()
    
    # Callbacks
    fold_callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=2,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=10,
            min_lr=1e-7,
            verbose=1
        ),
        ModelCheckpoint(
            filepath=str(output_dir / f'{MODEL_NAME}_fold_{fold}.keras'),
            monitor='val_accuracy',
            save_best_only=True,
            mode='max',
            verbose=1
        )
    ]
    
    # Class weights
    from sklearn.utils.class_weight import compute_class_weight
    class_weights_array = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(y_labels),
        y=y_labels
    )
    MAX_WEIGHT = 3.0
    class_weights_array_capped = np.clip(class_weights_array, None, MAX_WEIGHT)
    class_weights = dict(enumerate(class_weights_array_capped))
    
    # Train
    print(f"\nTraining fold {fold}...")
    history = model_fold.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        class_weight=class_weights,
        callbacks=fold_callbacks,
        verbose=1
    )
    
    # Evaluate
    val_loss, val_acc, val_precision, val_recall = model_fold.evaluate(X_val, y_val, batch_size=2, verbose=0)
    val_f1 = 2 * (val_precision * val_recall) / (val_precision + val_recall) if (val_precision + val_recall) > 0 else 0
    
    print(f"\n{'='*50}")
    print(f"Fold {fold} Results:")
    print(f"{'='*50}")
    print(f"Loss:      {val_loss:.4f}")
    print(f"Accuracy:  {val_acc:.4f}")
    print(f"Precision: {val_precision:.4f}")
    print(f"Recall:    {val_recall:.4f}")
    print(f"F1-Score:  {val_f1:.4f}")
    
    fold_results.append({
        'fold': fold,
        'val_loss': val_loss,
        'val_accuracy': val_acc,
        'val_precision': val_precision,
        'val_recall': val_recall,
        'val_f1': val_f1
    })
    
    fold_histories.append(history.history)
    
    # Clear memory
    keras.backend.clear_session()
    gc.collect()

# Results
results_df = pd.DataFrame(fold_results)

print("\n" + "="*80)
print("LMU-ONLY RESULTS ACROSS ALL FOLDS")
print("="*80)
print(results_df.to_string(index=False))

mean_results = results_df.mean(numeric_only=True)
std_results = results_df.std(numeric_only=True)

print("\n" + "="*80)
print("LMU-ONLY AVERAGE PERFORMANCE")
print("="*80)
print(f"Accuracy:  {mean_results['val_accuracy']:.4f} ± {std_results['val_accuracy']:.4f}")
print(f"Precision: {mean_results['val_precision']:.4f} ± {std_results['val_precision']:.4f}")
print(f"Recall:    {mean_results['val_recall']:.4f} ± {std_results['val_recall']:.4f}")
print(f"F1-Score:  {mean_results['val_f1']:.4f} ± {std_results['val_f1']:.4f}")

In [ ]:
#!/usr/bin/env python3
"""
Improved LMU-only training with stronger regularization to combat overfitting.

Changes from original:
1. Increased L1L2 regularization (1e-4 → 5e-4)
2. Added dropout to LMU layer (0.3)
3. Added recurrent dropout (0.2)
4. Earlier early stopping (patience 5 → 3)
5. Reduced LMU capacity (memory_d=32, order=32 instead of 64/64)
"""

import numpy as np
import tensorflow as tf
from tensorflow import keras
from pathlib import Path
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
import json

# Import KerasLMU
import keras_lmu

print("="*80)
print("LMU-ONLY TRAINING - IMPROVED REGULARIZATION")
print("="*80)

# ============================================================================
# CONFIGURATION
# ============================================================================

# Paths
base_dir = Path.home() / 'repos/summerschool2023/projects/fall-detection/fall_detection_data'
processed_dir = base_dir / 'processed'
model_dir = base_dir / 'models'

# Training hyperparameters
BATCH_SIZE = 2
LEARNING_RATE = 1e-5
EPOCHS = 50
K_FOLDS = 5

# LMU parameters - REDUCED CAPACITY
MEMORY_D = 32      # Was 64 - reduced to prevent overfitting
ORDER = 32         # Was 64 - reduced to prevent overfitting
THETA = 200.0

# Regularization - INCREASED
L1_REG = 5e-4      # Was 1e-4 - 5x stronger
L2_REG = 5e-4      # Was 1e-4 - 5x stronger
DROPOUT = 0.3      # NEW - dropout on LMU output
RECURRENT_DROPOUT = 0.2  # NEW - recurrent dropout in LMU

# Early stopping - MORE AGGRESSIVE
EARLY_STOP_PATIENCE = 3  # Was 10 - stop much earlier
REDUCE_LR_PATIENCE = 2   # Was 5 - reduce LR faster

CLASS_NAMES = [
    'Walking',
    'Jogging',
    'Walking_stairs_updown',
    'Stumble_while_walking',
    'Fall_Initiation',
    'Impact_Aftermath'
]

# ============================================================================
# LOAD DATA
# ============================================================================

print("\nLoading data...")
X = np.load(processed_dir / 'X_data_6class.npy')
y = np.load(processed_dir / 'y_labels_6class.npy')

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

# Convert to categorical
y_categorical = keras.utils.to_categorical(y, num_classes=6)

# Calculate class weights
from sklearn.utils.class_weight import compute_class_weight
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y),
    y=y
)
class_weights = {i: weight for i, weight in enumerate(class_weights_array)}

print(f"\nClass weights:")
for i, weight in class_weights.items():
    print(f"  Class {i} ({CLASS_NAMES[i]:30s}): {weight:.4f}")

# ============================================================================
# BUILD MODEL FUNCTION - WITH IMPROVED REGULARIZATION
# ============================================================================

def build_lmu_model(input_shape, num_classes):
    """Build LMU-only model with strong regularization"""
    
    inputs = keras.Input(shape=input_shape, name='input')
    
    # LMU layer with dropout and regularization
    x = keras_lmu.LMU(
        memory_d=MEMORY_D,
        order=ORDER,
        theta=THETA,
        hidden_cell=None,
        kernel_regularizer=keras.regularizers.L1L2(l1=L1_REG, l2=L2_REG),
        recurrent_regularizer=keras.regularizers.L1L2(l1=L1_REG, l2=L2_REG),
        dropout=DROPOUT,
        recurrent_dropout=RECURRENT_DROPOUT,
        name='lmu'
    )(inputs)
    
    # Dense layers with strong regularization and dropout
    x = keras.layers.Dense(
        128,
        activation='relu',
        kernel_regularizer=keras.regularizers.L1L2(l1=L1_REG, l2=L2_REG),
        name='dense1'
    )(x)
    x = keras.layers.Dropout(0.4, name='dropout1')(x)  # Increased from typical 0.3
    
    x = keras.layers.Dense(
        64,
        activation='relu',
        kernel_regularizer=keras.regularizers.L1L2(l1=L1_REG, l2=L2_REG),
        name='dense2'
    )(x)
    x = keras.layers.Dropout(0.3, name='dropout2')(x)
    
    x = keras.layers.Dense(
        32,
        activation='relu',
        kernel_regularizer=keras.regularizers.L1L2(l1=L1_REG, l2=L2_REG),
        name='dense3'
    )(x)
    x = keras.layers.Dropout(0.2, name='dropout3')(x)
    
    outputs = keras.layers.Dense(
        num_classes,
        activation='softmax',
        name='output'
    )(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs, name='lmu_regularized')
    
    return model

# ============================================================================
# TRAINING LOOP
# ============================================================================

# Setup k-fold cross-validation
skf = StratifiedKFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

results = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
    print("\n" + "="*80)
    print(f"FOLD {fold}/{K_FOLDS}")
    print("="*80)
    
    # Split data
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y_categorical[train_idx], y_categorical[val_idx]
    
    print(f"Train size: {len(X_train)}, Val size: {len(X_val)}")
    
    # Build model
    model = build_lmu_model(input_shape=(200, 6), num_classes=6)
    
    # Print model summary (only first fold)
    if fold == 1:
        print("\nModel Architecture:")
        model.summary()
        print(f"\nRegularization settings:")
        print(f"  L1/L2: {L1_REG}/{L2_REG}")
        print(f"  LMU dropout: {DROPOUT}")
        print(f"  LMU recurrent dropout: {RECURRENT_DROPOUT}")
        print(f"  Dense dropout: 0.4, 0.3, 0.2")
        print(f"  LMU capacity: memory_d={MEMORY_D}, order={ORDER} (reduced from 64)")
    
    # Compile
    optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()],
        jit_compile=True
    )
    
    # Callbacks - MORE AGGRESSIVE EARLY STOPPING
    callbacks = [
        keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=EARLY_STOP_PATIENCE,  # Reduced to 3
            restore_best_weights=True,
            verbose=1
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_accuracy',
            factor=0.5,
            patience=REDUCE_LR_PATIENCE,  # Reduced to 2
            min_lr=1e-7,
            verbose=1
        ),
        keras.callbacks.ModelCheckpoint(
            filepath=model_dir / f'lmu_regularized_fold_{fold}.keras',
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        )
    ]
    
    # Train
    print(f"\nTraining fold {fold}...")
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        callbacks=callbacks,
        class_weight=class_weights,
        verbose=1
    )
    
    # Evaluate
    print(f"\nEvaluating fold {fold}...")
    val_loss, val_acc, val_prec, val_rec = model.evaluate(X_val, y_val, verbose=0)
    val_f1 = 2 * (val_prec * val_rec) / (val_prec + val_rec + 1e-7)
    
    # Get predictions for detailed metrics
    y_pred_probs = model.predict(X_val, verbose=0)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.argmax(y_val, axis=1)
    
    # Per-class metrics
    report = classification_report(y_true, y_pred, target_names=CLASS_NAMES, output_dict=True)
    
    results.append({
        'fold': fold,
        'val_loss': val_loss,
        'val_accuracy': val_acc,
        'val_precision': val_prec,
        'val_recall': val_rec,
        'val_f1': val_f1,
        'per_class': report
    })
    
    print(f"\nFold {fold} Results:")
    print(f"  Val Loss:      {val_loss:.6f}")
    print(f"  Val Accuracy:  {val_acc:.6f}")
    print(f"  Val Precision: {val_prec:.6f}")
    print(f"  Val Recall:    {val_rec:.6f}")
    print(f"  Val F1:        {val_f1:.6f}")
    
    # Fall_Initiation specific
    fall_metrics = report['Fall_Initiation']
    print(f"\n  Fall_Initiation:")
    print(f"    Precision: {fall_metrics['precision']:.4f}")
    print(f"    Recall:    {fall_metrics['recall']:.4f}")
    print(f"    F1-Score:  {fall_metrics['f1-score']:.4f}")
    
    # Check for overfitting
    train_acc = max(history.history['accuracy'])
    val_acc_best = max(history.history['val_accuracy'])
    gap = train_acc - val_acc_best
    print(f"\n  Overfitting check:")
    print(f"    Best train acc: {train_acc:.4f}")
    print(f"    Best val acc:   {val_acc_best:.4f}")
    print(f"    Gap:            {gap:.4f} ({gap*100:.1f}%)")
    if gap > 0.10:
        print(f"    ⚠️  Still overfitting by >10%")
    elif gap > 0.05:
        print(f"    ⚠️  Moderate overfitting (5-10%)")
    else:
        print(f"    ✓ Good generalization (<5% gap)")

# ============================================================================
# AGGREGATE RESULTS
# ============================================================================

print("\n" + "="*80)
print("LMU-REGULARIZED RESULTS ACROSS ALL FOLDS")
print("="*80)

import pandas as pd
df = pd.DataFrame(results)
print(df[['fold', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_f1']].to_string(index=False))

print("\n" + "="*80)
print("LMU-REGULARIZED AVERAGE PERFORMANCE")
print("="*80)
print(f"Accuracy:  {df['val_accuracy'].mean():.4f} ± {df['val_accuracy'].std():.4f}")
print(f"Precision: {df['val_precision'].mean():.4f} ± {df['val_precision'].std():.4f}")
print(f"Recall:    {df['val_recall'].mean():.4f} ± {df['val_recall'].std():.4f}")
print(f"F1-Score:  {df['val_f1'].mean():.4f} ± {df['val_f1'].std():.4f}")

# Compare to original LMU
print("\n" + "="*80)
print("COMPARISON TO ORIGINAL LMU")
print("="*80)
print(f"Original LMU:     {0.8072:.4f} ± {0.0166:.4f}")
print(f"Regularized LMU:  {df['val_accuracy'].mean():.4f} ± {df['val_accuracy'].std():.4f}")
improvement = df['val_accuracy'].mean() - 0.8072
print(f"Improvement:      {improvement:+.4f} ({improvement*100:+.1f}%)")

# Save results
output = {
    'config': {
        'memory_d': MEMORY_D,
        'order': ORDER,
        'theta': THETA,
        'l1_reg': L1_REG,
        'l2_reg': L2_REG,
        'dropout': DROPOUT,
        'recurrent_dropout': RECURRENT_DROPOUT,
        'early_stop_patience': EARLY_STOP_PATIENCE,
        'reduce_lr_patience': REDUCE_LR_PATIENCE
    },
    'results': results,
    'summary': {
        'mean_accuracy': float(df['val_accuracy'].mean()),
        'std_accuracy': float(df['val_accuracy'].std()),
        'mean_precision': float(df['val_precision'].mean()),
        'std_precision': float(df['val_precision'].std()),
        'mean_recall': float(df['val_recall'].mean()),
        'std_recall': float(df['val_recall'].std()),
        'mean_f1': float(df['val_f1'].mean()),
        'std_f1': float(df['val_f1'].std())
    }
}

output_file = model_dir / 'lmu_regularized_summary.json'
with open(output_file, 'w') as f:
    json.dump(output, f, indent=2)

print(f"\n✓ Results saved to {output_file}")
print("\n" + "="*80)
print("✓ TRAINING COMPLETE")
print("="*80)

In [ ]:
#!/usr/bin/env python3
"""
Improved LMU-only training with stronger regularization to combat overfitting.

Changes from original:
1. Increased L1L2 regularization (1e-4 → 5e-4)
2. Added dropout to LMU layer (0.3)
3. Added recurrent dropout (0.2)
4. Earlier early stopping (patience 5 → 3)
5. Reduced LMU capacity (memory_d=32, order=32 instead of 64/64)
"""

import numpy as np
import tensorflow as tf
from tensorflow import keras
from pathlib import Path
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
import json

# Import KerasLMU
import keras_lmu

print("="*80)
print("LMU-ONLY TRAINING - IMPROVED REGULARIZATION")
print("="*80)

# ============================================================================
# CONFIGURATION
# ============================================================================

# Paths
base_dir = Path.home() / 'repos/summerschool2023/projects/fall-detection/fall_detection_data'
processed_dir = base_dir / 'processed'
model_dir = base_dir / 'models'

# Training hyperparameters
BATCH_SIZE = 2
LEARNING_RATE = 1e-5
EPOCHS = 50
K_FOLDS = 5

# LMU parameters - MODERATE CAPACITY
MEMORY_D = 48      # Moderate - between 32 (too small) and 64 (too big)
ORDER = 48         # Moderate - between 32 (too small) and 64 (too big)
THETA = 200.0

# Regularization - MODERATE (between weak and strong)
L1_REG = 2e-4      # Moderate - between 1e-4 (weak) and 5e-4 (strong)
L2_REG = 2e-4      # Moderate - between 1e-4 (weak) and 5e-4 (strong)
DROPOUT = 0.2      # Moderate - lighter than 0.3
RECURRENT_DROPOUT = 0.1  # Light - half of previous

# Early stopping - BALANCED
EARLY_STOP_PATIENCE = 5  # Moderate - between 3 (too aggressive) and 10 (too lenient)
REDUCE_LR_PATIENCE = 3   # Moderate - between 2 and 5

CLASS_NAMES = [
    'Walking',
    'Jogging',
    'Walking_stairs_updown',
    'Stumble_while_walking',
    'Fall_Initiation',
    'Impact_Aftermath'
]

# ============================================================================
# LOAD DATA
# ============================================================================

print("\nLoading data...")
X = np.load(processed_dir / 'X_data_6class.npy')
y = np.load(processed_dir / 'y_labels_6class.npy')

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

# Convert to categorical
y_categorical = keras.utils.to_categorical(y, num_classes=6)

# Calculate class weights
from sklearn.utils.class_weight import compute_class_weight
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y),
    y=y
)
class_weights = {i: weight for i, weight in enumerate(class_weights_array)}

print(f"\nClass weights:")
for i, weight in class_weights.items():
    print(f"  Class {i} ({CLASS_NAMES[i]:30s}): {weight:.4f}")

# ============================================================================
# BUILD MODEL FUNCTION - WITH IMPROVED REGULARIZATION
# ============================================================================

def build_lmu_model(input_shape, num_classes):
    """Build LMU-only model with strong regularization"""
    
    inputs = keras.Input(shape=input_shape, name='input')
    
    # LMU layer with dropout and regularization
    x = keras_lmu.LMU(
        memory_d=MEMORY_D,
        order=ORDER,
        theta=THETA,
        hidden_cell=None,
        kernel_regularizer=keras.regularizers.L1L2(l1=L1_REG, l2=L2_REG),
        recurrent_regularizer=keras.regularizers.L1L2(l1=L1_REG, l2=L2_REG),
        dropout=DROPOUT,
        recurrent_dropout=RECURRENT_DROPOUT,
        name='lmu'
    )(inputs)
    
    # Dense layers with moderate regularization and dropout
    x = keras.layers.Dense(
        128,
        activation='relu',
        kernel_regularizer=keras.regularizers.L1L2(l1=L1_REG, l2=L2_REG),
        name='dense1'
    )(x)
    x = keras.layers.Dropout(0.3, name='dropout1')(x)  # Moderate
    
    x = keras.layers.Dense(
        64,
        activation='relu',
        kernel_regularizer=keras.regularizers.L1L2(l1=L1_REG, l2=L2_REG),
        name='dense2'
    )(x)
    x = keras.layers.Dropout(0.25, name='dropout2')(x)  # Moderate
    
    x = keras.layers.Dense(
        32,
        activation='relu',
        kernel_regularizer=keras.regularizers.L1L2(l1=L1_REG, l2=L2_REG),
        name='dense3'
    )(x)
    x = keras.layers.Dropout(0.2, name='dropout3')(x)  # Light
    
    outputs = keras.layers.Dense(
        num_classes,
        activation='softmax',
        name='output'
    )(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs, name='lmu_regularized')
    
    return model

# ============================================================================
# TRAINING LOOP
# ============================================================================

# Setup k-fold cross-validation
skf = StratifiedKFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

results = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
    print("\n" + "="*80)
    print(f"FOLD {fold}/{K_FOLDS}")
    print("="*80)
    
    # Split data
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y_categorical[train_idx], y_categorical[val_idx]
    
    print(f"Train size: {len(X_train)}, Val size: {len(X_val)}")
    
    # Build model
    model = build_lmu_model(input_shape=(200, 6), num_classes=6)
    
    # Print model summary (only first fold)
    if fold == 1:
        print("\nModel Architecture:")
        model.summary()
        print(f"\nRegularization settings:")
        print(f"  L1/L2: {L1_REG}/{L2_REG} (moderate)")
        print(f"  LMU dropout: {DROPOUT}")
        print(f"  LMU recurrent dropout: {RECURRENT_DROPOUT}")
        print(f"  Dense dropout: 0.3, 0.25, 0.2")
        print(f"  LMU capacity: memory_d={MEMORY_D}, order={ORDER} (moderate)")
        print(f"  Early stop patience: {EARLY_STOP_PATIENCE}")
    
    # Compile
    optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()],
        jit_compile=True
    )
    
    # Callbacks - MORE AGGRESSIVE EARLY STOPPING
    callbacks = [
        keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=EARLY_STOP_PATIENCE,  # Reduced to 3
            restore_best_weights=True,
            verbose=1
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_accuracy',
            factor=0.5,
            patience=REDUCE_LR_PATIENCE,  # Reduced to 2
            min_lr=1e-7,
            verbose=1
        ),
        keras.callbacks.ModelCheckpoint(
            filepath=model_dir / f'lmu_regularized_fold_{fold}.keras',
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        )
    ]
    
    # Train
    print(f"\nTraining fold {fold}...")
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        callbacks=callbacks,
        class_weight=class_weights,
        verbose=1
    )
    
    # Evaluate
    print(f"\nEvaluating fold {fold}...")
    val_loss, val_acc, val_prec, val_rec = model.evaluate(X_val, y_val, verbose=0)
    val_f1 = 2 * (val_prec * val_rec) / (val_prec + val_rec + 1e-7)
    
    # Get predictions for detailed metrics
    y_pred_probs = model.predict(X_val, verbose=0)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.argmax(y_val, axis=1)
    
    # Per-class metrics
    report = classification_report(y_true, y_pred, target_names=CLASS_NAMES, output_dict=True)
    
    results.append({
        'fold': fold,
        'val_loss': val_loss,
        'val_accuracy': val_acc,
        'val_precision': val_prec,
        'val_recall': val_rec,
        'val_f1': val_f1,
        'per_class': report
    })
    
    print(f"\nFold {fold} Results:")
    print(f"  Val Loss:      {val_loss:.6f}")
    print(f"  Val Accuracy:  {val_acc:.6f}")
    print(f"  Val Precision: {val_prec:.6f}")
    print(f"  Val Recall:    {val_rec:.6f}")
    print(f"  Val F1:        {val_f1:.6f}")
    
    # Fall_Initiation specific
    fall_metrics = report['Fall_Initiation']
    print(f"\n  Fall_Initiation:")
    print(f"    Precision: {fall_metrics['precision']:.4f}")
    print(f"    Recall:    {fall_metrics['recall']:.4f}")
    print(f"    F1-Score:  {fall_metrics['f1-score']:.4f}")
    
    # Check for overfitting
    train_acc = max(history.history['accuracy'])
    val_acc_best = max(history.history['val_accuracy'])
    gap = train_acc - val_acc_best
    print(f"\n  Overfitting check:")
    print(f"    Best train acc: {train_acc:.4f}")
    print(f"    Best val acc:   {val_acc_best:.4f}")
    print(f"    Gap:            {gap:.4f} ({gap*100:.1f}%)")
    if gap > 0.10:
        print(f"    ⚠️  Still overfitting by >10%")
    elif gap > 0.05:
        print(f"    ⚠️  Moderate overfitting (5-10%)")
    else:
        print(f"    ✓ Good generalization (<5% gap)")

# ============================================================================
# AGGREGATE RESULTS
# ============================================================================

print("\n" + "="*80)
print("LMU-REGULARIZED RESULTS ACROSS ALL FOLDS")
print("="*80)

import pandas as pd
df = pd.DataFrame(results)
print(df[['fold', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_f1']].to_string(index=False))

print("\n" + "="*80)
print("LMU-REGULARIZED AVERAGE PERFORMANCE")
print("="*80)
print(f"Accuracy:  {df['val_accuracy'].mean():.4f} ± {df['val_accuracy'].std():.4f}")
print(f"Precision: {df['val_precision'].mean():.4f} ± {df['val_precision'].std():.4f}")
print(f"Recall:    {df['val_recall'].mean():.4f} ± {df['val_recall'].std():.4f}")
print(f"F1-Score:  {df['val_f1'].mean():.4f} ± {df['val_f1'].std():.4f}")

# Compare to original LMU
print("\n" + "="*80)
print("COMPARISON TO ORIGINAL LMU")
print("="*80)
print(f"Original LMU:     {0.8072:.4f} ± {0.0166:.4f}")
print(f"Regularized LMU:  {df['val_accuracy'].mean():.4f} ± {df['val_accuracy'].std():.4f}")
improvement = df['val_accuracy'].mean() - 0.8072
print(f"Improvement:      {improvement:+.4f} ({improvement*100:+.1f}%)")

# Save results
output = {
    'config': {
        'memory_d': MEMORY_D,
        'order': ORDER,
        'theta': THETA,
        'l1_reg': L1_REG,
        'l2_reg': L2_REG,
        'dropout': DROPOUT,
        'recurrent_dropout': RECURRENT_DROPOUT,
        'early_stop_patience': EARLY_STOP_PATIENCE,
        'reduce_lr_patience': REDUCE_LR_PATIENCE
    },
    'results': results,
    'summary': {
        'mean_accuracy': float(df['val_accuracy'].mean()),
        'std_accuracy': float(df['val_accuracy'].std()),
        'mean_precision': float(df['val_precision'].mean()),
        'std_precision': float(df['val_precision'].std()),
        'mean_recall': float(df['val_recall'].mean()),
        'std_recall': float(df['val_recall'].std()),
        'mean_f1': float(df['val_f1'].mean()),
        'std_f1': float(df['val_f1'].std())
    }
}

output_file = model_dir / 'lmu_regularized_summary.json'
with open(output_file, 'w') as f:
    json.dump(output, f, indent=2)

print(f"\n✓ Results saved to {output_file}")
print("\n" + "="*80)
print("✓ TRAINING COMPLETE")
print("="*80)

In [ ]:
from tensorflow import keras
import numpy as np
from pathlib import Path
from sklearn.metrics import accuracy_score, recall_score, precision_score, classification_report

# Paths
base_dir = Path.home() / 'repos/summerschool2023/projects/fall-detection/fall_detection_data'
model_dir = base_dir / 'models'
processed_dir = base_dir / 'processed'

# Load data
X = np.load(processed_dir / 'X_data_6class.npy')
y = np.load(processed_dir / 'y_labels_6class.npy')

# Load models
cnn_models = [keras.models.load_model(model_dir / f'cnn_only_fold_{i}.keras') for i in range(1,6)]
lmu_models = [keras.models.load_model(model_dir / f'lmu_regularized_fold_{i}.keras') for i in range(1,6)]

# Setup k-fold (same as training)
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

results = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
    print(f"\n{'='*60}")
    print(f"FOLD {fold}/5")
    print(f"{'='*60}")
    
    X_val = X[val_idx]
    y_val = y[val_idx]
    
    # Get predictions
    cnn_pred = cnn_models[fold-1].predict(X_val, verbose=0)
    lmu_pred = lmu_models[fold-1].predict(X_val, verbose=0)
    
    # Individual performance
    cnn_class = np.argmax(cnn_pred, axis=1)
    lmu_class = np.argmax(lmu_pred, axis=1)
    
    # Ensemble (average probabilities)
    ensemble_pred = 0.5 * cnn_pred + 0.5 * lmu_pred
    ensemble_class = np.argmax(ensemble_pred, axis=1)
    
    # Calculate metrics
    cnn_acc = accuracy_score(y_val, cnn_class)
    cnn_rec = recall_score(y_val, cnn_class, average='macro')
    cnn_prec = precision_score(y_val, cnn_class, average='macro')
    
    lmu_acc = accuracy_score(y_val, lmu_class)
    lmu_rec = recall_score(y_val, lmu_class, average='macro')
    lmu_prec = precision_score(y_val, lmu_class, average='macro')
    
    ens_acc = accuracy_score(y_val, ensemble_class)
    ens_rec = recall_score(y_val, ensemble_class, average='macro')
    ens_prec = precision_score(y_val, ensemble_class, average='macro')
    
    # Error analysis
    cnn_errors = (cnn_class != y_val)
    lmu_errors = (lmu_class != y_val)
    ens_errors = (ensemble_class != y_val)
    
    both_wrong = cnn_errors & lmu_errors
    cnn_rescues_lmu = lmu_errors & ~cnn_errors
    lmu_rescues_cnn = cnn_errors & ~lmu_errors
    ensemble_rescues = both_wrong & ~ens_errors
    
    print(f"\nCNN-only:      Acc={cnn_acc:.4f}, Prec={cnn_prec:.4f}, Rec={cnn_rec:.4f}")
    print(f"LMU-only:      Acc={lmu_acc:.4f}, Prec={lmu_prec:.4f}, Rec={lmu_rec:.4f}")
    print(f"CNN-LMU Ens:   Acc={ens_acc:.4f}, Prec={ens_prec:.4f}, Rec={ens_rec:.4f}")
    
    print(f"\nError Analysis:")
    print(f"  CNN rescues LMU: {cnn_rescues_lmu.sum()} cases")
    print(f"  LMU rescues CNN: {lmu_rescues_cnn.sum()} cases")
    print(f"  Both wrong:      {both_wrong.sum()} cases")
    print(f"  Ensemble rescues both: {ensemble_rescues.sum()} cases")
    
    # Gain from ensemble
    gain_vs_cnn = ens_acc - cnn_acc
    gain_vs_lmu = ens_acc - lmu_acc
    print(f"\nEnsemble gain vs CNN: {gain_vs_cnn:+.4f} ({gain_vs_cnn*100:+.1f}%)")
    print(f"Ensemble gain vs LMU: {gain_vs_lmu:+.4f} ({gain_vs_lmu*100:+.1f}%)")
    
    results.append({
        'fold': fold,
        'cnn_acc': cnn_acc,
        'cnn_rec': cnn_rec,
        'lmu_acc': lmu_acc,
        'lmu_rec': lmu_rec,
        'ens_acc': ens_acc,
        'ens_rec': ens_rec,
        'cnn_rescues': int(cnn_rescues_lmu.sum()),
        'lmu_rescues': int(lmu_rescues_cnn.sum())
    })

print(f"\n{'='*60}")
print("SUMMARY ACROSS ALL FOLDS")
print(f"{'='*60}")

import pandas as pd
df = pd.DataFrame(results)

print(f"\nCNN-only:     Acc={df['cnn_acc'].mean():.4f}±{df['cnn_acc'].std():.4f}, Rec={df['cnn_rec'].mean():.4f}±{df['cnn_rec'].std():.4f}")
print(f"LMU-only:     Acc={df['lmu_acc'].mean():.4f}±{df['lmu_acc'].std():.4f}, Rec={df['lmu_rec'].mean():.4f}±{df['lmu_rec'].std():.4f}")
print(f"CNN-LMU Ens:  Acc={df['ens_acc'].mean():.4f}±{df['ens_acc'].std():.4f}, Rec={df['ens_rec'].mean():.4f}±{df['ens_rec'].std():.4f}")

print(f"\nTotal rescues:")
print(f"  CNN rescued LMU: {df['cnn_rescues'].sum()} cases")
print(f"  LMU rescued CNN: {df['lmu_rescues'].sum()} cases")

# Critical question
mean_ens_recall = df['ens_rec'].mean()
if mean_ens_recall >= 0.80:
    print(f"\n✅ ENSEMBLE RECALL ≥ 80% ({mean_ens_recall:.1%}) - VIABLE FOR SNN CONVERSION!")
else:
    print(f"\n⚠️  ENSEMBLE RECALL < 80% ({mean_ens_recall:.1%}) - Consider CNN-LSTM instead")

In [ ]:
from tensorflow import keras
from pathlib import Path

model_dir = Path.home() / 'repos/summerschool2023/projects/fall-detection/fall_detection_data/models'

# Check what the "fallnet" models actually contain
print("Checking fallnet_fold_1.keras:")
model = keras.models.load_model(model_dir / 'fallnet_fold_1.keras')
print("\nModel architecture:")
model.summary()

print("\n" + "="*60)
print("Layer types:")
for layer in model.layers:
    print(f"  {layer.name}: {type(layer).__name__}")
    
# Check if it has LSTM or LMU
has_lstm = any('LSTM' in type(layer).__name__ for layer in model.layers)
has_lmu = any('LMU' in type(layer).__name__ for layer in model.layers)

print("\n" + "="*60)
print(f"Contains LSTM: {has_lstm}")
print(f"Contains LMU: {has_lmu}")

if has_lmu:
    print("\n⚠️  WARNING: fallnet models contain LMU (overwritten!)")
    print("Original CNN-LSTM ensemble was lost")
elif has_lstm:
    print("\n✓ fallnet models still contain LSTM (original preserved)")

In [ ]:
from tensorflow import keras
from pathlib import Path
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, precision_score
import numpy as np
import pandas as pd

# Paths
base_dir = Path.home() / 'repos/summerschool2023/projects/fall-detection/fall_detection_data'
model_dir = base_dir / 'models'
processed_dir = base_dir / 'processed'

# Load data
X = np.load(processed_dir / 'X_data_6class.npy')
y = np.load(processed_dir / 'y_labels_6class.npy')

# Load models - now we have the ensemble models too!
cnn_models = [keras.models.load_model(model_dir / f'cnn_only_fold_{i}.keras') for i in range(1,6)]
lmu_models = [keras.models.load_model(model_dir / f'lmu_regularized_fold_{i}.keras') for i in range(1,6)]
ensemble_models = [keras.models.load_model(model_dir / f'fallnet_fold_{i}.keras') for i in range(1,6)]

# Setup k-fold (same as training)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

results = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
    print(f"\n{'='*60}")
    print(f"FOLD {fold}/5")
    print(f"{'='*60}")
    
    X_val = X[val_idx]
    y_val = y[val_idx]
    
    # Get predictions from all three model types
    cnn_pred = cnn_models[fold-1].predict(X_val, verbose=0)
    lmu_pred = lmu_models[fold-1].predict(X_val, verbose=0)
    ensemble_pred = ensemble_models[fold-1].predict(X_val, verbose=0)
    
    # Convert to class predictions
    cnn_class = np.argmax(cnn_pred, axis=1)
    lmu_class = np.argmax(lmu_pred, axis=1)
    ensemble_class = np.argmax(ensemble_pred, axis=1)
    
    # Calculate metrics
    cnn_acc = accuracy_score(y_val, cnn_class)
    cnn_rec = recall_score(y_val, cnn_class, average='macro')
    cnn_prec = precision_score(y_val, cnn_class, average='macro')
    
    lmu_acc = accuracy_score(y_val, lmu_class)
    lmu_rec = recall_score(y_val, lmu_class, average='macro')
    lmu_prec = precision_score(y_val, lmu_class, average='macro')
    
    ens_acc = accuracy_score(y_val, ensemble_class)
    ens_rec = recall_score(y_val, ensemble_class, average='macro')
    ens_prec = precision_score(y_val, ensemble_class, average='macro')
    
    # Error analysis
    cnn_errors = (cnn_class != y_val)
    lmu_errors = (lmu_class != y_val)
    ens_errors = (ensemble_class != y_val)
    
    both_wrong = cnn_errors & lmu_errors
    cnn_rescues_lmu = lmu_errors & ~cnn_errors
    lmu_rescues_cnn = cnn_errors & ~lmu_errors
    ensemble_rescues = both_wrong & ~ens_errors
    
    print(f"\nCNN-only:          Acc={cnn_acc:.4f}, Prec={cnn_prec:.4f}, Rec={cnn_rec:.4f}")
    print(f"LMU-only:          Acc={lmu_acc:.4f}, Prec={lmu_prec:.4f}, Rec={lmu_rec:.4f}")
    print(f"CNN-LMU Ensemble:  Acc={ens_acc:.4f}, Prec={ens_prec:.4f}, Rec={ens_rec:.4f}")
    
    print(f"\nError Analysis:")
    print(f"  CNN rescues LMU: {cnn_rescues_lmu.sum()} cases")
    print(f"  LMU rescues CNN: {lmu_rescues_cnn.sum()} cases")
    print(f"  Both wrong:      {both_wrong.sum()} cases")
    print(f"  Ensemble rescues both: {ensemble_rescues.sum()} cases")
    
    # Gain from ensemble
    gain_vs_cnn = ens_acc - cnn_acc
    gain_vs_lmu = ens_acc - lmu_acc
    print(f"\nEnsemble gain vs CNN: {gain_vs_cnn:+.4f} ({gain_vs_cnn*100:+.1f}%)")
    print(f"Ensemble gain vs LMU: {gain_vs_lmu:+.4f} ({gain_vs_lmu*100:+.1f}%)")
    
    results.append({
        'fold': fold,
        'cnn_acc': cnn_acc,
        'cnn_rec': cnn_rec,
        'lmu_acc': lmu_acc,
        'lmu_rec': lmu_rec,
        'ens_acc': ens_acc,
        'ens_rec': ens_rec,
        'cnn_rescues': int(cnn_rescues_lmu.sum()),
        'lmu_rescues': int(lmu_rescues_cnn.sum()),
        'ensemble_rescues': int(ensemble_rescues.sum())
    })

print(f"\n{'='*60}")
print("SUMMARY ACROSS ALL FOLDS")
print(f"{'='*60}")

df = pd.DataFrame(results)

print(f"\nCNN-only:          Acc={df['cnn_acc'].mean():.4f}±{df['cnn_acc'].std():.4f}, Rec={df['cnn_rec'].mean():.4f}±{df['cnn_rec'].std():.4f}")
print(f"LMU-only:          Acc={df['lmu_acc'].mean():.4f}±{df['lmu_acc'].std():.4f}, Rec={df['lmu_rec'].mean():.4f}±{df['lmu_rec'].std():.4f}")
print(f"CNN-LMU Ensemble:  Acc={df['ens_acc'].mean():.4f}±{df['ens_acc'].std():.4f}, Rec={df['ens_rec'].mean():.4f}±{df['ens_rec'].std():.4f}")

print(f"\nTotal rescues:")
print(f"  CNN rescued LMU: {df['cnn_rescues'].sum()} cases")
print(f"  LMU rescued CNN: {df['lmu_rescues'].sum()} cases")
print(f"  Ensemble rescued both: {df['ensemble_rescues'].sum()} cases")

# Compare to your manual ensemble (averaging predictions)
manual_ensemble_acc = 0.9000  # From your previous analysis
trained_ensemble_acc = df['ens_acc'].mean()

print(f"\n{'='*60}")
print("ENSEMBLE COMPARISON")
print(f"{'='*60}")
print(f"Manual averaging:  {manual_ensemble_acc:.4f} (averaging CNN + LMU predictions)")
print(f"Trained ensemble:  {trained_ensemble_acc:.4f} (CNN-LMU model with learned weights)")
print(f"Difference:        {(trained_ensemble_acc - manual_ensemble_acc):+.4f}")

# Critical question
mean_ens_recall = df['ens_rec'].mean()
if mean_ens_recall >= 0.80:
    print(f"\n✅ ENSEMBLE RECALL ≥ 80% ({mean_ens_recall:.1%}) - VIABLE FOR SNN CONVERSION!")
else:
    print(f"\n⚠️  ENSEMBLE RECALL < 80% ({mean_ens_recall:.1%}) - Consider CNN-LSTM instead")

# Detailed table
print(f"\n{'='*60}")
print("DETAILED RESULTS BY FOLD")
print(f"{'='*60}")
print(df[['fold', 'cnn_acc', 'lmu_acc', 'ens_acc', 'cnn_rec', 'lmu_rec', 'ens_rec']].to_string(index=False))